### Capstone  Project
By Mónica Atiaga

Batch - DSNB1222

<h1>SHOE BRAND CAPSTONE PROJECT</h1>


### Phase 1                                                                                                                                            
#### Web scrapping the below given six URL :
(21 marks)
- https://www.nike.com
- https://www.adidas.com
- https://us.puma.com/us/en
- https://www.skechers.com
- https://www.columbia.com/
- https://www.woodlandworldwide.com/

Make the table in  csv format  (3 csv files)  
1. Table1.csv  
    - ShoeName ,Category(Men/Women), no.of colors,price  
2. Table2.csv
    - count of sizes, colors 1, color 2, color3 ,color4, color5, Style code/product code  
3. Table3.csv 
    - reviews, Size, comfort, durability /quality/performance(quantification), star rating,#

In [1]:
# Importing libraries 
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, ElementClickInterceptedException


from bs4 import BeautifulSoup
import time
import regex as re
import ast


#### My Custom Functions (at the end section)


* get_attribute_el(webelement, attribute)
* text_el(webelement)
* remove_popup(xpath)
* n_shoes_to_scrap(xpath, pattern = r'(\d+)' )
* follow_path_to(config_path)
* scroll_to_recovered_shoes(n_shoes, xpath)
* load_items_by_xpath(driver, n=None, **kwargs)
* scrape_items_by_group(driver, xpath_grp, **kwargs)
* bs_scrape_item_detail(html_string, **kwargs)
* scrape_details_by_URL_in_batches(df_urls, soup_config, **kwargs)

All functions used are my own. As we have progressed during the PGP in Data Science, I have been improving the code so that they are more general and can be applied in different scenarios. I decided to do it this way to improve and progress my knowledge of Python.
Probably, find some familiarity with the functions I developed during my internship so here are the repositories in reference. https://github.com/matiaga/Internship

_By Monica Atiaga_ 

In [310]:
# Define the sites to scrap
sites = {
'nike': 'https://www.nike.com',
'adidas': 'https://www.adidas.com/us',
'puma': 'https://us.puma.com/us/en',
'skechers': 'https://www.skechers.com',
'columbia': 'https://www.columbia.com/',
'woodlww': 'https://www.woodlandworldwide.com/',
}

## 1. Nike.com

In [15]:
%%time

# Initialization of the dataframe
df_shoes = pd.DataFrame()

# Scrap the main data for Men and Women in nike.com
for category in ['Women','Men']:

    # Open the Driver 
    driver = webdriver.Chrome()

    url = sites['nike']
    driver.maximize_window()
    driver.get(url) 

    # Going to United State Market
    path_nike = {'msg': ['United State market...', f'\n{category} Category...', 'Looking for Shoes...\n'],
                 'xpaths': ['/html/body/div[5]/div/div/nav/div/div/div[2]/div/a[9]/div/div[2]/p',
                            f'//a[@aria-label="{category}"]', 
                             '//a[@aria-label="Shoes"]'],
                'wait_time': [ 5, 5, 8]}

    follow_path_to(path_nike) 
    
    # Find out the number of shoes to retrieve in the category
    pattern = r'\((\d+)\)' # Remove the parentheses
    n_shoes = n_shoes_to_scrap('//span[@class="wall-header__item_count"]', pattern)  
    print(f"Shoes to recover: {n_shoes}\n" )

    # Define the xpath for shoes card 
    xpath = '//div[@class="product-card product-grid__card  css-1t0asop"]'

    # Scroll until recovered all shoes
    recovered_shoes, _ = scroll_to_recovered_shoes(n_shoes, xpath)

    print(f"\nTotal of shoes recovered: {recovered_shoes}\n")

    print('*'*50)
    print(' '*5, f'S C R A P I N G - Category: {category}')
    print('*'*50)

    # Scraping ShoeName ,Category(Men/Women), no.of colors,price
    attributes = {'ShoeName': ['//div[@class="product-card__title"]'],
                  'URL': ['//a[@data-testid="product-card__link-overlay"]', fn_href],
                  'no_colors': ['//div[@class="product-card__product-count"]'],
                  'price_reduce': ['//div[@data-testid="product-card__price"]'],
                  'price': ['//div[@data-testid="product-price"]']}

    df_shoes_cat = load_items_by_xpath(driver, None, **attributes) 

    # Free driver
    driver.quit()
    
    # Setting the category
    df_shoes_cat['Category'] = category

    # Append the new data 
    df_shoes = pd.concat([df_shoes, df_shoes_cat])
    
    print( f'Products in category {category} retrieved successfully: {len(df_shoes)}' )


Wait time ** 5 **
United State market...
Wait time ** 5 **

Women Category...
Wait time ** 8 **
Looking for Shoes...

Shoes to recover: 653

Recovered shoes: 24
Scrolling 1 times...
Recovered shoes: 72, still to recovered: 581
Scrolling 2 times...
Recovered shoes: 120, still to recovered: 533
Scrolling 3 times...
Recovered shoes: 168, still to recovered: 485
Scrolling 4 times...
Recovered shoes: 216, still to recovered: 437
Scrolling 5 times...
Recovered shoes: 264, still to recovered: 389
Scrolling 6 times...
Recovered shoes: 312, still to recovered: 341
Scrolling 7 times...
Recovered shoes: 360, still to recovered: 293
Scrolling 8 times...
Recovered shoes: 408, still to recovered: 245
Scrolling 9 times...
Recovered shoes: 456, still to recovered: 197
Scrolling 10 times...
Recovered shoes: 504, still to recovered: 149
Scrolling 11 times...
Recovered shoes: 552, still to recovered: 101
Scrolling 12 times...
Recovered shoes: 600, still to recovered: 53
Scrolling 13 times...
Recovered sh

In [17]:
# Backup dataframe detail brand Nike
df_nike_backup = df_shoes.copy()

# Save a file with the original data recovered
df_nike_backup.to_csv('datasets/raw_nike.csv')

In [20]:
# Cleaning the data frame
df_shoes['no_colors'] = df_shoes['no_colors'].str.extract(r'(\d+)')
df_shoes['price_reduce'] = df_shoes['price_reduce'].str.extract(r'\$(\d+(\.\d+)?)')[0]
df_shoes['price'] = df_shoes['price'].str.extract(r'\$(\d+(\.\d+)?)')[0]

# Setting up the dataframe
df_shoes['Brand'] = 'Nike'
df_shoes.reset_index(drop=True, inplace=True)

In [22]:
# Saving the data frame
df_shoes.to_csv('datasets/Table1_nike.csv')
display(df_shoes)

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand
0,,https://www.nike.com/t/dunk-low-womens-shoes-4...,1,115,115,Women,Nike
1,Nike Gamma Force,https://www.nike.com/t/gamma-force-womens-shoe...,9,72.97,90,Women,Nike
2,Nike Court Vision Low Next Nature,https://www.nike.com/t/court-vision-low-next-n...,6,60.97,80,Women,Nike
3,Nike Air Max 270,https://www.nike.com/t/air-max-270-womens-shoe...,12,120.97,160,Women,Nike
4,Nike Air Max 1,https://www.nike.com/t/air-max-1-womens-shoes-...,6,140,140,Women,Nike
...,...,...,...,...,...,...,...
1670,Nike Offcourt (MLB Baltimore Orioles),https://www.nike.com/t/offcourt-baltimore-orio...,1,40,40,Men,Nike
1671,Nike Offcourt (MLB Houston Astros),https://www.nike.com/t/offcourt-houston-astros...,1,40,40,Men,Nike
1672,Nike Offcourt (MLB Seattle Mariners),https://www.nike.com/t/offcourt-seattle-marine...,1,40,40,Men,Nike
1673,Nike Offcourt (MLB Cleveland),https://www.nike.com/t/offcourt-cleveland-slid...,1,40,40,Men,Nike


In [24]:
# Nike shoes with more than 5 colors availables
df_shoes[df_shoes['no_colors'].astype(int) > 5]['URL']

1       https://www.nike.com/t/gamma-force-womens-shoe...
2       https://www.nike.com/t/court-vision-low-next-n...
3       https://www.nike.com/t/air-max-270-womens-shoe...
4       https://www.nike.com/t/air-max-1-womens-shoes-...
5       https://www.nike.com/t/free-metcon-5-womens-wo...
                              ...                        
1369    https://www.nike.com/t/mercurial-vapor-15-acad...
1612    https://www.nike.com/t/pegasus-40-tampa-bay-bu...
1613    https://www.nike.com/t/offcourt-atlanta-falcon...
1614    https://www.nike.com/t/pegasus-40-buffalo-bill...
1648    https://www.nike.com/t/offcourt-philadelphia-p...
Name: URL, Length: 270, dtype: object

###  1.1 Nike details

We will recovered the data from the Table1 records.

In [4]:
df_shoes_nike = pd.read_csv('datasets/Table1_nike.csv', index_col=0)
df_shoes_nike

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand,product_code
0,NaN,https://www.nike.com/t/dunk-low-womens-shoes-4...,1,115.00,115,Women,Nike,DD1503-101
1,Nike Gamma Force,https://www.nike.com/t/gamma-force-womens-shoe...,9,72.97,90,Women,Nike,DX9176-101
2,Nike Court Vision Low Next Nature,https://www.nike.com/t/court-vision-low-next-n...,6,60.97,80,Women,Nike,DH3158-107
3,Nike Air Max 270,https://www.nike.com/t/air-max-270-womens-shoe...,12,120.97,160,Women,Nike,DZ7736-002
4,Nike Air Max 1,https://www.nike.com/t/air-max-1-womens-shoes-...,6,140.00,140,Women,Nike,DZ2628-102
...,...,...,...,...,...,...,...,...
1670,Nike Offcourt (MLB Baltimore Orioles),https://www.nike.com/t/offcourt-baltimore-orio...,1,40.00,40,Men,Nike,NaN
1671,Nike Offcourt (MLB Houston Astros),https://www.nike.com/t/offcourt-houston-astros...,1,40.00,40,Men,Nike,NaN
1672,Nike Offcourt (MLB Seattle Mariners),https://www.nike.com/t/offcourt-seattle-marine...,1,40.00,40,Men,Nike,NaN
1673,Nike Offcourt (MLB Cleveland),https://www.nike.com/t/offcourt-cleveland-slid...,1,40.00,40,Men,Nike,NaN


We will recover the data in batches to avoid the site blocking us from scraping the data. It is possible to scrape the data with an algorithm that automates the process but considering the amount of the data is safer to recover the data in batches of 100 records each time. This way if some error out of our control happened the process could be continued later without losing the progress.

#### Configuring the main parameters

In [140]:
# Defines the configuration to retrieve the data
config = {
    'xpath_show_reviews': '//details[@data-test="reviewsAccordionClick"]',
    'xpath_more_reviews': '//button[@data-test="more-reviews"]',
    'xpath_market_esc' : '//button[@aria-label="Close Menu" and @data-var="closeNavBtn"]',
    'n_scrolls': 3,
    'height_scroll': 400,
    'wait_time': 4,
}
# Defines the attributes to recover with all the special configuration according to the property.
soup_config = {
            'count_of_sizes':['input', {'name': 'skuAndSize'}, 
                              lambda el: el.get('value', '-'),
                              lambda el: 'disabled' not in el.attrs,  '-'], 
            'colors': ['label', {"data-colorway-id": re.compile(r'^Colorway-')}, 
                       lambda el: el.find('img')['alt'] if el.find('img') is not None else '-', 
                       None, '-'],
            'color': ['li',{'class':"description-preview__color-description ncss-li"}, 
                       None, 
                       lambda el: el.text.strip() and 'Shown:' in el.text.strip() , '-'],
            'product_code': ['li', {'class':"description-preview__style-color ncss-li"}, 
                       None, 
                       lambda el: el.text.strip() and 'Style:' in el.text.strip() , '-'],
            'reviews': ['span', {}, 
                        None, 
                        lambda el: el.text.strip() and 'Reviews ' in el.text.strip(), '-'],
            'star_rating':['div',{'class':'css-n209rx'}, 
                       lambda el: el.get('aria-label', '-'), 
                       None ,'-'],
            'Size': ['div', {'class':"tt-u-clip-hide"}, 
                     None, 
                     lambda el: 'between Runs Small and True to Size' in el.text.strip(), '-'],
            'comfort': ['div', {'class':"tt-u-clip-hide"}, 
                     None, 
                     lambda el: 'between Uncomfortable and Average' in el.text.strip(), '-'],
            'quality': ['div',{'class':"tt-c-summary-dim__text tt-u-align-center tt-u-spacing--left--xs"},
                       None,
                       None, '-']
        }

Since quality is not evaluated on the Nike site specifically, we will calculate the percentage of recommendations using the yes recommendations versus the total reviews. Then, we will average this percentage with size and comfort valuations.

In [108]:
def clean_df_det_nike(df):
    
    df_det = df.copy()
    # Cleaning data frame
    # Obtain the color to use when the shoe model have only one color
    df_det['color'] = df_det['color'].dropna().apply(lambda x: re.search(r"Shown: (.*?)'", str(x)).group(1) if re.search(r"Shown: (.*?)'", str(x)) else None)

    # Transform in list some columns
    # for col in ['count_of_sizes', 'colors']:
    #     df_det[col] = df_det[col].apply(lambda x: ast.literal_eval(x))

    # Joins the column 'colors' and 'color' in case there is only one color for the shoe model
    df_det['colors'] = df_det.apply(lambda row: row['colors'] if row['colors']  else [row['color']], axis=1)

    df_det['count_of_sizes'] = df_det['count_of_sizes'].apply(len)

    temp = df_det['product_code'].dropna().apply(lambda x: re.findall(r'Style: (\w+-\d+)', str(x)))
    df_det['product_code'] = temp.apply(lambda x: x[0] if x else None)

    temp = df_det['reviews'].dropna().apply(lambda x: re.findall(r'Reviews\D*(\d+)\D*', str(x)))
    df_det['n_reviews'] = temp.apply(lambda x: x[0] if x else None)

    temp = df_det['star_rating'].dropna().apply(lambda x: re.findall(r'(\d+(\.\d*)?)', str(x)))
    df_det['star_rating'] = temp.apply(lambda x: x[0][0] if x else None)

    temp = df_det['Size'].dropna().apply(lambda x: re.findall(r'(\d+.*\d+)%\D*', str(x)))
    df_det['Size'] = temp.apply(lambda x: x[0] if x else None)

    temp = df_det['comfort'].dropna().apply(lambda x: re.findall(r'(\d+.*\d+)%\D*', str(x)))
    df_det['comfort'] = temp.apply(lambda x: x[0] if x else None)

    # Since quality is not evaluated on the Nike site specifically, 
    # we will calculate the percentage of recommendations using the yes recommendations versus the total reviews. 
    temp = df_det['quality'].dropna().apply(lambda x: re.findall(r'Yes\D*(\d+)\D*', str(x)))
    df_det['yes_recommendations'] = temp.apply(lambda x: x[0] if x else None)

    # Transform numeric columns:
    for col in ['Size', 'comfort', 'n_reviews', 'yes_recommendations']:
        df_det[col] = pd.to_numeric(df_det[col], errors='coerce')

    # Here, we will average this percentage with size and comfort valuations. 
    df_det['%_recommended'] = round(df_det['yes_recommendations']/df_det['n_reviews']*100, 2)
    df_det['quality'] = round(df_det[['Size', 'comfort', '%_recommended']].mean(axis=1), 2)
    
    return df_det

#### First Batch - Nike

In [6]:
df_shoes_nike.shape

(1675, 8)

In [18]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][:100]

# Scrape the first 100 records
df_det, url_error = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/100
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/dunk-low-womens-shoes-4W2Z5P/DD1503-101

Retrieving Details of record: 2/100
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/gamma-force-womens-shoes-4TR9Xf/DX9176-101

Retrieving Details of record: 3/100
Show reviews of 3
Click more reviews of 3
Get the HTML code: https://www.nike.com/t/court-vision-low-next-nature-womens-shoes-ZkMMBG/DH3158-107

Retrieving Details of record: 4/100
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/air-max-270-womens-shoes-Pgb94t/DZ7736-002

Retrieving Details of record: 5/100
Show reviews of 5
Click more reviews of 5
Get the HTML code: https://www.nike.com/t/air-max-1-womens-shoes-0vW0ds/DZ2628-102

Retrieving Details of record: 6/100
Show reviews of 6
Click more reviews of 6
Get the HTML code: https://www.nike.com/t/free-metcon-5-womens-workout-shoes-h4Zl5h/DV3950

Retrieving Details of record: 48/100
Show reviews of 48
More-Review Exception of 47
Get the HTML code: https://www.nike.com/t/blazer-low-platform-womens-shoes-X8Nl4M/DJ0292-101

Retrieving Details of record: 49/100
Show reviews of 49
Click more reviews of 49
Get the HTML code: https://www.nike.com/t/cortez-womens-shoes-tZtTd5/DN1791-100

Retrieving Details of record: 50/100
Show reviews of 50
Click more reviews of 50
Get the HTML code: https://www.nike.com/t/kiger-9-womens-trail-running-shoes-bsbcr4/DR2694-500

Retrieving Details of record: 51/100
Show reviews of 51
Click more reviews of 51
Get the HTML code: https://www.nike.com/t/wildhorse-8-womens-trail-running-shoes-TlD93l/DR2689-601

Retrieving Details of record: 52/100
Show reviews of 52
Click more reviews of 52
Get the HTML code: https://www.nike.com/t/juniper-trail-2-gore-tex-womens-waterproof-trail-running-shoes-XmqHnc/FB2065-500

Retrieving Details of record: 53/100
Show reviews of 53
Click more reviews of 53
Get the HTML cod

More-Review Exception of 86
Get the HTML code: https://www.nike.com/t/free-rn-2018-womens-running-shoes-zE8Je3/942837-001

Retrieving Details of record: 88/100
Show reviews of 88
Click more reviews of 88
Get the HTML code: https://www.nike.com/t/air-jordan-1-mid-womens-shoes-BVgBmq/DV0991-101

Retrieving Details of record: 89/100
Show reviews of 89
Click more reviews of 89
Get the HTML code: https://www.nike.com/t/court-royale-2-womens-shoe-pgRlMB/DH3159-100

Retrieving Details of record: 90/100
Show reviews of 90
Click more reviews of 90
Get the HTML code: https://www.nike.com/t/air-jordan-1-mid-se-womens-shoes-0k55l8/FB9892-670

Retrieving Details of record: 91/100
Show reviews of 91
Click more reviews of 91
Get the HTML code: https://www.nike.com/t/calm-mens-mules-m5tTKn/FD5130-200

Retrieving Details of record: 92/100
Show reviews of 92
Click more reviews of 92
Get the HTML code: https://www.nike.com/t/air-jordan-1-low-mens-shoes-0LXhbn/553558-136

Retrieving Details of record: 93/

In [21]:
# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')

In [32]:
display(url_error)
# Save a file with the url to review 
url_error.to_csv('errors/nike_batch1_err.csv')

,index,URL,Exception,msg
0,47,https://www.nike.com/t/blazer-low-platform-wom...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
1,75,https://www.nike.com/t/air-jordan-1-mid-se-wom...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
2,86,https://www.nike.com/t/free-rn-2018-womens-run...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
3,86,https://www.nike.com/t/free-rn-2018-womens-run...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...


#### 2nd Batch - Nike

In [33]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][100:200]

# Scrape from 100 to 200 records
df_det2, url_error2 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/100
Show reviews of 1
More-Review Exception of 100
Get the HTML code: https://www.nike.com/t/air-jordan-1-retro-high-og-womens-shoes-SsFGW3/FN6622-201

Retrieving Details of record: 2/100
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/air-jordan-13-wheat-mens-shoes-zp4pp0/414571-171

Retrieving Details of record: 3/100
Show reviews of 3
Click more reviews of 3
Get the HTML code: https://www.nike.com/t/air-jordan-8-winterized-mens-shoes-HBQzn3/FD1334-001

Retrieving Details of record: 4/100
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/air-jordan-1-element-shoes-Kcsdzl/DB2889-501

Retrieving Details of record: 5/100
Show reviews of 5
Click more reviews of 5
Get the HTML code: https://www.nike.com/t/air-jordan-3-fear-mens-shoes-5M3ZlK/CT8532-080

Retrieving Details of record: 6/100
Item Dunk High Warped Unlocked By You Greatest Gift has not reviews.
Item Dunk High Warped Unlocked By

Click more reviews of 44
Get the HTML code: https://www.nike.com/t/jumpman-two-trey-womens-shoes-VKM3mw/DR9631-801

Retrieving Details of record: 45/100
Selecting the market was not necessary.
Item Nike Air Max 1 '87 Premium has not reviews.
Item Nike Air Max 1 '87 Premium has not reviews.
Get the HTML code: https://www.nike.com/launch/r/FN7200-224

Retrieving Details of record: 46/100
Show reviews of 46
Click more reviews of 46
Get the HTML code: https://www.nike.com/t/nikecourt-air-zoom-nxt-womens-hard-court-tennis-shoes-j36m8b/DV3282-103

Retrieving Details of record: 47/100
Show reviews of 47
Click more reviews of 47
Get the HTML code: https://www.nike.com/t/elevate-2-basketball-shoes-v0S9N8/CW3406-004

Retrieving Details of record: 48/100
Show reviews of 48
Click more reviews of 48
Get the HTML code: https://www.nike.com/t/air-jordan-1-low-flyease-mens-easy-on-off-shoes-D3wW7l/DM1206-174

Retrieving Details of record: 49/100
Show reviews of 49
Click more reviews of 49
Get the HTML

More-Review Exception of 181
Get the HTML code: https://www.nike.com/t/waffle-one-womens-shoes-qk5cz3/DC2533-600

Retrieving Details of record: 83/100
Show reviews of 83
Click more reviews of 83
Get the HTML code: https://www.nike.com/t/hyperdiamond-4-keystone-womens-softball-cleats-6gFh7z/CZ5918-012

Retrieving Details of record: 84/100
Show reviews of 84
Item Nike Air Max 97 Next Nature has not reviews.
Get the HTML code: https://www.nike.com/t/air-max-97-next-nature-womens-shoes-Fr6rM4/FN7104-010

Retrieving Details of record: 85/100
Show reviews of 85
Click more reviews of 85
Get the HTML code: https://www.nike.com/t/react-infinity-3-womens-road-running-shoes-XpNmlR/DD3024-101

Retrieving Details of record: 86/100
Show reviews of 86
Item Nike Cortez 23 Premium has not reviews.
Get the HTML code: https://www.nike.com/t/cortez-23-premium-womens-shoes-0FZSGV/FB6877-100

Retrieving Details of record: 87/100
Show reviews of 87
Click more reviews of 87
Get the HTML code: https://www.nike

In [34]:
url_error2

,index,URL,Exception,msg
0,100,https://www.nike.com/t/air-jordan-1-retro-high...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
1,121,https://www.nike.com/t/ultrafly-womens-trail-r...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
2,141,https://www.nike.com/t/blazer-low-77-jumbo-wom...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
3,169,https://www.nike.com/t/nikecourt-air-zoom-pro-...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
4,181,https://www.nike.com/t/waffle-one-womens-shoes...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
5,181,https://www.nike.com/t/waffle-one-womens-shoes...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...


In [55]:
# Join the batches
df_det = pd.concat([df_det, df_det2])
url_error = pd.concat([url_error, url_error2])

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 3rd Batch - Nike

In [19]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][200:300]

# Scrape from 200 to 300 records
df_det3, url_error3 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/100
Wait time ** 5 ** 
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/air-jordan-1-high-method-of-make-womens-shoes-hCCcBh/FB9891-100

Retrieving Details of record: 2/100
Wait time ** 5 ** 
Item Nike Dunk High By You has not reviews.
Item Nike Dunk High By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-dunk-high-by-you-shoes-10001379/9626270046

Retrieving Details of record: 3/100
Wait time ** 5 ** 
Item Nike Air Force 1 Mid By You has not reviews.
Item Nike Air Force 1 Mid By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-air-force-1-mid-by-you-shoes-10001367/7186321658

Retrieving Details of record: 4/100
Wait time ** 5 ** 
Item Nike Metcon 9 By You has not reviews.
Item Nike Metcon 9 By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-metcon-9-by-you-10001601/2497842746

Retrieving Details of record: 5/100
Wait time ** 5 ** 
Item Nike

Retrieving Details of record: 41/100
Wait time ** 5 ** 
Show reviews of 41
Click more reviews of 41
Get the HTML code: https://www.nike.com/t/air-max-pre-day-se-womens-shoes-TNgd9M/DX2312-200

Retrieving Details of record: 42/100
Wait time ** 5 ** 
Show reviews of 42
Click more reviews of 42
Get the HTML code: https://www.nike.com/t/savaleos-weightlifting-shoes-zvPDl7/CV5708-010

Retrieving Details of record: 43/100
Wait time ** 5 ** 
Show reviews of 43
Click more reviews of 43
Get the HTML code: https://www.nike.com/t/legend-essential-2-premium-womens-training-shoes-pvvmXq/CZ3668-017

Retrieving Details of record: 44/100
Wait time ** 5 ** 
Show reviews of 44
More-Review Exception of 243
Get the HTML code: https://www.nike.com/t/ryz-365-2-womens-shoes-LkNlGp/CU4874-600

Retrieving Details of record: 45/100
Wait time ** 5 ** 
Review Exception of 244-Message: element click intercepted: Element <details data-nr-comp="details" data-nr-animatable="false" data-nr-animation="false" class=" cs

Click more reviews of 68
Get the HTML code: https://www.nike.com/t/downshifter-12-womens-road-running-shoes-ZJlqCr/DD9294-104

Retrieving Details of record: 69/100
Wait time ** 5 ** 
Show reviews of 69
Click more reviews of 69
Get the HTML code: https://www.nike.com/t/air-max-flyknit-racer-womens-shoes-m3w0cV/DM9073-700

Retrieving Details of record: 70/100
Wait time ** 5 ** 
Show reviews of 70
Click more reviews of 70
Get the HTML code: https://www.nike.com/t/cosmic-unity-2-basketball-shoes-L53XJP/DH1537-002

Retrieving Details of record: 71/100
Wait time ** 5 ** 
Show reviews of 71
Click more reviews of 71
Get the HTML code: https://www.nike.com/t/dbreak-premium-womens-shoes-61jl9b/DR5377-001

Retrieving Details of record: 72/100
Wait time ** 5 ** 
Show reviews of 72
Click more reviews of 72
Get the HTML code: https://www.nike.com/t/air-zoom-lj-elite-track-field-jumping-spikes-pSDffB/CT0079-101

Retrieving Details of record: 73/100
Wait time ** 5 ** 
Item Nike Dunk High Unlocked By Y

In [31]:
url_error3

,index,URL,Exception,msg
0,208,https://www.nike.com/t/blazer-mid-77-vintage-w...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
1,219,https://www.nike.com/t/streakfly-mens-road-rac...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
2,227,https://www.nike.com/t/air-adjust-force-2023-w...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
3,243,https://www.nike.com/t/ryz-365-2-womens-shoes-...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
4,244,https://www.nike.com/t/chuck-taylor-all-star-l...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
5,244,https://www.nike.com/t/chuck-taylor-all-star-l...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
6,255,https://www.nike.com/t/rival-jump-track-field-...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
7,255,https://www.nike.com/t/rival-jump-track-field-...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...


In [41]:
# Join the batches
df_det = pd.concat([df_det, df_det3])
url_error = pd.concat([url_error, url_error3])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 4th Batch - Nike

In [48]:
df_shoes_nike.iloc[379]

ShoeName                                         Sabrina 1 By You
URL             https://www.nike.com/u/custom-sabrina-1-by-you...
no_colors                                                       1
price_reduce                                                160.0
price                                                         160
Category                                                    Women
Brand                                                        Nike
product_code                                                  NaN
Name: 379, dtype: object

In [52]:
print(f"The product with URL: {df_shoes_nike.iloc[379]['URL']} it is not longer available.")

# Drop the record 379
df_shoes_nike = df_shoes_nike.drop(379)

# Saving the data frame
df_shoes_nike.to_csv('datasets/Table1_nike.csv')
print(f"Table 1 for Nike was updated. Records: {df_shoes_nike.shape}")

The product with URL: https://www.nike.com/u/custom-sabrina-1-by-you-10001746/5615997567 it is not longer available.
Table 1 for Nike was updated. Records: (1674, 8)


In [53]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][300:400]

# Scrape from 300 to 400 records
df_det4, url_error4 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/100
Wait time ** 5 ** 
Show reviews of 1
Item Nike Air Force 1 '07 Mid has not reviews.
Get the HTML code: https://www.nike.com/t/air-force-1-07-mid-womens-shoes-2NP34w/DZ5211-100

Retrieving Details of record: 2/100
Wait time ** 5 ** 
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/superrep-cycle-2-next-nature-womens-cycling-shoes-cqGbFr/DH3395-102

Retrieving Details of record: 3/100
Wait time ** 5 ** 
Item Nike Dunk Low By Kamryn has not reviews.
Item Nike Dunk Low By Kamryn has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-dunk-low-by-you-shoes-10001709/5851778339

Retrieving Details of record: 4/100
Wait time ** 5 ** 
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/acg-mountain-fly-low-se-shoes-rN2cSR/DQ1979-500

Retrieving Details of record: 5/100
Wait time ** 5 ** 
Show reviews of 5
Click more reviews of 5
Get the HTML code: https://www.nike.com/t/bella-ka

Show reviews of 38
Click more reviews of 38
Get the HTML code: https://www.nike.com/t/air-jordan-5-navy-mens-shoes-Qw4dtb/FD6812-400

Retrieving Details of record: 39/100
Wait time ** 5 ** 
Show reviews of 39
Click more reviews of 39
Get the HTML code: https://www.nike.com/t/hyperdiamond-4-pro-mcs-womens-softball-cleats-cWdWB2/DC8990-002

Retrieving Details of record: 40/100
Wait time ** 5 ** 
Show reviews of 40
Click more reviews of 40
Get the HTML code: https://www.nike.com/t/air-force-1-07-mid-womens-shoe-6ch4BD/DD9625-400

Retrieving Details of record: 41/100
Wait time ** 5 ** 
Show reviews of 41
Item Nike Revolution 7 has not reviews.
Get the HTML code: https://www.nike.com/t/revolution-7-womens-road-running-shoes-wide-m4WCT6/FZ6829-002

Retrieving Details of record: 42/100
Wait time ** 5 ** 
Show reviews of 42
Click more reviews of 42
Get the HTML code: https://www.nike.com/t/jordan-hex-mule-womens-shoes-4Lm6XS/DX6405-600

Retrieving Details of record: 43/100
Selecting the market

Retrieving Details of record: 65/100
Wait time ** 5 ** 
Show reviews of 65
Item Nike Air Huarache has not reviews.
Get the HTML code: https://www.nike.com/t/air-huarache-womens-shoes-GZpbBG/DV2220-100

Retrieving Details of record: 66/100
Wait time ** 5 ** 
Item Nike Cortez Platform Unlocked By You has not reviews.
Item Nike Cortez Platform Unlocked By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-cortez-platform-unlocked-by-you-10001611/4868453426

Retrieving Details of record: 67/100
Wait time ** 5 ** 
Show reviews of 67
Item Nike Alphafly 3 Proto has not reviews.
Get the HTML code: https://www.nike.com/t/alphafly-3-proto-womens-road-racing-shoes-rjl2qk/FD8357-100

Retrieving Details of record: 68/100
Wait time ** 5 ** 
Show reviews of 68
Click more reviews of 68
Get the HTML code: https://www.nike.com/t/alpha-huarache-elite-4-turf-womens-softball-shoes-253w2V/DV0496-006

Retrieving Details of record: 69/100
Wait time ** 5 ** 
Show reviews of 69
Click more rev

More-Review Exception of 388
Get the HTML code: https://www.nike.com/t/air-jordan-1-low-womens-shoes-Bz4GT4/DC0774-102

Retrieving Details of record: 89/100
Wait time ** 5 ** 
Show reviews of 89
Click more reviews of 89
Get the HTML code: https://www.nike.com/t/woodside-chukka-2-womens-boots-lZ4o48/537345-001

Retrieving Details of record: 90/100
Wait time ** 5 ** 
Show reviews of 90
Click more reviews of 90
Get the HTML code: https://www.nike.com/t/zoom-javelin-elite-3-track-field-throwing-spikes-sq6fM6/AJ8119-002

Retrieving Details of record: 91/100
Wait time ** 5 ** 
Show reviews of 91
Click more reviews of 91
Get the HTML code: https://www.nike.com/t/cosmic-unity-3-aja-wilson-womens-basketball-shoes-C0NgDW/FQ9395-600

Retrieving Details of record: 92/100
Wait time ** 5 ** 
Show reviews of 92
Click more reviews of 92
Get the HTML code: https://www.nike.com/t/ryz-365-womens-shoes-DRgdX7/BQ4153-100

Retrieving Details of record: 93/100
Wait time ** 5 ** 
Show reviews of 93
Item Nike 

In [54]:
url_error4

,index,URL,Exception,msg
0,346,https://www.nike.com/t/chuck-taylor-all-star-l...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
1,346,https://www.nike.com/t/chuck-taylor-all-star-l...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
2,356,https://www.nike.com/t/jumpman-mvp-womens-shoe...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
3,356,https://www.nike.com/t/jumpman-mvp-womens-shoe...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
4,383,https://www.nike.com/t/wmns-initiator-qqTA0Yrv...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
5,384,https://www.nike.com/t/freak-4-birthstone-bask...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
6,384,https://www.nike.com/t/freak-4-birthstone-bask...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button typ...
7,388,https://www.nike.com/t/air-jordan-1-low-womens...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
8,388,https://www.nike.com/t/air-jordan-1-low-womens...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...


##### Recovered previous gathered data 

In [56]:
df_det, url_error = recover_det_raw_data('datasets/det_raw_nike.csv', 'errors/nike_batch_err.csv' )
display(df_det.tail(2))

Data Recovered: (300, 9) from datasets/det_raw_nike.csv


,count_of_sizes,colors,color,product_code,reviews,star_rating,Size,comfort,quality
298,"[28976171:5, 28976170:6, 28976175:9, 28976165:...","[Sky J Purple/Sky J Purple, Light Silver/Light...",[Shown: Light Silver/Light Silver],[Style: FD2652-002],[Reviews (1)],[5],[100% between Runs Small and True to Size],[100% between Uncomfortable and Average],[Yes (1)]
299,"[29809179:5, 29809185:5.5, 29809176:6, 2980918...","[Black/Metallic Black/Black, Phantom/Smokey Ma...",[Shown: Black/Metallic Black/Black],[Style: FN0784-004],[Reviews (37)],[4.6],[49% between Runs Small and True to Size],[96% between Uncomfortable and Average],[Yes (8)No (2)]


In [58]:
# Join the batches
df_det = pd.concat([df_det, df_det4])
url_error = pd.concat([url_error, url_error4])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 5th Batch - Nike

In [70]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][400:550]

# Scrape from 400 to 550 records
df_det4, url_error4 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/150
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/pegasus-trail-3-gore-tex-womens-waterproof-trail-running-shoes-NqQQNS/DC8794-002

Retrieving Details of record: 2/150
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/zoom-rival-sd-2-track-field-throwing-shoes-oLb8zO/685134-400

Retrieving Details of record: 3/150
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Get the HTML code: https://www.nike.com/t/luka-1-basketball-shoes-69X9Vs/DN1772-108

Retrieving Details of record: 4/150
Wait time ** 4 ** 
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/zoom-blazer-mid-pro-gt-skate-shoes-MLRQq9/FN4182-001

Retrieving Details of record: 5/150
Wait time ** 4 ** 
Item Nike Cortez Unlocked By You Greatest Gift has not reviews.
Item Nike Cortez Unlocked By You Greatest Gift has not reviews.
Get the HTML code: h

Item Nike Dunk Low By Kelcey has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-dunk-low-by-you-shoes-10001710/6183844464

Retrieving Details of record: 34/150
Wait time ** 4 ** 
Show reviews of 34
Click more reviews of 34
Get the HTML code: https://www.nike.com/t/sb-ishod-wair-premium-skate-shoes-xjNjBC/DZ5648-100

Retrieving Details of record: 35/150
Wait time ** 4 ** 
Show reviews of 35
Click more reviews of 35
Get the HTML code: https://www.nike.com/t/zoomx-dragoy-xc-cross-country-spikes-gjBdMM/DX7992-700

Retrieving Details of record: 36/150
Wait time ** 4 ** 
Show reviews of 36
Click more reviews of 36
Get the HTML code: https://www.nike.com/t/air-max-koko-womens-sandals-GPgRtj/CW9705-200

Retrieving Details of record: 37/150
Wait time ** 4 ** 
Review Exception of 437-Message: element click intercepted: Element <details data-nr-comp="details" data-nr-animatable="false" data-nr-animation="false" class=" css-1hbr3d8" data-test="reviewsAccordionClick" name="pdp-r

Get the HTML code: https://www.nike.com/t/air-zoom-lj-elite-track-field-jumping-spikes-mzRDhg/CT0079-400

Retrieving Details of record: 60/150
Wait time ** 4 ** 
Show reviews of 60
Click more reviews of 60
Get the HTML code: https://www.nike.com/t/zion-3-mud-sweat-and-tears-basketball-shoes-GWLnqd/DR0675-300

Retrieving Details of record: 61/150
Wait time ** 4 ** 
Show reviews of 61
Click more reviews of 61
Get the HTML code: https://www.nike.com/t/air-force-1-07-se-womens-shoes-dxBKk9/DO9458-100

Retrieving Details of record: 62/150
Wait time ** 4 ** 
Show reviews of 62
Click more reviews of 62
Get the HTML code: https://www.nike.com/t/roshe-one-womens-shoe-ol9k1O/844994-002

Retrieving Details of record: 63/150
Wait time ** 4 ** 
Item Nike Dunk Low Twist has not reviews.
Item Nike Dunk Low Twist has not reviews.
Get the HTML code: https://www.nike.com/t/dunk-low-twist-womens-shoes-MLblZl/DZ2794-102

Retrieving Details of record: 64/150
Wait time ** 4 ** 
Item Nike Dunk Low By Italia 

Retrieving Details of record: 99/150
Wait time ** 4 ** 
Show reviews of 99
Click more reviews of 99
Get the HTML code: https://www.nike.com/t/ajko-1-mens-shoes-GNcCR9/DO5047-411

Retrieving Details of record: 100/150
Wait time ** 4 ** 
Item Nike Air Max 1 '87 By You has not reviews.
Item Nike Air Max 1 '87 By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-air-max-1-unlocked-by-you-10001714/8309443440

Retrieving Details of record: 101/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Nike Air Max 1 '87 has not reviews.
Item Nike Air Max 1 '87 has not reviews.
Get the HTML code: https://www.nike.com/launch/r/FQ8900-440

Retrieving Details of record: 102/150
Wait time ** 4 ** 
Show reviews of 102
Click more reviews of 102
Get the HTML code: https://www.nike.com/t/blazer-mid-77-womens-shoes-25h71W/CZ1055-101

Retrieving Details of record: 103/150
Wait time ** 4 ** 
Show reviews of 103
Click more reviews of 103
Get the HTML code: https://www.nike.co

Get the HTML code: https://www.nike.com/u/custom-reactx-infinity-rn-4-by-you-10001739/2146020008

Retrieving Details of record: 139/150
Wait time ** 4 ** 
Show reviews of 139
Click more reviews of 139
Get the HTML code: https://www.nike.com/t/air-jordan-1-mid-se-craft-mens-shoes-5VjD18/FD6817-003

Retrieving Details of record: 140/150
Wait time ** 4 ** 
Show reviews of 140
Item Jumpman MVP has not reviews.
Get the HTML code: https://www.nike.com/t/jumpman-mvp-womens-shoes-pkJccB/FQ8337-121

Retrieving Details of record: 141/150
Wait time ** 4 ** 
Show reviews of 141
Click more reviews of 141
Get the HTML code: https://www.nike.com/t/sb-nyjah-free-2-skate-shoes-s4Kt9q/BV2078-100

Retrieving Details of record: 142/150
Wait time ** 4 ** 
Show reviews of 142
Click more reviews of 142
Get the HTML code: https://www.nike.com/t/sb-zoom-blazer-mid-x-mason-silva-skate-shoes-1FNC6x/DZ7260-400

Retrieving Details of record: 143/150
Wait time ** 4 ** 
Show reviews of 143
Click more reviews of 143


In [73]:
df_det5 = df_det4.copy()
url_error5 = url_error4.copy()

In [74]:
url_error5

,index,URL,Exception,msg
0,413,https://www.nike.com/t/air-huarache-x-serena-w...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
1,413,https://www.nike.com/t/air-huarache-x-serena-w...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
2,437,https://www.nike.com/t/m2k-tekno-shoes-kkgRr9/...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
3,437,https://www.nike.com/t/m2k-tekno-shoes-kkgRr9/...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
4,438,https://www.nike.com/t/lebron-19-low-basketbal...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
5,454,https://www.nike.com/t/air-jordan-1-low-og-wom...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
6,454,https://www.nike.com/t/air-jordan-1-low-og-wom...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...
7,548,https://www.nike.com/t/air-max-270-womens-shoe...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <details da...
8,548,https://www.nike.com/t/air-max-270-womens-shoe...,<class 'selenium.common.exceptions.ElementNotI...,element not interactable\n (Session info: chr...


In [77]:
# Join the batches - raw data 400 ->
df_det = pd.concat([df_det, df_det5])
url_error = pd.concat([url_error, url_error5])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det5.to_csv('datasets/det5_raw_nike.csv')
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 6th Batch - Nike

In [141]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][550:700]

# Scrape from 550 to 700 records
df_det6, url_error6 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

Retrieving Details of record: 1/150
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/air-jordan-1-retro-low-slip-womens-shoes-3lqhns/AV3918-102

Retrieving Details of record: 2/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Converse x Martine Ali Chuck 70 De Luxe Wedge has not reviews.
Item Converse x Martine Ali Chuck 70 De Luxe Wedge has not reviews.
Get the HTML code: https://www.nike.com/launch/r/A08650C-001

Retrieving Details of record: 3/150
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Get the HTML code: https://www.nike.com/t/ebernon-low-womens-shoe-6dfCc8/AQ1779-100

Retrieving Details of record: 4/150
Wait time ** 4 ** 
Show reviews of 4
Item Air Jordan 1 Mid SE has not reviews.
Get the HTML code: https://www.nike.com/t/air-jordan-1-mid-se-womens-shoes-spwLJ0/FJ3448-200

Retrieving Details of record: 5/150
Wait time ** 4 ** 
Show reviews of 5
Click more reviews of 5
Get the HTML code: h

Get the HTML code: https://www.nike.com/t/victory-pro-3-mens-golf-shoes-SsxNJ0/DV6800-110

Retrieving Details of record: 41/150
Wait time ** 4 ** 
Show reviews of 41
Click more reviews of 41
Get the HTML code: https://www.nike.com/t/air-jordan-1-low-g-nrg-mens-golf-shoes-9dJHhQ/FD6848-001

Retrieving Details of record: 42/150
Wait time ** 4 ** 
Show reviews of 42
Click more reviews of 42
Get the HTML code: https://www.nike.com/t/phantom-gx-elite-firm-ground-low-top-soccer-cleat-WGRzk4/DC9968-705

Retrieving Details of record: 43/150
Wait time ** 4 ** 
Show reviews of 43
Click more reviews of 43
Get the HTML code: https://www.nike.com/t/mercurial-vapor-15-elite-firm-ground-low-top-soccer-cleats-NLZxDR/DJ4978-605

Retrieving Details of record: 44/150
Wait time ** 4 ** 
Show reviews of 44
Click more reviews of 44
Get the HTML code: https://www.nike.com/t/phantom-gx-pro-firm-ground-high-top-soccer-cleats-fDn8NS/DD9465-600

Retrieving Details of record: 45/150
Wait time ** 4 ** 
Item Nike M

Show reviews of 76
Click more reviews of 76
Get the HTML code: https://www.nike.com/t/mercurial-superfly-9-elitehigh-top-soccer-cleats-0tSNhw/DJ5165-040

Retrieving Details of record: 77/150
Wait time ** 4 ** 
Item Nike Phantom Luna FG By You has not reviews.
Item Nike Phantom Luna FG By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-phantom-luna-academy-by-you-10001552/4333794322

Retrieving Details of record: 78/150
Wait time ** 4 ** 
Show reviews of 78
Click more reviews of 78
Get the HTML code: https://www.nike.com/t/phantom-gt2-elite-fg-firm-ground-soccer-cleats-GNHLws/CZ9890-007

Retrieving Details of record: 79/150
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Get the HTML code: https://www.nike.com/t/mercurial-superfly-9-academy-turf-high-top-soccer-shoes-pJ76Rf/DJ5629-605

Retrieving Details of record: 80/150
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Get the HTML code: https://www.nike.com/t/mercurial-vapor-15-club-i

Show reviews of 114
Click more reviews of 114
Get the HTML code: https://www.nike.com/t/kd16-bad-basketball-shoes-H3gcDG/DV2917-500

Retrieving Details of record: 115/150
Wait time ** 4 ** 
Show reviews of 115
Click more reviews of 115
Get the HTML code: https://www.nike.com/t/tiger-woods-13-mens-golf-shoes-HBZwkB/DR5752-100

Retrieving Details of record: 116/150
Wait time ** 4 ** 
Show reviews of 116
Click more reviews of 116
Get the HTML code: https://www.nike.com/t/air-jordan-13-wheat-mens-shoes-zp4pp0/414571-171

Retrieving Details of record: 117/150
Wait time ** 4 ** 
Show reviews of 117
Click more reviews of 117
Get the HTML code: https://www.nike.com/t/air-jordan-8-winterized-mens-shoes-HBQzn3/FD1334-001

Retrieving Details of record: 118/150
Wait time ** 4 ** 
Show reviews of 118
Click more reviews of 118
Get the HTML code: https://www.nike.com/t/air-jordan-3-fear-mens-shoes-5M3ZlK/CT8532-080

Retrieving Details of record: 119/150
Wait time ** 4 ** 
Show reviews of 119
Click mo

In [142]:
url_error6

,index,URL,Exception,msg


In [144]:
# Join the batches - raw data 400 ->
df_det = pd.concat([df_det, df_det6])
url_error = pd.concat([url_error, url_error6])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det6.to_csv('datasets/det6_raw_nike.csv')
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 7th Batch - Nike

In [154]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][700:900]

# Scrape from 700 to 900 records
df_det7, url_error7 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

df_det7.to_csv('datasets/det7_raw_nike.csv')


Retrieving Details of record: 1/200
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/gt-cut-2-gte-basketball-shoes-k20cn7/FV4145-001

Retrieving Details of record: 2/200
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/gt-hustle-2-gte-basketball-shoes-8FSgBX/FV4137-001

Retrieving Details of record: 3/200
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Get the HTML code: https://www.nike.com/t/gt-jump-2-basketball-shoes-JgT85c/FV1895-001

Retrieving Details of record: 4/200
Wait time ** 4 ** 
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/freak-5-basketball-shoes-jZzrx4/DX4985-600

Retrieving Details of record: 5/200
Wait time ** 4 ** 
Show reviews of 5
Click more reviews of 5
Get the HTML code: https://www.nike.com/t/giannis-immortality-3-basketball-shoes-fPJbRd/DZ7533-600

Retrieving Details of record: 6/200
Wait time ** 4 ** 
S

Retrieving Details of record: 44/200
Wait time ** 4 ** 
Show reviews of 44
Click more reviews of 44
Get the HTML code: https://www.nike.com/t/jordan-nu-retro-1-low-mens-shoes-L6Xk8z/DV5141-611

Retrieving Details of record: 45/200
Wait time ** 4 ** 
Show reviews of 45
Click more reviews of 45
Get the HTML code: https://www.nike.com/t/air-jordan-1-mid-shoes-X5pM09/554724-136

Retrieving Details of record: 46/200
Wait time ** 4 ** 
Show reviews of 46
Click more reviews of 46
Get the HTML code: https://www.nike.com/t/air-force-1-low-sp-x-undercover-mens-shoes-6M1qZM/DQ7558-001

Retrieving Details of record: 47/200
Wait time ** 4 ** 
Show reviews of 47
Click more reviews of 47
Get the HTML code: https://www.nike.com/t/air-force-1-07-lv8-mens-shoes-J3l1hH/FD2592-002

Retrieving Details of record: 48/200
Wait time ** 4 ** 
Show reviews of 48
Click more reviews of 48
Get the HTML code: https://www.nike.com/t/air-jordan-1-element-shoes-Kcsdzl/DB2889-501

Retrieving Details of record: 49/200
Wa

Retrieving Details of record: 86/200
Wait time ** 4 ** 
Show reviews of 86
Click more reviews of 86
Get the HTML code: https://www.nike.com/t/sb-zoom-janoski-og-shoes-p7gRgw/DV5475-300

Retrieving Details of record: 87/200
Wait time ** 4 ** 
Show reviews of 87
Click more reviews of 87
Get the HTML code: https://www.nike.com/t/jordan-one-take-4-basketball-shoes-nS8tqx/DZ3338-106

Retrieving Details of record: 88/200
Wait time ** 4 ** 
Show reviews of 88
Click more reviews of 88
Get the HTML code: https://www.nike.com/t/zoom-superfly-elite-2-track-field-sprinting-spikes-Lj6WXS/CD4382-400

Retrieving Details of record: 89/200
Wait time ** 4 ** 
Show reviews of 89
Click more reviews of 89
Get the HTML code: https://www.nike.com/t/free-metcon-5-team-mens-workout-shoes-H0Qszz/FN6616-800

Retrieving Details of record: 90/200
Wait time ** 4 ** 
Show reviews of 90
Click more reviews of 90
Get the HTML code: https://www.nike.com/t/alabama-amoffcourt-slides-7Fptmr/DX5635-002

Retrieving Details o

Retrieving Details of record: 125/200
Wait time ** 4 ** 
Show reviews of 125
Click more reviews of 125
Get the HTML code: https://www.nike.com/t/sb-ishod-wair-skate-shoes-ztstT9/DC7232-001

Retrieving Details of record: 126/200
Wait time ** 4 ** 
Show reviews of 126
Click more reviews of 126
Get the HTML code: https://www.nike.com/t/pegasus-turbo-mens-road-running-shoes-pnz45r/DM3413-002

Retrieving Details of record: 127/200
Wait time ** 4 ** 
Show reviews of 127
Click more reviews of 127
Get the HTML code: https://www.nike.com/t/pegasus-shield-mens-weatherized-road-running-shoes-4rL2wd/DO7625-001

Retrieving Details of record: 128/200
Wait time ** 4 ** 
Item Nike Dunk Low Unlocked By You has not reviews.
Item Nike Dunk Low Unlocked By You has not reviews.
Get the HTML code: https://www.nike.com/u/nike-dunk-low-unlocked-by-you-10001614/5150220534

Retrieving Details of record: 129/200
Wait time ** 4 ** 
Show reviews of 129
Click more reviews of 129
Get the HTML code: https://www.nike.

Retrieving Details of record: 164/200
Wait time ** 4 ** 
Show reviews of 164
Click more reviews of 164
Get the HTML code: https://www.nike.com/t/air-force-1-mid-07-mens-shoes-CmXM62/DV0806-102

Retrieving Details of record: 165/200
Wait time ** 4 ** 
Show reviews of 165
Click more reviews of 165
Get the HTML code: https://www.nike.com/t/griffey-2-mens-baseball-cleats-mfb0D1/DZ4637-001

Retrieving Details of record: 166/200
Wait time ** 4 ** 
Show reviews of 166
Click more reviews of 166
Get the HTML code: https://www.nike.com/t/tiempo-legend-10-pro-turf-low-top-soccer-shoes-PbtHQn/DV4336-040

Retrieving Details of record: 167/200
Wait time ** 4 ** 
Item Nike Metcon 9 By You has not reviews.
Item Nike Metcon 9 By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-metcon-9-by-you-10001600/6172845189

Retrieving Details of record: 168/200
Wait time ** 4 ** 
Show reviews of 168
Click more reviews of 168
Get the HTML code: https://www.nike.com/t/air-max-systm-florida

In [156]:
url_error7

,index,URL,Exception,msg


In [157]:
# Join the batches - raw data 400 ->
df_det = pd.concat([df_det, df_det7])
url_error = pd.concat([url_error, url_error7])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 8th Batch - Nike

In [163]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][900:1200]

# Scrape from 900 to 1200 records
df_det8, url_error8 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

df_det8.to_csv('datasets/det8_raw_nike.csv')


Retrieving Details of record: 1/300
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/luka-2-the-pitch-basketball-shoes-chLlzW/FQ9044-800

Retrieving Details of record: 2/300
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/air-max-sc-mens-shoes-LR42xg/FQ0205-133

Retrieving Details of record: 3/300
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Get the HTML code: https://www.nike.com/t/jordan-xii-g-golf-shoes-hnlQtM/DM9015-105

Retrieving Details of record: 4/300
Wait time ** 4 ** 
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/alpha-menace-pro-3-mens-football-cleats-JV7FzF/CT6649-414

Retrieving Details of record: 5/300
Wait time ** 4 ** 
Show reviews of 5
Item Nike Vapor Edge Elite 360 2 has not reviews.
Get the HTML code: https://www.nike.com/t/vapor-edge-elite-360-2-mens-football-cleats-LdsghK/DA5457-010

Retrieving Details o

Retrieving Details of record: 41/300
Wait time ** 4 ** 
Show reviews of 41
Click more reviews of 41
Get the HTML code: https://www.nike.com/t/space-hippie-01-shoes-gV6H79/DJ3056-004

Retrieving Details of record: 42/300
Wait time ** 4 ** 
Show reviews of 42
Click more reviews of 42
Get the HTML code: https://www.nike.com/t/air-max-genome-mens-shoes-f4SHr3/CW1648-003

Retrieving Details of record: 43/300
Wait time ** 4 ** 
Show reviews of 43
Item Nike Huarache 9 Elite LAX has not reviews.
Get the HTML code: https://www.nike.com/t/huarache-9-elite-lax-lacrosse-cleats-r025t2/FD0088-100

Retrieving Details of record: 44/300
Wait time ** 4 ** 
Show reviews of 44
Click more reviews of 44
Get the HTML code: https://www.nike.com/t/quest-4-mens-road-running-shoes-RlpLvS/DA1105-400

Retrieving Details of record: 45/300
Wait time ** 4 ** 
Item Nike Dunk Mid has not reviews.
Item Nike Dunk Mid has not reviews.
Get the HTML code: https://www.nike.com/t/dunk-mid-mens-shoes-RHtFM2/DV0830-001

Retriev

Retrieving Details of record: 82/300
Wait time ** 4 ** 
Show reviews of 82
Item Delaware State has not reviews.
Get the HTML code: https://www.nike.com/t/delaware-stateoffcourt-slides-5cB5Sp/DX5643-002

Retrieving Details of record: 83/300
Wait time ** 4 ** 
Item Nike Cortez Unlocked By You has not reviews.
Item Nike Cortez Unlocked By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-cortez-unlocked-by-you-10001612/5568752196

Retrieving Details of record: 84/300
Wait time ** 4 ** 
Show reviews of 84
Click more reviews of 84
Get the HTML code: https://www.nike.com/t/p-6000-shoes-zj8hcJ/CD6404-400

Retrieving Details of record: 85/300
Wait time ** 4 ** 
Show reviews of 85
Click more reviews of 85
Get the HTML code: https://www.nike.com/t/offcourt-mens-slides-Xkb8kb/BQ4639-203

Retrieving Details of record: 86/300
Wait time ** 4 ** 
Show reviews of 86
Click more reviews of 86
Get the HTML code: https://www.nike.com/t/acg-mountain-fly-2-low-mens-shoes-kl94gN/DV7903-00

Wait time ** 4 ** 
Show reviews of 122
Click more reviews of 122
Get the HTML code: https://www.nike.com/t/air-flight-89-mens-shoes-bBlMQz/CN0050-100

Retrieving Details of record: 123/300
Wait time ** 4 ** 
Show reviews of 123
Click more reviews of 123
Get the HTML code: https://www.nike.com/t/lunar-force-1-mens-duckboot-tTZklH/DZ5320-200

Retrieving Details of record: 124/300
Wait time ** 4 ** 
Show reviews of 124
Click more reviews of 124
Get the HTML code: https://www.nike.com/t/pegasus-40-uswnt-mens-road-running-shoes-BWM0x2/FN0096-401

Retrieving Details of record: 125/300
Wait time ** 4 ** 
Show reviews of 125
Click more reviews of 125
Get the HTML code: https://www.nike.com/t/nikecourt-air-zoom-vapor-11-mens-hard-court-tennis-shoes-jdHWL8/DR6966-102

Retrieving Details of record: 126/300
Wait time ** 4 ** 
Show reviews of 126
Click more reviews of 126
Get the HTML code: https://www.nike.com/t/air-zoom-gt-cut-2-womens-basketball-shoes-DGHHnK/FN0234-104

Retrieving Details of rec

Show reviews of 162
Click more reviews of 162
Get the HTML code: https://www.nike.com/t/lunargato-ii-indoor-court-low-top-soccer-shoes-qqTAoz9v/580456-500

Retrieving Details of record: 163/300
Wait time ** 4 ** 
Show reviews of 163
Click more reviews of 163
Get the HTML code: https://www.nike.com/t/phantom-gx-academy-multi-ground-low-top-soccer-cleats-1810WZ/DD9473-040

Retrieving Details of record: 164/300
Wait time ** 4 ** 
Item Nike Air Max 97 By You has not reviews.
Item Nike Air Max 97 By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-air-max-97-shoes-by-you-10001609/6663364575

Retrieving Details of record: 165/300
Wait time ** 4 ** 
Item Nike Dunk High By You has not reviews.
Item Nike Dunk High By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-dunk-high-by-you-shoes-10001603/4473252113

Retrieving Details of record: 166/300
Wait time ** 4 ** 
Show reviews of 166
Click more reviews of 166
Get the HTML code: https://www.nik

Item Nike Air Zoom HyperAce 2 SE has not reviews.
Get the HTML code: https://www.nike.com/t/air-zoom-hyperace-2-se-volleyball-shoes-qvGKBC/DM8199-064

Retrieving Details of record: 201/300
Wait time ** 4 ** 
Show reviews of 201
Click more reviews of 201
Get the HTML code: https://www.nike.com/t/tennessee-stateoffcourt-slides-8psDDC/DZ0962-002

Retrieving Details of record: 202/300
Wait time ** 4 ** 
Show reviews of 202
Click more reviews of 202
Get the HTML code: https://www.nike.com/t/metcon-9-mens-workout-shoes-00kjPz/DZ2617-101

Retrieving Details of record: 203/300
Wait time ** 4 ** 
Show reviews of 203
Click more reviews of 203
Get the HTML code: https://www.nike.com/t/luka-2-team-bank-basketball-shoes-qvvhqz/FN7400-140

Retrieving Details of record: 204/300
Wait time ** 4 ** 
Show reviews of 204
Click more reviews of 204
Get the HTML code: https://www.nike.com/t/converse-chuck-taylor-all-star-high-top-unisex-shoes-xX439O/M9160-000

Retrieving Details of record: 205/300
Wait time 

The process managed to recover only 231 records.

In [169]:
url_error8

,index,URL,Exception,msg


In [182]:
# Join the batches - raw data 400 ->
df_det = pd.concat([df_det, df_det8])
url_error = pd.concat([url_error, url_error8])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 9th Batch - Nike

In [188]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][1131:1400]

# Scrape from 1131 to 1400 records
df_det9, url_error9 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

df_det9.to_csv('datasets/det9_raw_nike.csv')


Retrieving Details of record: 1/269
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Get the HTML code: https://www.nike.com/t/air-jordan-legacy-312-low-mens-shoes-B931hr/CD7069-160

Retrieving Details of record: 2/269
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Get the HTML code: https://www.nike.com/t/air-max-ltd-3-mens-shoe-RJzJf1/BV1171-100

Retrieving Details of record: 3/269
Wait time ** 4 ** 
Item KD16 By You has not reviews.
Item KD16 By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-kd-16-by-you-10001667/6375549502

Retrieving Details of record: 4/269
Wait time ** 4 ** 
Show reviews of 4
Click more reviews of 4
Get the HTML code: https://www.nike.com/t/react-infinity-3-premium-mens-road-running-shoes-X4PKW0/DZ3025-001

Retrieving Details of record: 5/269
Wait time ** 4 ** 
Show reviews of 5
Click more reviews of 5
Get the HTML code: https://www.nike.com/t/air-force-1-react-mens-shoes-WSjfs8/DM0573-101

Retrieving Details

Get the HTML code: https://www.nike.com/t/alpha-huarache-elite-4-low-team-mens-baseball-cleats-9C4p8L/DR6851-100

Retrieving Details of record: 40/269
Wait time ** 4 ** 
Show reviews of 40
Click more reviews of 40
Get the HTML code: https://www.nike.com/t/alphafly-2-mens-road-racing-shoes-Bstm8X/DN3555-600

Retrieving Details of record: 41/269
Wait time ** 4 ** 
Show reviews of 41
Item Nike ACG Air Deschutz+ SE has not reviews.
Get the HTML code: https://www.nike.com/t/acg-air-deschutz-se-shoes-WtcmP1/DR1020-400

Retrieving Details of record: 42/269
Wait time ** 4 ** 
Item Nike Dunk Low By You has not reviews.
Item Nike Dunk Low By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-nike-dunk-low-by-you-shoes-10001477/3255832916

Retrieving Details of record: 43/269
Wait time ** 4 ** 
Show reviews of 43
Item Nike Vapor 15 Academy Mercurial Dream Speed has not reviews.
Get the HTML code: https://www.nike.com/t/vapor-15-academy-mercurial-dream-speed-ic-low-top-soccer-sh

Retrieving Details of record: 78/269
Wait time ** 4 ** 
Show reviews of 78
Click more reviews of 78
Get the HTML code: https://www.nike.com/t/superrep-go-3-next-nature-flyknit-mens-workout-shoes-9BPXdD/DH3394-008

Retrieving Details of record: 79/269
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Get the HTML code: https://www.nike.com/t/sb-force-58-skate-shoe-Fslk1b/CZ2959-402

Retrieving Details of record: 80/269
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Get the HTML code: https://www.nike.com/t/structure-24-mens-road-running-shoes-W9Q4jg/DA8535-001

Retrieving Details of record: 81/269
Wait time ** 4 ** 
Item Freak 5 By You has not reviews.
Item Freak 5 By You has not reviews.
Get the HTML code: https://www.nike.com/u/custom-freak-5-by-you-10001743/1513602991

Retrieving Details of record: 82/269
Wait time ** 4 ** 
Show reviews of 82
Click more reviews of 82
Get the HTML code: https://www.nike.com/t/air-max-systm-michigan-mens-shoes-34ctxb/DZ7739

Show reviews of 117
Click more reviews of 117
Get the HTML code: https://www.nike.com/t/air-jordan-12-low-golf-shoes-0fSmzT/DH4120-101

Retrieving Details of record: 118/269
Wait time ** 4 ** 
Show reviews of 118
Click more reviews of 118
Get the HTML code: https://www.nike.com/t/alpha-huarache-4-keystone-mens-baseball-cleats-dGXhG3/DJ6524-104

Retrieving Details of record: 119/269
Wait time ** 4 ** 
Show reviews of 119
Click more reviews of 119
Get the HTML code: https://www.nike.com/t/revolution-6-flyease-mens-easy-on-off-road-running-shoes-SNbtsR/DC8992-003

Retrieving Details of record: 120/269
Wait time ** 4 ** 
Show reviews of 120
Click more reviews of 120
Get the HTML code: https://www.nike.com/t/converse-chuck-taylor-all-star-low-top-unisex-shoe-NLRAok/M9697-410

Retrieving Details of record: 121/269
Wait time ** 4 ** 
Show reviews of 121
Click more reviews of 121
Get the HTML code: https://www.nike.com/t/react-hyperset-indoor-court-shoes-dkQgjQ/CI2955-160

Retrieving Details o

Retrieving Details of record: 157/269
Wait time ** 4 ** 
Show reviews of 157
Click more reviews of 157
Get the HTML code: https://www.nike.com/t/alpha-menace-varsity-3-mens-football-cleats-7hjdSW/CV0586-616

Retrieving Details of record: 158/269
Wait time ** 4 ** 
Show reviews of 158
Click more reviews of 158
Get the HTML code: https://www.nike.com/t/air-zoom-infinity-tour-mens-golf-shoes-d64khm/DC5221-041

Retrieving Details of record: 159/269
Wait time ** 4 ** 
Show reviews of 159
Item Nike Air Force 1 '07 has not reviews.
Get the HTML code: https://www.nike.com/t/air-force-1-07-mens-shoes-0D8pNp/FZ3592-259

Retrieving Details of record: 160/269
Wait time ** 4 ** 
Show reviews of 160
Click more reviews of 160
Get the HTML code: https://www.nike.com/t/cosmic-unity-3-aja-wilson-womens-basketball-shoes-C0NgDW/FQ9395-600

Retrieving Details of record: 161/269
Wait time ** 4 ** 
Show reviews of 161
Item Nike Pegasus 40 (Kentucky) has not reviews.
Get the HTML code: https://www.nike.com/t/

Wait time ** 4 ** 
Show reviews of 196
Click more reviews of 196
Get the HTML code: https://www.nike.com/t/precision-6-flyease-basketball-shoes-5c6wsS/DJ7552-002

Retrieving Details of record: 197/269
Wait time ** 4 ** 
Show reviews of 197
Item Nike Pegasus 40 (Georgia) has not reviews.
Get the HTML code: https://www.nike.com/t/pegasus-40-georgia-mens-road-running-shoes-G3lljJ/DZ5977-100

Retrieving Details of record: 198/269
Wait time ** 4 ** 
Show reviews of 198
Item Nike Offcourt (Tennessee) has not reviews.
Get the HTML code: https://www.nike.com/t/offcourt-tennessee-slide-dSdQp2/DD0549-002

Retrieving Details of record: 199/269
Wait time ** 4 ** 
Show reviews of 199
Click more reviews of 199
Get the HTML code: https://www.nike.com/t/pegasus-flyease-mens-easy-on-off-road-running-shoes-744b9V/DJ7381-001

Retrieving Details of record: 200/269
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Nike Magmascape x sacai has not reviews.
Item Nike Magmascape x sacai has not r

Item Jordan Stadium 90 has not reviews.
Get the HTML code: https://www.nike.com/t/jordan-stadium-90-mens-shoes-zTH0FR/HF5258-102

Retrieving Details of record: 235/269
Wait time ** 4 ** 
Show reviews of 235
Click more reviews of 235
Get the HTML code: https://www.nike.com/t/offcourt-arizona-slide-9wl1PV/DD0505-002

Retrieving Details of record: 236/269
Wait time ** 4 ** 
Show reviews of 236
Item NikeCourt Air Zoom Vapor 11 Attack has not reviews.
Get the HTML code: https://www.nike.com/t/nikecourt-air-zoom-vapor-11-attack-mens-hard-court-tennis-shoes-Rqwv3l/FN2152-001

Retrieving Details of record: 237/269
Wait time ** 4 ** 
Show reviews of 237
Click more reviews of 237
Get the HTML code: https://www.nike.com/t/court-vision-low-next-nature-mens-shoes-m7TQr0/FB8942-133

Retrieving Details of record: 238/269
Wait time ** 4 ** 
Show reviews of 238
Item Nike Mercurial Vapor 15 Academy has not reviews.
Get the HTML code: https://www.nike.com/t/mercurial-vapor-15-academy-turf-low-top-soccer-

In [193]:
url_error9

,index,URL,Exception,msg


In [192]:
# Join the batches - raw data 400 ->
df_det = pd.concat([df_det, df_det9])
url_error = pd.concat([url_error, url_error9])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

#### 10th Batch - Nike (Final)

In [210]:
%%time

# Defines the urls 
df_urls = df_shoes_nike[['ShoeName', 'URL']][1400:]

# Scrape from 1400 to 1674 records
df_det10, url_error10 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)

df_det10.to_csv('datasets/det10_raw_nike.csv')


Retrieving Details of record: 1/274
Wait time ** 4 ** 
Show reviews of 1
Item Nike Offcourt (Arkansas) has not reviews.
Get the HTML code: https://www.nike.com/t/offcourt-arkansas-slide-zKf8M9/DD0507-002

Retrieving Details of record: 2/274
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Nike Attack has not reviews.
Item Nike Attack has not reviews.
Get the HTML code: https://www.nike.com/launch/r/FB8938-101

Retrieving Details of record: 3/274
Wait time ** 4 ** 
Show reviews of 3
Item Nike Pegasus 40 (Alabama A&M) has not reviews.
Get the HTML code: https://www.nike.com/t/pegasus-40-alabama-am-mens-road-running-shoes-N9BTNC/DZ5992-100

Retrieving Details of record: 4/274
Wait time ** 4 ** 
Show reviews of 4
Item Nike Blazer Mid Pro Club has not reviews.
Get the HTML code: https://www.nike.com/t/blazer-mid-pro-club-mens-shoes-vv1b19/FB8891-001

Retrieving Details of record: 5/274
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Nike Zoom LeBron 4 has not 

Wait time ** 4 ** 
Show reviews of 41
Click more reviews of 41
Get the HTML code: https://www.nike.com/t/air-force-1-high-07-lv8-mens-shoes-lR6frj/DV0790-001

Retrieving Details of record: 42/274
Wait time ** 4 ** 
Show reviews of 42
Click more reviews of 42
Get the HTML code: https://www.nike.com/t/pegasus-trail-3-gore-tex-mens-waterproof-trail-running-shoes-kzkHrx/DC8793-401

Retrieving Details of record: 43/274
Wait time ** 4 ** 
Show reviews of 43
Click more reviews of 43
Get the HTML code: https://www.nike.com/t/benassi-n7-slides-BrVJ9q/FV5914-010

Retrieving Details of record: 44/274
Wait time ** 4 ** 
Show reviews of 44
Click more reviews of 44
Get the HTML code: https://www.nike.com/t/alpha-huarache-8-elite-lacrosse-cleats-s6FTxq/CW4440-011

Retrieving Details of record: 45/274
Wait time ** 4 ** 
Show reviews of 45
Click more reviews of 45
Get the HTML code: https://www.nike.com/t/zion-1-basketball-shoes-bJ0hLJ/DA3130-241

Retrieving Details of record: 46/274
Wait time ** 4 ** 

Retrieving Details of record: 81/274
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Nike Air Penny 2 QS has not reviews.
Item Nike Air Penny 2 QS has not reviews.
Get the HTML code: https://www.nike.com/launch/r/FN4438-400

Retrieving Details of record: 82/274
Wait time ** 4 ** 
Show reviews of 82
Item Nike Pegasus 40 (Tennessee State) has not reviews.
Get the HTML code: https://www.nike.com/t/pegasus-40-tennessee-state-mens-road-running-shoes-1znWMD/DZ5978-100

Retrieving Details of record: 83/274
Wait time ** 4 ** 
Show reviews of 83
Click more reviews of 83
Get the HTML code: https://www.nike.com/t/air-force-1-high-07-lv8-emb-mens-shoes-g2WGZn/DX4980-001

Retrieving Details of record: 84/274
Wait time ** 4 ** 
Show reviews of 84
Click more reviews of 84
Get the HTML code: https://www.nike.com/t/air-jordan-xxxvii-basketball-shoes-MttxV5/DD6958-001

Retrieving Details of record: 85/274
Wait time ** 4 ** 
Show reviews of 85
Click more reviews of 85
Get the HTML code: h

Wait time ** 4 ** 
Show reviews of 120
Click more reviews of 120
Get the HTML code: https://www.nike.com/t/jordan-6-rings-mens-shoes-0ZjmZB/322992-067

Retrieving Details of record: 121/274
Wait time ** 4 ** 
Show reviews of 121
Click more reviews of 121
Get the HTML code: https://www.nike.com/t/dunk-low-retro-mens-shoes-7zsVZN/DV0833-600

Retrieving Details of record: 122/274
Wait time ** 4 ** 
Show reviews of 122
Item Nike Pegasus 40 (Spelman) has not reviews.
Get the HTML code: https://www.nike.com/t/pegasus-40-spelman-mens-road-running-shoes-hPmBtn/DZ5956-100

Retrieving Details of record: 123/274
Wait time ** 4 ** 
Show reviews of 123
Click more reviews of 123
Get the HTML code: https://www.nike.com/t/offline-2-mens-mules-Q46CBn/DJ6229-200

Retrieving Details of record: 124/274
Wait time ** 4 ** 
Show reviews of 124
Click more reviews of 124
Get the HTML code: https://www.nike.com/t/acg-lowcate-x-future-movement-mens-shoes-sKqFzL/FB9761-300

Retrieving Details of record: 125/274
W

Get the HTML code: https://www.nike.com/t/pegasus-40-penn-state-mens-road-running-shoes-vmQhzj/DZ5944-100

Retrieving Details of record: 161/274
Wait time ** 4 ** 
Show reviews of 161
Item Nike SFB B2 Realtree® has not reviews.
Get the HTML code: https://www.nike.com/t/sfb-b2-realtree-mens-boots-Rrv9j1/FN3721-200

Retrieving Details of record: 162/274
Wait time ** 4 ** 
Show reviews of 162
Click more reviews of 162
Get the HTML code: https://www.nike.com/t/air-jordan-1-low-se-craft-mens-shoes-lcTJxC/DZ4135-002

Retrieving Details of record: 163/274
Wait time ** 4 ** 
Item LeBron Witness 6 (Team) has not reviews.
Item LeBron Witness 6 (Team) has not reviews.
Get the HTML code: https://www.nike.com/t/lebron-witness-6-team-basketball-shoes-d6BdZB/DO9843-100

Retrieving Details of record: 164/274
Wait time ** 4 ** 
Show reviews of 164
Item Nike Offcourt (Clemson) has not reviews.
Get the HTML code: https://www.nike.com/t/offcourt-clemson-slide-BD9FPm/DD0518-002

Retrieving Details of recor

Retrieving Details of record: 199/274
Wait time ** 4 ** 
Show reviews of 199
Click more reviews of 199
Get the HTML code: https://www.nike.com/t/offcourt-michigan-slide-mpg9l0/DD0533-002

Retrieving Details of record: 200/274
Wait time ** 4 ** 
Show reviews of 200
Click more reviews of 200
Get the HTML code: https://www.nike.com/t/jordan-delta-3-low-mens-shoes-Zcx4B1/DN2647-100

Retrieving Details of record: 201/274
Wait time ** 4 ** 
Show reviews of 201
Item Duke has not reviews.
Get the HTML code: https://www.nike.com/t/dukeoffcourt-slides-7FZDqv/DX5211-002

Retrieving Details of record: 202/274
Wait time ** 4 ** 
Show reviews of 202
Click more reviews of 202
Get the HTML code: https://www.nike.com/t/air-max-97-og-mens-shoes-zV9FQs/DM0028-400

Retrieving Details of record: 203/274
Wait time ** 4 ** 
Show reviews of 203
Click more reviews of 203
Get the HTML code: https://www.nike.com/t/court-legacy-canvas-mens-shoes-S38RF1/CW6539-100

Retrieving Details of record: 204/274
Wait time *

Get the HTML code: https://www.nike.com/t/offcourt-los-angeles-chargers-slide-pxCVwB/DD0516-002

Retrieving Details of record: 240/274
Wait time ** 4 ** 
Show reviews of 240
Click more reviews of 240
Get the HTML code: https://www.nike.com/t/offcourt-arizona-cardinals-slide-MmWZ1t/DD0514-002

Retrieving Details of record: 241/274
Wait time ** 4 ** 
Show reviews of 241
Click more reviews of 241
Get the HTML code: https://www.nike.com/t/offcourt-cleveland-browns-slide-5n9JGH/DD0512-002

Retrieving Details of record: 242/274
Wait time ** 4 ** 
Show reviews of 242
Item Nike Offcourt (NFL Atlanta Falcons) has not reviews.
Get the HTML code: https://www.nike.com/t/offcourt-atlanta-falcons-slide-18S7pH/DD0521-002

Retrieving Details of record: 243/274
Wait time ** 4 ** 
Show reviews of 243
Click more reviews of 243
Get the HTML code: https://www.nike.com/t/offcourt-miami-dolphins-slide-6zpB3R/DD0520-002

Retrieving Details of record: 244/274
Wait time ** 4 ** 
Show reviews of 244
Click more r

In [211]:
url_error10

,index,URL,Exception,msg


In [212]:
# Join the batches - raw data 400 ->
df_det = pd.concat([df_det, df_det10])
url_error = pd.concat([url_error, url_error10])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Nike
df_det_nike_backup = df_det.copy()

# Save a file with the original data recovered
df_det_nike_backup.to_csv('datasets/det_raw_nike.csv')
url_error.to_csv('errors/nike_batch_err.csv') 

### 1.2 Data Processing

<!-- df_proc_backup = df_proc.copy() -->

The first 400 records in df_det are already clean, so will clean only the last 1274, and concat all in df_proc. Save the Nike detail data processed in a file, then we will extract the required data for Table 2 and Table 3 according to the specifications.

In [296]:
# Cleans data frame  
# The first 400 records are already clean, so will apart them
df_400 = df_det[:400]

# Cleans the rest of the records, the last 1274, and concat all in df_proc
df_proc_batch = clean_df_det_nike(df_det[400:])
df_proc = pd.concat([df_400, df_proc_batch]) 

# Drop columns no longer needed
df_proc.drop(['reviews', 'color'], axis=1, inplace=True)

# Reset index
df_proc.reset_index(drop=True, inplace=True)

# Save the detail data processed of Nike for reference
df_proc.to_csv('datasets/det_processed_nike.csv')

In [302]:
print(f"Nike detail data: {df_proc.shape}")
display(df_proc.head())
display(df_proc.tail())

Nike detail data: (1674, 10)


,count_of_sizes,colors,product_code,star_rating,Size,comfort,quality,n_reviews,yes_recommendations,%_recommended
0,13,[White/White/Black],DD1503-101,4.8,49.0,78.0,44.66,617.0,43.0,6.97
1,15,"[White/Rugged Orange/Campfire Orange/Cedar, Wh...",DX9176-101,4.5,61.0,75.0,50.78,147.0,24.0,16.33
2,15,"[White/White/Black, White/University Blue, Whi...",DH3158-107,4.6,59.0,78.0,46.67,199.0,6.0,3.02
3,15,[Light Bone/White/Metallic Silver/Diffused Blu...,DZ7736-002,5,NaN,NaN,NaN,2.0,NaN,NaN
4,13,"[Midnight Navy/Natural/Varsity Maize, Neutral ...",DZ2628-102,4.6,43.0,78.0,54.62,70.0,30.0,42.86


,count_of_sizes,colors,product_code,star_rating,Size,comfort,quality,n_reviews,yes_recommendations,%_recommended
1669,10,[Black/University Orange/White],DH6989-002,0,NaN,NaN,NaN,0.0,NaN,NaN
1670,10,[Black/College Navy/University Orange],DH6978-002,0,NaN,NaN,NaN,0.0,NaN,NaN
1671,10,[Black/Mardi Gras/Wolf Grey],DH6985-002,0,NaN,NaN,NaN,0.0,NaN,NaN
1672,10,[Black/College Navy/Sport Red],DH6987-002,0,NaN,NaN,NaN,0.0,NaN,NaN
1673,10,[Black/University Gold/White],DH7002-002,5,NaN,NaN,NaN,1.0,NaN,NaN


In [299]:
# Setting tables 2 and 3
df_table2_nike = df_proc[['product_code', 'count_of_sizes', 'colors']]
df_table3_nike = df_proc[['product_code', 'n_reviews', 'quality', 'star_rating','Size','comfort']]

# Saving the data frame
df_table2_nike.to_csv('datasets/Table2_nike.csv')
display(df_table2_nike)

df_table3_nike.to_csv('datasets/Table3_nike.csv')
display(df_table3_nike)

,product_code,count_of_sizes,colors
0,DD1503-101,13,[White/White/Black]
1,DX9176-101,15,"[White/Rugged Orange/Campfire Orange/Cedar, Wh..."
2,DH3158-107,15,"[White/White/Black, White/University Blue, Whi..."
3,DZ7736-002,15,[Light Bone/White/Metallic Silver/Diffused Blu...
4,DZ2628-102,13,"[Midnight Navy/Natural/Varsity Maize, Neutral ..."
...,...,...,...
1669,DH6989-002,10,[Black/University Orange/White]
1670,DH6978-002,10,[Black/College Navy/University Orange]
1671,DH6985-002,10,[Black/Mardi Gras/Wolf Grey]
1672,DH6987-002,10,[Black/College Navy/Sport Red]


,product_code,n_reviews,quality,star_rating,Size,comfort
0,DD1503-101,617.0,44.66,4.8,49.0,78.0
1,DX9176-101,147.0,50.78,4.5,61.0,75.0
2,DH3158-107,199.0,46.67,4.6,59.0,78.0
3,DZ7736-002,2.0,NaN,5,NaN,NaN
4,DZ2628-102,70.0,54.62,4.6,43.0,78.0
...,...,...,...,...,...,...
1669,DH6989-002,0.0,NaN,0,NaN,NaN
1670,DH6978-002,0.0,NaN,0,NaN,NaN
1671,DH6985-002,0.0,NaN,0,NaN,NaN
1672,DH6987-002,0.0,NaN,0,NaN,NaN


Now we will assign the product_code to Table 1.

In [268]:
# Establishing the product_code in Table1
df_shoes_nike.loc[:, 'product_code'] = df_proc['product_code']

# Save the Table1 including the product_code
df_shoes_nike.to_csv('datasets/Table1_nike.csv')
df_shoes_nike

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand,product_code
0,NaN,https://www.nike.com/t/dunk-low-womens-shoes-4...,1,115.00,115,Women,Nike,DD1503-101
1,Nike Gamma Force,https://www.nike.com/t/gamma-force-womens-shoe...,9,72.97,90,Women,Nike,DX9176-101
2,Nike Court Vision Low Next Nature,https://www.nike.com/t/court-vision-low-next-n...,6,60.97,80,Women,Nike,DH3158-107
3,Nike Air Max 270,https://www.nike.com/t/air-max-270-womens-shoe...,12,120.97,160,Women,Nike,DZ7736-002
4,Nike Air Max 1,https://www.nike.com/t/air-max-1-womens-shoes-...,6,140.00,140,Women,Nike,DZ2628-102
...,...,...,...,...,...,...,...,...
1670,Nike Offcourt (MLB Baltimore Orioles),https://www.nike.com/t/offcourt-baltimore-orio...,1,40.00,40,Men,Nike,DH6978-002
1671,Nike Offcourt (MLB Houston Astros),https://www.nike.com/t/offcourt-houston-astros...,1,40.00,40,Men,Nike,DH6985-002
1672,Nike Offcourt (MLB Seattle Mariners),https://www.nike.com/t/offcourt-seattle-marine...,1,40.00,40,Men,Nike,DH6987-002
1673,Nike Offcourt (MLB Cleveland),https://www.nike.com/t/offcourt-cleveland-slid...,1,40.00,40,Men,Nike,DH7002-002


For now, we have gathered all the required data from Nike. Later on, we will reorganize the colors with the rest of the data recovered.

In [312]:
driver.quit()

## 2. Adidas.com

In [314]:
%%time

# Initialization of the dataframe
df_shoes_adidas = pd.DataFrame()
n_shoes_in_cat = 0

# Scrap the main data for Men and Women in adidas.com
for category in ['WOMEN','MEN']:
    
    # Config to scrape ShoeName, Category(Men/Women), no.of colors, price
    url = sites['adidas']
    xpath_grp = '//div[@class="grid-item"]'
    soup_config = {
        'ShoeName':['p',{'data-auto-id':"product-card-title"},None, None ,'-'], 
        'URL': ['div', {'data-auto-id':"glass-product-card"},
                lambda el: el.find('a')['href'] if el.find('a') is not None else '-', 
                None, '-'],
        'no_colors': ['span', {'data-auto-id': "product-card-colvar-count"},None, None, '-'],
        'price_reduce': ['div', {'class':"gl-price-item gl-price-item--sale notranslate"}, None,None, '-' ],
        'price': ['div', {'class':"gl-price-item gl-price-item--crossed notranslate"},None, None, '-'],
        'price_': ['div', {'class':"gl-price-item notranslate"}, None,None, '-'],  
        'Category': ['p',{'data-auto-id': "product-card-category"}, None, None, category]
    }
    
    try:
        # Open the Driver 
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(url) 

        wait_time = 3
        time.sleep(wait_time)

        # 1. Go to the Category - Shoes
        path_adidas = {'msg': [f'\n{category} Category...', 'Choosing shoes...'],
                       'xpaths': [ f'//span[contains(text(), "SHOP {category}")]', '//a[contains(text(),"Shoes")]' ],
                       'wait_time': [ 5, 4 ]}
        follow_path_to(path_adidas) 

        # 2.Closing pop-up ESC
        xpath = '//h3[contains(text(),"Your adiClub benefits await")]'
        time.sleep(wait_time*2)
        try:
            button = driver.find_element(By.XPATH, xpath)
            ActionChains(driver).send_keys(Keys.ESCAPE).perform() # Sending a ESC
            print(f"Escape of Adidas Club pop-up .")
        except NoSuchElementException as e:
            print(f"No Adidas Club pop-up .")

        # 3. Retrieve the number of pages    
        pages = get_number_of_pages(driver, 
                                    xpath ='//span[@data-auto-id="pagination-pages-container"]', 
                                    pattern=r'(\d+)')

        for i in range(pages):

            # 4.Scrolling to load all the data 
            smooth_scroll(driver, scrolls=20, scroll_speed=2, scroll_break=15)

            print('*'*50)
            print(' '*5, f'S C R A P I N G - Category: {category}, Page {i+1}')
            print('*'*50)

            # 5.Scraping ShoeName ,Category(Men/Women), no.of colors, price
            time.sleep(2)
            df_shoes_cat = scrape_items_by_group(driver, xpath_grp, **soup_config)

            # 6. Checks the category and Append the new data 
            df_shoes_cat['Category'] = df_shoes_cat['Category'].apply(lambda x: add_category(category, x))
            df_shoes_adidas = pd.concat([df_shoes_adidas, df_shoes_cat])

            # 7. Go to next page
            try:
                go_next_page(driver, '//span[@data-auto-id="pagination-right-button" and contains(text(),"Next")]')
            except NoSuchElementException as e:
                print("No more pages.")
                break

        n_shoes_in_cat = (len(df_shoes_adidas) - n_shoes_in_cat) if n_shoes_in_cat!=0 else len(df_shoes_adidas)
        print( f'Products in category {category} retrieved successfully: {n_shoes_in_cat}' )

        # Free driver
        driver.quit()

    except Exception as e:
        print(f"Error: {e.msg}")



Wait time ** 5 **

WOMEN Category...
Wait time ** 4 **
Choosing shoes...
Escape of Adidas Club pop-up .
Number of pages: 19
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: WOMEN, Page 1
**************************************************
Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: WOMEN, Page 2
**************************************************
Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: WOMEN, Page 3
**************************************************
Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: WOMEN, Page 4
********************

Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: MEN, Page 15
**************************************************
Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: MEN, Page 16
**************************************************
Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: MEN, Page 17
**************************************************
Recovered group-tags: 48
Records converted to HTML strings: 48
Going to NEXT
Scrolled 16 times.
**************************************************
      S C R A P I N G - Category: MEN, Page 18
**************************************************
Recovered group-tags

In [315]:
df_shoes_adidas

,ShoeName,URL,no_colors,price_reduce,price,price_,Category
0,Superstar XLG Shoes,https://www.adidas.com/us/superstar-xlg-shoes/...,12 colors,$88,$110,-,WOMEN Originals
1,Cloudfoam Pure Shoes,https://www.adidas.com/us/cloudfoam-pure-shoes...,2 colors,$45,$75,-,Women's Essentials
2,AdiFOM SST Boot Shoes,https://www.adidas.com/us/adifom-sst-boot-shoe...,3 colors,$44,$110,-,Women's Originals
3,Racer TR23 Shoes,https://www.adidas.com/us/racer-tr23-shoes/IF3...,11 colors,$56,$80,-,Women's Sportswear
4,Gazelle Indoor Shoes,https://www.adidas.com/us/gazelle-indoor-shoes...,2 colors,$105,$150,-,WOMEN Originals
...,...,...,...,...,...,...,...
6,TERREX Soulstride Trail Running Shoes,https://www.adidas.com/us/terrex-soulstride-tr...,3 colors,$48,$80,-,Men's TERREX
7,Adizero Afterburner NWV Cleats,https://www.adidas.com/us/adizero-afterburner-...,10 colors,$84,$140,-,Men's Baseball
8,Stabil Next Gen Shoes,https://www.adidas.com/us/stabil-next-gen-shoe...,-,$96,$160,-,Men's Training
9,Tour360 22 Golf Shoes,https://www.adidas.com/us/tour360-22-golf-shoe...,7 colors,$126,$180,-,Men's Golf


In [316]:
# Backup dataframe detail brand Nike
df_adidas_backup = df_shoes_adidas.copy()

# Save a file with the original data recovered
df_adidas_backup.to_csv('datasets/raw_adidas.csv')

In [317]:
# Cleaning the data frame
def clean_df_adidas(df_shoes):
    df_shoes['no_colors'] = df_shoes['no_colors'].str.extract(r'(\d+)')

    df_shoes['price'].fillna(df_shoes['price_'], inplace=True)
    df_shoes['price_reduce'] = df_shoes['price_reduce'].str.extract(r'(\d+)')
    df_shoes['price'] = df_shoes['price'].str.extract(r'(\d+)')

    df_shoes.drop(['price_'], axis=1, inplace=True)

    # Setting up the dataframe
    df_shoes['Brand'] = 'Adidas'
    df_shoes.reset_index(drop=True, inplace=True)
    
    return df_shoes

In [318]:
# Cleaning the data frame
df_shoes_adidas = clean_df_adidas(df_shoes_adidas)

# Saving the data frame
df_shoes_adidas.to_csv('datasets/Table1_adidas.csv')
display(df_shoes_adidas)

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand
0,Superstar XLG Shoes,https://www.adidas.com/us/superstar-xlg-shoes/...,12,88,110,WOMEN Originals,Adidas
1,Cloudfoam Pure Shoes,https://www.adidas.com/us/cloudfoam-pure-shoes...,2,45,75,Women's Essentials,Adidas
2,AdiFOM SST Boot Shoes,https://www.adidas.com/us/adifom-sst-boot-shoe...,3,44,110,Women's Originals,Adidas
3,Racer TR23 Shoes,https://www.adidas.com/us/racer-tr23-shoes/IF3...,11,56,80,Women's Sportswear,Adidas
4,Gazelle Indoor Shoes,https://www.adidas.com/us/gazelle-indoor-shoes...,2,105,150,WOMEN Originals,Adidas
...,...,...,...,...,...,...,...
1933,TERREX Soulstride Trail Running Shoes,https://www.adidas.com/us/terrex-soulstride-tr...,3,48,80,Men's TERREX,Adidas
1934,Adizero Afterburner NWV Cleats,https://www.adidas.com/us/adizero-afterburner-...,10,84,140,Men's Baseball,Adidas
1935,Stabil Next Gen Shoes,https://www.adidas.com/us/stabil-next-gen-shoe...,NaN,96,160,Men's Training,Adidas
1936,Tour360 22 Golf Shoes,https://www.adidas.com/us/tour360-22-golf-shoe...,7,126,180,Men's Golf,Adidas



### 2.1 Adidas details
#### Configuring the main parameters

In [788]:
# Defines the configuration to retrieve the data
config = {
    'xpath_show_reviews': '//h2[contains(text(),"Reviews")]',
    'xpath_more_reviews': '//h2[contains(text(),"Details")]',
    'xpath_market_esc': '//h3[contains(text(),"Your adiClub benefits await")]',
    'xpath_wait_for': ['//div[@data-auto-id="size-selector"]'],
    'xpath_reload_sizes': '//span[contains(text(),"Reload sizes")]',
    'n_scrolls': 8,
    'height_scroll': 200,
    'wait_time': 4,
}

def find_percentage(webelement):
    if webelement is None:
        return None
    next_tag = webelement.find_next('div', class_='gl-comparison-bar__indicator')
    style_txt = next_tag['style'] if next_tag is not None else None
    percentage_match = re.search(r'left:\s*([\d.-]+)%', style_txt)
    percentage_value = percentage_match.group(1) if percentage_match else 0
    return percentage_value

def find_color(webelement):
    if webelement is None:
        return None
    alt_txt = webelement.find('img')['alt'] if webelement.find('img') is not None else ''
    color_match = re.search(r'Product color:\s*(.*)', alt_txt)
    color = color_match.group(1) if color_match else ''
    return color

# Defines the attributes to recover with all the special configuration according to the property.
soup_config = { 
            'count_of_sizes':['button', {'class':'gl-label size___2lbev'}, 
                              None, 
                              None,  '-'], 
            'colors': ['a', {'class': re.compile(r'^variation___tBgJN')}, 
                       find_color,
                       None, ''],
            'color': ['div', {'class':"single-color-label___29kFh"},
                     None, None, ''],
            'product_code': ['li', {'class':"gl-vspace-bpall-small"}, 
                       None, 
                       lambda el: el.text.strip() and 'Product code:' in el.text.strip() , '-'],
            'reviews': ['h2', {}, 
                       None, 
                       lambda el: el.text.strip() and 'Reviews ' in el.text.strip(), '-'],
            'star_rating':['div',{'class':'ratings-label-container___13pr-'}, 
                       None,
                       None ,'-'],
            'Size':  ['div', {'class':"gl-comparison-bar__title"}, 
                       find_percentage, 
                       lambda el: 'Size' in el.text.strip(), '-'],
            'comfort': ['div', {'class':"gl-comparison-bar__title"}, 
                       find_percentage, 
                       lambda el: 'Comfort' in el.text.strip(), '-'],
            'quality': ['div', {'class':"gl-comparison-bar__title"}, 
                       find_percentage,
                       lambda el: 'Quality' in el.text.strip(), '-'],
        }

In [390]:
driver.quit()

In [789]:
config

{'xpath_show_reviews': '//h2[contains(text(),"Reviews")]',
 'xpath_more_reviews': '//h2[contains(text(),"Details")]',
 'xpath_market_esc': '//h3[contains(text(),"Your adiClub benefits await")]',
 'xpath_wait_for': ['//div[@data-auto-id="size-selector"]'],
 'xpath_reload_sizes': '//span[contains(text(),"Reload sizes")]',
 'n_scrolls': 8,
 'height_scroll': 200,
 'wait_time': 4}

#### 1st Batch - Adidas

In [377]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][:100]

# Scrape the first 100 records
df_det, url_error = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det.to_csv('datasets/det1_raw_adidas.csv')

Retrieving Details of record: 1/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/superstar-xlg-shoes/IG9777.html

Retrieving Details of record: 2/100
Wait time ** 6 ** 
Show reviews of 2
Click more reviews of 2
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/cloudfoam-pure-shoes/IG2530.html

Retrieving Details of record: 3/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 3
Click more reviews of 3
Wait for Exception of 2, xpath = //div[@data-auto-id="size-selector"]
Get the HTML code: https://www.adidas.com/us/adifom-sst-boot-shoes/IE4614.html

Retrieving Details of record: 4/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 4
Click more reviews of 4
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/racer-tr23-shoes/IF3330.html

Retrieving Detail

Retrieving Details of record: 34/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 34
Click more reviews of 34
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/copa-icon-firm-ground-soccer-cleats/GZ2528.html

Retrieving Details of record: 35/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 35
Click more reviews of 35
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/stan-smith-shoes/IE4634.html

Retrieving Details of record: 36/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 36
Click more reviews of 36
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/terrex-swift-r3-mid-gore-tex-hiking-shoes/IF2401.html

Retrieving Details of record: 37/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 37
Click more reviews of 37
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/nm

Wait time ** 6 ** 
Show reviews of 65
Click more reviews of 65
Wait for Exception of 64, xpath = //div[@data-auto-id="size-selector"]
Get the HTML code: https://www.adidas.com/us/miami-ultraboost-1.0-shoes/IG5877.html

Retrieving Details of record: 66/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 66
Click more reviews of 66
Wait for Exception of 65, xpath = //div[@data-auto-id="size-selector"]
Get the HTML code: https://www.adidas.com/us/barricade-tennis-shoes/ID1555.html

Retrieving Details of record: 67/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 67
Click more reviews of 67
Wait for Exception of 66, xpath = //div[@data-auto-id="size-selector"]
Get the HTML code: https://www.adidas.com/us/ultra-4d-running-shoes/IG2260.html

Retrieving Details of record: 68/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 68
Click more reviews of 68
Elelment loaded: size-selector
Get the HTML code: https:/

Retrieving Details of record: 95/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 95
Click more reviews of 95
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/y-3-superstar/IG3043.html

Retrieving Details of record: 96/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 96
Click more reviews of 96
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-gore-tex-2.0-hiking-shoes/HP7494.html

Retrieving Details of record: 97/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 97
Click more reviews of 97
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/ultraboost-1.0-atr-shoes/ID1747.html

Retrieving Details of record: 98/100
Selecting the market was not necessary.
Wait time ** 6 ** 
Show reviews of 98
Click more reviews of 98
Elelment loaded: size-selector
Get the HTML code: https://www.adidas.com/us/duramo-speed-r

In [378]:
url_error

,index,URL,Exception,msg
0,2,https://www.adidas.com/us/adifom-sst-boot-shoe...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,59,https://www.adidas.com/us/predator-accuracy-in...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,60,https://www.adidas.com/us/nmd_r1-shoes/GW5699....,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,61,https://www.adidas.com/us/adilette-shower-slid...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,62,https://www.adidas.com/us/copa-icon-firm-groun...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
5,63,https://www.adidas.com/us/adifom-sst-boot-shoe...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
6,64,https://www.adidas.com/us/miami-ultraboost-1.0...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
7,65,https://www.adidas.com/us/barricade-tennis-sho...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
8,66,https://www.adidas.com/us/ultra-4d-running-sho...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
9,68,https://www.adidas.com/us/supernova-2.0-runnin...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


In [385]:
# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

#### 2nd Batch - Adidas

In [395]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][100:200]

# Scrape the records from 100 to 200
df_det2, url_error2 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det2.to_csv('datasets/det2_raw_adidas.csv')

Retrieving Details of record: 1/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item Shmoofoil Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/shmoofoil-shoes/IG5266.html

Retrieving Details of record: 2/100
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Wait for Exception of 101, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 2
Get the HTML code: https://www.adidas.com/us/superstar-shoes/IF7610.html

Retrieving Details of record: 3/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item TERREX AX4 GORE-TEX Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-ax4-gore-tex-hiking-shoes/IF4863.html

Retrieving Details of record: 4/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Item NMD_R1 Shoes ha

Elelment loaded: size-selector
Item 4DFWD 2 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/4dfwd-2-running-shoes/GX9271.html

Retrieving Details of record: 27/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Elelment loaded: size-selector
Item Ultraboost Light Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-light-running-shoes/IG0747.html

Retrieving Details of record: 28/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Superstar PRIDE RM Shoes has not reviews.
Click more reviews of 28
Elelment loaded: size-selector
Item Superstar PRIDE RM Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/superstar-pride-rm-shoes/ID7493.html

Retrieving Details of record: 29/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 29
Click more reviews of 29
Elelment loaded: s

Show reviews of 52
Click more reviews of 52
Elelment loaded: size-selector
Item CODECHAOS 22 Spikeless Golf Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/codechaos-22-spikeless-golf-shoes/GX2613.html

Retrieving Details of record: 53/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item Dropset 2 Training Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/dropset-2-training-shoes/IF3197.html

Retrieving Details of record: 54/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item Avryn Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/avryn-shoes/HP5972.html

Retrieving Details of record: 55/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 55
Click more reviews of 55
Elelment loaded:

Elelment loaded: size-selector
Item Ultraboost Light Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-light-running-shoes/HQ1399.html

Retrieving Details of record: 79/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item Mississippi State Ultraboost 1.0 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/mississippi-state-ultraboost-1.0-shoes/IG5888.html

Retrieving Details of record: 80/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item Five Ten Sleuth Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-sleuth-mountain-bike-shoes/IG0374.html

Retrieving Details of record: 81/100
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 81
Click more revi

In [401]:
url_error2

,index,URL,Exception,msg
0,101,https://www.adidas.com/us/superstar-shoes/IF76...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,103,https://www.adidas.com/us/nmd_r1-shoes/IG8337....,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,108,https://www.adidas.com/us/ultraboost-light-run...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,113,https://www.adidas.com/us/superstar-millencon-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,116,https://www.adidas.com/us/adilette-platform-sl...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
5,166,https://www.adidas.com/us/nc-state-ultraboost-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
6,170,https://www.adidas.com/us/x_plrboost-shoes/ID9...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
7,171,https://www.adidas.com/us/indiana-ultraboost-1...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
8,172,https://www.adidas.com/us/georgia-tech-ultrabo...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
9,173,https://www.adidas.com/us/purehustle-2-elite-c...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


In [404]:
# Join the batches
df_det = pd.concat([df_det, df_det2])
url_error = pd.concat([url_error, url_error2])

# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

#### 3rd Batch - Adidas

In [536]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][200:350]

# Scrape from 200 to 350 records
df_det3, url_error3 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det3.to_csv('datasets/det3_raw_adidas.csv')

Retrieving Details of record: 1/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Ultraboost Light Running Shoes has not reviews.
Click more reviews of 1
Wait for Exception of 200, xpath = //div[@data-auto-id="size-selector"]
Item Ultraboost Light Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-light-running-shoes/IE1778.html

Retrieving Details of record: 2/150
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Wait for Exception of 201, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 2
Get the HTML code: https://www.adidas.com/us/superstar-xlg-shoes/IF8069.html

Retrieving Details of record: 3/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item adidas 4DFWD 2 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-4dfwd-2-running-shoes/HP7647.html

Retrieving De

Retrieving Details of record: 27/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Wait for Exception of 226, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 27
Get the HTML code: https://www.adidas.com/us/forum-mid-shoes/G57984.html

Retrieving Details of record: 28/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Elelment loaded: size-selector
Item Superstar Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/superstar-shoes/HQ1937.html

Retrieving Details of record: 29/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 29
Click more reviews of 29
Elelment loaded: size-selector
Item Avryn Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/avryn-shoes/IG0647.html

Retrieving Details of record: 30/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show review

Retrieving Details of record: 53/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Wait for Exception of 252, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 53
Get the HTML code: https://www.adidas.com/us/rapidmove-adv-training-shoes/IF3201.html

Retrieving Details of record: 54/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item Duramo SL 2.0 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/duramo-sl-2.0-running-shoes/GX0709.html

Retrieving Details of record: 55/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 55
Click more reviews of 55
Elelment loaded: size-selector
Item TERREX Swift R3 GORE-TEX Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-swift-r3-gore-tex-hiking-shoes/FW2779.html

Retrieving Details

Get the HTML code: https://www.adidas.com/us/forum-mid-shoes/IE7416.html

Retrieving Details of record: 79/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item Ultraboost 21 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-21-running-shoes/FY0404.html

Retrieving Details of record: 80/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item Superstar Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/superstar-shoes/IF7616.html

Retrieving Details of record: 81/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 81
Click more reviews of 81
Wait for Exception of 280, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 81
Get the HTML code: https://www.adidas.com/us/adizero-select-basketball-shoe

Wait time ** 4 ** 
Show reviews of 104
Click more reviews of 104
Elelment loaded: size-selector
Item Solarmotion BOA Golf Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/solarmotion-boa-golf-shoes/GV9394.html

Retrieving Details of record: 105/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item Dropset Training Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/dropset-training-shoes/GW3898.html

Retrieving Details of record: 106/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 106
Click more reviews of 106
Elelment loaded: size-selector
Item The Velosamba Made with Nature Cycling Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/the-velosamba-made-with-nature-cycling-shoes/IE7023.html

Retrieving Details of record: 107/150
Selecting the market was not necessary.
Wai

Reload sizes of 129
Get the HTML code: https://www.adidas.com/us/pureboost-22-running-shoes/GW0906.html

Retrieving Details of record: 130/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 130
Click more reviews of 130
Wait for Exception of 329, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 130
Get the HTML code: https://www.adidas.com/us/codechaos-laceless-golf-shoes/IG5358.html

Retrieving Details of record: 131/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 131
Click more reviews of 131
Elelment loaded: size-selector
Item Five Ten Hiangle Climbing Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-hiangle-climbing-shoes/EE9071.html

Retrieving Details of record: 132/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 132
Click more reviews of 132
Wait for Exception of 331, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 132
Get 

In [537]:
url_error3

,index,URL,Exception,msg
0,200,https://www.adidas.com/us/ultraboost-light-run...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,201,https://www.adidas.com/us/superstar-xlg-shoes/...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,203,https://www.adidas.com/us/nmd_r1-shoes/GZ4963....,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,204,https://www.adidas.com/us/superstar-xlg-shoes/...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,205,https://www.adidas.com/us/superstar-shoes/FX74...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
...,...,...,...,...
65,345,https://www.adidas.com/us/adilette-shower-slid...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
66,346,https://www.adidas.com/us/switch-fwd-running-s...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
67,347,https://www.adidas.com/us/five-ten-trailcross-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
68,348,https://www.adidas.com/us/hyperturf-adventure-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


In [544]:
# Join the batches
df_det = pd.concat([df_det, df_det3])
url_error = pd.concat([url_error, url_error3])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

#### 4thd Batch - Adidas

In [630]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][350:500]

# Scrape from 350 to 500 records
df_det4, url_error4 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det4.to_csv('datasets/det4_raw_adidas.csv')

Retrieving Details of record: 1/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item Five Ten Freerider Pro Canvas Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-freerider-pro-canvas-mountain-bike-shoes/HQ2123.html

Retrieving Details of record: 2/150
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Elelment loaded: size-selector
Item The Road Cycling Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/the-road-cycling-shoes/IE7018.html

Retrieving Details of record: 3/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item Five Ten Freerider Pro Canvas Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-freerider-pro-canvas-mountain-bike-shoes/HQ2124.html

Retri

Get the HTML code: https://www.adidas.com/us/stan-smith-lux-shoes/IG3389.html

Retrieving Details of record: 26/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 26
Click more reviews of 26
Elelment loaded: size-selector
Item X_PLRBOOST Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/x_plrboost-shoes/ID9443.html

Retrieving Details of record: 27/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Elelment loaded: size-selector
Item Five Ten Kestrel BOA sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-kestrel-boa/HQ6031.html

Retrieving Details of record: 28/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Elelment loaded: size-selector
Item Washington Rivalry Low Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/washington-rivalry-low-shoes/IE

Click more reviews of 51
Elelment loaded: size-selector
Item adidas by Stella McCartney Sportswear Shoe sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-by-stella-mccartney-sportswear-shoe/IG2441.html

Retrieving Details of record: 52/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 52
Click more reviews of 52
Elelment loaded: size-selector
Item Hyperturf Adventure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/hyperturf-adventure-shoes/HQ4303.html

Retrieving Details of record: 53/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item aSMC x TERREX HIKING BOOT sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/asmc-x-terrex-hiking-boot/IG0146.html

Retrieving Details of record: 54/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more r

Retrieving Details of record: 77/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Item NMD_S1 Shoes has not reviews.
Click more reviews of 77
Elelment loaded: size-selector
Item NMD_S1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_s1-shoes/HQ4465.html

Retrieving Details of record: 78/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 78
Click more reviews of 78
Elelment loaded: size-selector
Item Women's Summervent Spikeless Golf Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/womens-summervent-spikeless-golf-shoes/GZ3281.html

Retrieving Details of record: 79/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item adidas by Stella McCartney Winter Boots sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-by-stella-mccartney-winter-boots/ID9610.html

R

Get the HTML code: https://www.adidas.com/us/five-ten-hellcat-mountain-bike-shoes/FW4206.html

Retrieving Details of record: 103/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 103
Click more reviews of 103
Elelment loaded: size-selector
Item TERREX FREE HIKER 2 MWN sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2-mwn/HQ8914.html

Retrieving Details of record: 104/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 104
Click more reviews of 104
Elelment loaded: size-selector
Item TERREX Trail Rider GORE-TEX Trail Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-trail-rider-gore-tex-trail-running-shoes/HQ1238.html

Retrieving Details of record: 105/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item Superstar Shoes sizes do not need to be relo

Reload sizes of 115
Get the HTML code: https://www.adidas.com/us/the-road-boa-cycling-shoes/IE7021.html

Retrieving Details of record: 116/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 116
Click more reviews of 116
Elelment loaded: size-selector
Item Y-3 Slides sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/y-3-slides/HR1940.html

Retrieving Details of record: 117/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 117
Click more reviews of 117
Wait for Exception of 466, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 117
Get the HTML code: https://www.adidas.com/us/five-ten-hiangle-pro-competition-climbing-shoes/FV4744.html

Retrieving Details of record: 118/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 118
Click more reviews of 118
Wait for Exception of 467, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 118
Get the HTML code: https://w

Wait time ** 4 ** 
Show reviews of 141
Click more reviews of 141
Wait for Exception of 490, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 141
Get the HTML code: https://www.adidas.com/us/znsored-hi-shoes/IG0436.html

Retrieving Details of record: 142/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 142
Click more reviews of 142
Elelment loaded: size-selector
Item Web Boost Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/web-boost-shoes/HP3324.html

Retrieving Details of record: 143/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 143
Click more reviews of 143
Elelment loaded: size-selector
Item Avryn Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/avryn-shoes/HP5980.html

Retrieving Details of record: 144/150
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 144
Click more reviews of 144
Wait for Exception of 493, xpat

In [632]:
url_error4

,index,URL,Exception,msg
0,410,https://www.adidas.com/us/cloudfoam-pure-shoes...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,457,https://www.adidas.com/us/shmoofoil-shoes/IG52...,<class 'selenium.common.exceptions.ElementClic...,"element click intercepted: Element <h2 class=""..."
2,457,https://www.adidas.com/us/shmoofoil-shoes/IG52...,<class 'selenium.common.exceptions.ElementClic...,"element click intercepted: Element <h2 class=""..."
3,458,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.ElementClic...,"element click intercepted: Element <h2 class=""..."
4,458,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.ElementClic...,"element click intercepted: Element <h2 class=""..."
5,458,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
6,459,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.ElementClic...,"element click intercepted: Element <h2 class=""..."
7,459,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.ElementClic...,"element click intercepted: Element <h2 class=""..."
8,459,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
9,460,https://www.adidas.com/us/purehustle-2.0-cleat...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


In [633]:
# Join the batches
df_det = pd.concat([df_det, df_det4])
url_error = pd.concat([url_error, url_error4])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

#### 5thd Batch - Adidas

In [637]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][500:700]

# Scrape from 500 to 700 records
df_det5, url_error5 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det5.to_csv('datasets/det5_raw_adidas.csv')

Retrieving Details of record: 1/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item Adipower Weightlifting 3 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adipower-weightlifting-3-shoes/HP6846.html

Retrieving Details of record: 2/200
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Elelment loaded: size-selector
Item Mahomes 1 Impact FLX Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/mahomes-1-impact-flx-shoes/HQ2074.html

Retrieving Details of record: 3/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item Adimule Lea Slides sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adimule-lea-slides/GY2555.html

Retrieving Details of record: 4/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Sh

Show reviews of 26
Click more reviews of 26
Elelment loaded: size-selector
Item Five Ten Kirigami Climbing Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-kirigami-climbing-shoes/EE8935.html

Retrieving Details of record: 27/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Elelment loaded: size-selector
Item Superstar Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/superstar-shoes/IF7611.html

Retrieving Details of record: 28/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Elelment loaded: size-selector
Item Predator Accuracy.1 Firm Ground Soccer Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/predator-accuracy.1-firm-ground-soccer-cleats/GW4570.html

Retrieving Details of record: 29/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews

Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 52
Click more reviews of 52
Elelment loaded: size-selector
Item Cloudfoam Pure 2.0  Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/cloudfoam-pure-2.0--shoes/H00947.html

Retrieving Details of record: 53/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item Adilette Ayoon Slides sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adilette-ayoon-slides/IF7629.html

Retrieving Details of record: 54/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item adidas by Stella McCartney Seeulater Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-by-stella-mccartney-seeulater-shoes/ID1889.html

Retrieving Details of record: 55/200
Selecting the market

Get the HTML code: https://www.adidas.com/us/exhibit-select-mid-basketball-shoes/IE9327.html

Retrieving Details of record: 78/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 78
Click more reviews of 78
Elelment loaded: size-selector
Item Predator Accuracy.1 Firm Ground Soccer Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/predator-accuracy.1-firm-ground-soccer-cleats/GZ0032.html

Retrieving Details of record: 79/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item TERREX Skychaser Tech GORE-TEX Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-skychaser-tech-gore-tex-hiking-shoes/GV9034.html

Retrieving Details of record: 80/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item TERREX Free Hiker 

Retrieving Details of record: 103/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 103
Click more reviews of 103
Elelment loaded: size-selector
Item Solarmotion Spikeless Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/solarmotion-spikeless-shoes/GX4021.html

Retrieving Details of record: 104/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 104
Click more reviews of 104
Elelment loaded: size-selector
Item The Gravel Cycling Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/the-gravel-cycling-shoes/HP5400.html

Retrieving Details of record: 105/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item ZNTASY Capsule Collection Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/zntasy-capsule-collection-shoes/GZ2314.html

Retrieving Details of r

Get the HTML code: https://www.adidas.com/us/adistar-cs-2.0-running-shoes/HP9640.html

Retrieving Details of record: 129/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 129
Click more reviews of 129
Elelment loaded: size-selector
Item BYW Select Basketball Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/byw-select-basketball-shoes/IE9306.html

Retrieving Details of record: 130/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 130
Click more reviews of 130
Elelment loaded: size-selector
Item ZNSORED Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/znsored-shoes/HP5985.html

Retrieving Details of record: 131/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 131
Click more reviews of 131
Elelment loaded: size-selector
Item Court Team Bounce 2.0 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/court-t

Get the HTML code: https://www.adidas.com/us/runfalcon-3-running-shoes/ID2274.html

Retrieving Details of record: 154/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 154
Click more reviews of 154
Elelment loaded: size-selector
Item Y-3 Idoso Boost sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/y-3-idoso-boost/HQ5970.html

Retrieving Details of record: 155/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 155
Click more reviews of 155
Elelment loaded: size-selector
Item X_PLRBOOST Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/x_plrboost-shoes/HP3143.html

Retrieving Details of record: 156/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 156
Click more reviews of 156
Elelment loaded: size-selector
Item X_PLRPHASE Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/x_plrphase-shoes/ID2413.html

Retrievin

Retrieving Details of record: 179/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 179
Click more reviews of 179
Elelment loaded: size-selector
Item Pureboost 23 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/pureboost-23-running-shoes/IF2390.html

Retrieving Details of record: 180/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Item NMD_R1 Shoes has not reviews.
Click more reviews of 180
Wait for Exception of 679, xpath = //div[@data-auto-id="size-selector"]
Item NMD_R1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_r1-shoes/IE9624.html

Retrieving Details of record: 181/200
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Forum Low Shoes has not reviews.
Click more reviews of 181
Wait for Exception of 680, xpath = //div[@data-auto-id="size-selector"]
Item Forum Low Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.co

In [639]:
url_error5

,index,URL,Exception,msg
0,580,https://www.adidas.com/us/terrex-ax4-hiking-sh...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,581,https://www.adidas.com/us/duramo-speed-running...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,582,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,583,https://www.adidas.com/us/adicross-lo-boost-go...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,584,https://www.adidas.com/us/adilette-lite-slides...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
5,587,https://www.adidas.com/us/y-3-slides/HQ3665.html,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
6,588,https://www.adidas.com/us/terrex-trail-rider-g...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
7,594,https://www.adidas.com/us/rivalry-low-86-shoes...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
8,595,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
9,596,https://www.adidas.com/us/adi2000-shoes/GX4634...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


In [641]:
# Join the batches
df_det = pd.concat([df_det, df_det5])
url_error = pd.concat([url_error, url_error5])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

#### 6th Batch - Adidas

In [642]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][700:1000]

# Scrape from 700 to 1000 records
df_det6, url_error6 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det6.to_csv('datasets/det6_raw_adidas.csv')

Retrieving Details of record: 1/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item Superstar Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/superstar-shoes/ID4279.html

Retrieving Details of record: 2/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Cloudfoam Pure Shoes has not reviews.
Click more reviews of 2
Wait for Exception of 701, xpath = //div[@data-auto-id="size-selector"]
Item Cloudfoam Pure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/cloudfoam-pure-shoes/IG7376.html

Retrieving Details of record: 3/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item Adilette Comfort Slides sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adilette-comfort-slides/H03619.html

Retrieving Details of r

Reload sizes of 26
Get the HTML code: https://www.adidas.com/us/racer-tr23-shoes/IG7348.html

Retrieving Details of record: 27/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Wait for Exception of 726, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 27
Get the HTML code: https://www.adidas.com/us/solarcontrol-running-shoes/GY1657.html

Retrieving Details of record: 28/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Wait for Exception of 727, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 28
Get the HTML code: https://www.adidas.com/us/copa-pure.3-firm-ground-soccer-cleats/HQ8984.html

Retrieving Details of record: 29/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 29
Click more reviews of 29
Elelment loaded: size-selector
Item Solarmotion BOA Golf Shoes sizes do not need to be reload.
Get the HTML code: https://w

Get the HTML code: https://www.adidas.com/us/adilette-shower-slides/GZ5925.html

Retrieving Details of record: 53/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item Retropy E5 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/retropy-e5-shoes/IE7064.html

Retrieving Details of record: 54/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item Vulc Raid3r Skateboarding Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/vulc-raid3r-skateboarding-shoes/GX0872.html

Retrieving Details of record: 55/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Top Sala Competition Indoor Soccer Shoes has not reviews.
Click more reviews of 55
Wait for Exception of 754, xpath = //div[@data-auto-id="size-selector"]
Item Top Sala Competition Indoor 

Wait time ** 4 ** 
Show reviews of 78
Click more reviews of 78
Elelment loaded: size-selector
Item Adilette Sandals sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adilette-sandals/FY8649.html

Retrieving Details of record: 79/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item Adi2000 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adi2000-shoes/H03489.html

Retrieving Details of record: 80/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item TERREX Free Hiker COLD.RDY Hiking Boots sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-cold.rdy-hiking-boots/FU7224.html

Retrieving Details of record: 81/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 81
Click more reviews of 

Wait for Exception of 802, xpath = //div[@data-auto-id="size-selector"]
Item NMD_R1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_r1-shoes/FZ6000.html

Retrieving Details of record: 104/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item NMD_W1 Shoes has not reviews.
Click more reviews of 104
Wait for Exception of 803, xpath = //div[@data-auto-id="size-selector"]
Item NMD_W1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_w1-shoes/IE9594.html

Retrieving Details of record: 105/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item Duramo SL Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/duramo-sl-running-shoes/IF7870.html

Retrieving Details of record: 106/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Adilette Comfort Sandals has not revi

...

Elelment loaded: size-selector
Item TERREX Free Hiker 2.0 Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2.0-hiking-shoes/IF4921.html

Retrieving Details of record: 177/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 177
Click more reviews of 177
Elelment loaded: size-selector
Item Stan Smith Lux Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/stan-smith-lux-shoes/ID0985.html

Retrieving Details of record: 178/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 178
Click more reviews of 178
Elelment loaded: size-selector
Item X_PLRBOOST Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/x_plrboost-shoes/IF2921.html

Retrieving Details of record: 179/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 179
Click more reviews of 179
Wait for Exception of 878, xpath = //div[@data-

Wait time ** 4 ** 
Item Terrex Free Hiker 2.0 Low GORE-TEX Hiking Shoes has not reviews.
Click more reviews of 202
Elelment loaded: size-selector
Item Terrex Free Hiker 2.0 Low GORE-TEX Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2.0-low-gore-tex-hiking-shoes/IG5461.html

Retrieving Details of record: 203/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 203
Click more reviews of 203
Elelment loaded: size-selector
Item Ultraboost 1.0 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-1.0-shoes/ID9682.html

Retrieving Details of record: 204/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 204
Click more reviews of 204
Elelment loaded: size-selector
Item Y-3 Rivalry sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/y-3-rivalry/FZ6396.html

Retrieving Details of record: 205/300
Selecting the market 

Get the HTML code: https://www.adidas.com/us/adizero-adios-8-running-shoes/ID6902.html

Retrieving Details of record: 228/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 228
Click more reviews of 228
Elelment loaded: size-selector
Item TERREX Free Hiker 2.0 Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2.0-hiking-shoes/HQ8395.html

Retrieving Details of record: 229/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 229
Click more reviews of 229
Elelment loaded: size-selector
Item Terrex Free Hiker 2.0 Low GORE-TEX Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2.0-low-gore-tex-hiking-shoes/IG5460.html

Retrieving Details of record: 230/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 230
Click more reviews of 230
Elelment loaded: size-selector
Item TERREX SWIFT R3 GTX sizes do 

Retrieving Details of record: 253/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 253
Click more reviews of 253
Elelment loaded: size-selector
Item Terrex Free Hiker 2.0 Low GORE-TEX Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2.0-low-gore-tex-hiking-shoes/IG5459.html

Retrieving Details of record: 254/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 254
Click more reviews of 254
Elelment loaded: size-selector
Item Busenitz Vulc 2.0 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/busenitz-vulc-2.0-shoes/IG5246.html

Retrieving Details of record: 255/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 255
Click more reviews of 255
Elelment loaded: size-selector
Item Adifom Climacool Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adifom-climacool-shoes/IF3904.html

Ret

Get the HTML code: https://www.adidas.com/us/retrocross-spikeless-golf-shoes/GV6914.html

Retrieving Details of record: 279/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 279
Click more reviews of 279
Elelment loaded: size-selector
Item adidas Five Ten Freerider Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-five-ten-freerider-mountain-bike-shoes/HP9939.html

Retrieving Details of record: 280/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 280
Click more reviews of 280
Elelment loaded: size-selector
Item NMD_V3 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_v3-shoes/HQ3970.html

Retrieving Details of record: 281/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 281
Click more reviews of 281
Elelment loaded: size-selector
Item Stan Smith Lux Shoes sizes do not need to be reload.
Get the HTML code: htt

In [643]:
url_error6

,index,URL,Exception,msg
0,701,https://www.adidas.com/us/cloudfoam-pure-shoes...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,706,https://www.adidas.com/us/crazyflight-shoes/HR...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,709,https://www.adidas.com/us/adifom-trxn-shoes/IG...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,710,https://www.adidas.com/us/y-3-ajatu-run/HR1936...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,711,https://www.adidas.com/us/matchbreak-super-sho...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
...,...,...,...,...
78,962,https://www.adidas.com/us/predator-accuracy.1-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
79,963,https://www.adidas.com/us/rapidmove-training-s...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
80,964,https://www.adidas.com/us/five-ten-trailcross-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
81,970,https://www.adidas.com/us/terrex-free-hiker-2-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


In [646]:
# Join the batches
df_det = pd.concat([df_det, df_det6])
url_error = pd.concat([url_error, url_error6])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

#### 7th Batch - Adidas

In [792]:
# just in case something goes wrong
df_det_adidas_backup2 = df_det_adidas_backup.copy()

In [801]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][1000:1300]

# Scrape from 1000 to 1300 records
df_det7, url_error7 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det7.to_csv('datasets/det7_raw_adidas.csv')

Retrieving Details of record: 1/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item TERREX Free Hiker 2.0 Hiking Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2.0-hiking-shoes/HQ8398.html

Retrieving Details of record: 2/300
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Elelment loaded: size-selector
Item NMD_R1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_r1-shoes/GX9524.html

Retrieving Details of record: 3/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item Stan Smith Crepe Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/stan-smith-crepe-shoes/FZ6439.html

Retrieving Details of record: 4/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews o

Retrieving Details of record: 27/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Elelment loaded: size-selector
Item adizero 40 Football Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-40-football-cleats/GY4488.html

Retrieving Details of record: 28/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Elelment loaded: size-selector
Item Five Ten SLEUTH bike shoe sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-sleuth-bike-shoe/HQ1231.html

Retrieving Details of record: 29/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 29
Click more reviews of 29
Elelment loaded: size-selector
Item adidas Five Ten Freerider Canvas Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-five-ten-freerider-canvas-mountain-bike-shoes/HQ2

Wait time ** 4 ** 
Show reviews of 52
Click more reviews of 52
Elelment loaded: size-selector
Item Forum 84 Low Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/forum-84-low-shoes/FZ6293.html

Retrieving Details of record: 53/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item Adizero Boston 12 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-boston-12-running-shoes/H03612.html

Retrieving Details of record: 54/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item Hyperturf Adventure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/hyperturf-adventure-shoes/HQ9119.html

Retrieving Details of record: 55/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 55
Click more

Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 78
Click more reviews of 78
Elelment loaded: size-selector
Item Ultraboost Light Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-light-running-shoes/IG7963.html

Retrieving Details of record: 79/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Stan Smith PRIDE RM Shoes has not reviews.
Click more reviews of 79
Elelment loaded: size-selector
Item Stan Smith PRIDE RM Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/stan-smith-pride-rm-shoes/ID7494.html

Retrieving Details of record: 80/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item Solarglide 6 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/solarglide-6-running-shoes/HP7631.html

Retrieving Details of record: 81/300
Selec

Retrieving Details of record: 104/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 104
Click more reviews of 104
Elelment loaded: size-selector
Item NMD_R1 V2 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_r1-v2-shoes/HP9743.html

Retrieving Details of record: 105/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item Ultraboost 1.0 Texas A&M sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-1.0-texas-a-m/HQ5878.html

Retrieving Details of record: 106/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 106
Click more reviews of 106
Elelment loaded: size-selector
Item Ultraboost 1.0 Washington sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-1.0-washington/HQ5867.html

Retrieving Details of record: 107/300
Selecting the market 

...

Elelment loaded: size-selector
Item Adizero Ubersonic 4.1 Tennis Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-ubersonic-4.1-tennis-shoes/ID1562.html

Retrieving Details of record: 182/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 182
Click more reviews of 182
Elelment loaded: size-selector
Item OZWEEGO Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ozweego-shoes/ID9826.html

Retrieving Details of record: 183/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 183
Click more reviews of 183
Elelment loaded: size-selector
Item Box Hog 3 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/box-hog-3-shoes/FZ5306.html

Retrieving Details of record: 184/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 184
Click more reviews of 184
Elelment loaded: size-selector
Item Nizza Shoes sizes do not ne

Retrieving Details of record: 207/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 207
Click more reviews of 207
Elelment loaded: size-selector
Item X Crazyfast.1 Firm Ground Soccer Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/x-crazyfast.1-firm-ground-soccer-cleats/GY7416.html

Retrieving Details of record: 208/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 208
Click more reviews of 208
Elelment loaded: size-selector
Item Roverend Adventure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/roverend-adventure-shoes/GY1681.html

Retrieving Details of record: 209/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 209
Click more reviews of 209
Elelment loaded: size-selector
Item Adizero SL Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-sl-running-shoes/HQ1352.html

Retrieving De

Wait time ** 4 ** 
Show reviews of 232
Click more reviews of 232
Elelment loaded: size-selector
Item Tour360 22 Golf Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/tour360-22-golf-shoes/GV7247.html

Retrieving Details of record: 233/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 233
Click more reviews of 233
Elelment loaded: size-selector
Item Predator Accuracy+ Artificial Grass Soccer Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/predator-accuracy-plus-artificial-grass-soccer-cleats/IE9430.html

Retrieving Details of record: 234/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 234
Click more reviews of 234
Elelment loaded: size-selector
Item Predator Accuracy.1 Low Firm Ground Soccer Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/predator-accuracy.1-low-firm-ground-soccer-cleats/GW4575.html

Retrieving Details of r

Click more reviews of 257
Elelment loaded: size-selector
Item adidas by Stella McCartney Seeulater Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adidas-by-stella-mccartney-seeulater-shoes/ID1890.html

Retrieving Details of record: 258/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 258
Click more reviews of 258
Elelment loaded: size-selector
Item Switch FWD Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/switch-fwd-running-shoes/ID1780.html

Retrieving Details of record: 259/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 259
Click more reviews of 259
Elelment loaded: size-selector
Item Roverend Adventure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/roverend-adventure-shoes/GW2182.html

Retrieving Details of record: 260/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 260
Click m

Click more reviews of 282
Elelment loaded: size-selector
Item Made To Be Remade Ultraboost Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/made-to-be-remade-ultraboost-running-shoes/FV7827.html

Retrieving Details of record: 283/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 283
Click more reviews of 283
Elelment loaded: size-selector
Item Hyperturf Adventure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/hyperturf-adventure-shoes/HQ6605.html

Retrieving Details of record: 284/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 284
Click more reviews of 284
Elelment loaded: size-selector
Item Five Ten Freerider Pro Mid VCS Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-freerider-pro-mid-vcs-mountain-bike-shoes/H02024.html

Retrieving Details of record: 285/300
Selecting the market was not nec

In [802]:
url_error7

,index,URL,Exception,msg
0,1014,https://www.adidas.com/us/ultra-4dfwd-running-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,1038,https://www.adidas.com/us/4dfwd-2-running-shoe...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,1129,https://www.adidas.com/us/superstar-xlg-shoes/...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,1130,https://www.adidas.com/us/superstar-xlg-shoes/...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,1194,https://www.adidas.com/us/x_plrboost-shoes/ID9...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
5,1217,https://www.adidas.com/us/harden-vol.-7-basket...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
6,1229,https://www.adidas.com/us/orketro-shoes/GY2340...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


#### 8th Batch - Adidas

In [803]:
# sleeps the process for 30 mi
time.sleep(30*60)

In [804]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][1300:1600]

# Scrape from 1300 to 1600 records
df_det8, url_error8 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det8.to_csv('datasets/det8_raw_adidas.csv')

Retrieving Details of record: 1/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item Five Ten Trail Cross Mid Pro Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-trail-cross-mid-pro-mountain-bike-shoes/EF3010.html

Retrieving Details of record: 2/300
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Elelment loaded: size-selector
Item The Road Cycling Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/the-road-cycling-shoes/IE7018.html

Retrieving Details of record: 3/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item Ultrabounce Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultrabounce-running-shoes/HP5774.html

Retrieving Details of record: 4/300
Selecting the marke

Get the HTML code: https://www.adidas.com/us/ozmillen-shoes/IF4014.html

Retrieving Details of record: 27/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Elelment loaded: size-selector
Item Adizero Adios 7 Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-adios-7-running-shoes/GX6646.html

Retrieving Details of record: 28/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Elelment loaded: size-selector
Item Five Ten Freerider Pro Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-freerider-pro-mountain-bike-shoes/HQ1085.html

Retrieving Details of record: 29/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 29
Click more reviews of 29
Elelment loaded: size-selector
Item Impact FLX II Turf Training Shoes sizes do not need to be reload.
Get the

Get the HTML code: https://www.adidas.com/us/nebraska-rivalry-low-shoes/IE7700.html

Retrieving Details of record: 53/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item Ultraboost 1.0 x Disney 100 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ultraboost-1.0-x-disney-100-shoes/HQ6438.html

Retrieving Details of record: 54/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item Swift Run 22 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/swift-run-22-shoes/GY3047.html

Retrieving Details of record: 55/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 55
Click more reviews of 55
Elelment loaded: size-selector
Item Five Ten Hiangle Climbing Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas

Elelment loaded: size-selector
Item Icon 7 Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/icon-7-cleats/S23861.html

Retrieving Details of record: 79/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item NMD_G1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_g1-shoes/IG5782.html

Retrieving Details of record: 80/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item Response CL Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/response-cl-shoes/ID4593.html

Retrieving Details of record: 81/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 81
Click more reviews of 81
Elelment loaded: size-selector
Item Stan Smith Recon Shoes sizes do not need to be reload.
Get the HTML code: ht

Get the HTML code: https://www.adidas.com/us/terrex-free-hiker-2-mwn/HQ8914.html

Retrieving Details of record: 105/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item NMD_G1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_g1-shoes/IE4569.html

Retrieving Details of record: 106/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 106
Click more reviews of 106
Elelment loaded: size-selector
Item Alphaboost V1 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/alphaboost-v1-shoes/IE9974.html

Retrieving Details of record: 107/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 107
Click more reviews of 107
Elelment loaded: size-selector
Item Response CL Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/response-cl-shoes/IG3079.html

Retrievin

...

More-Review Exception of 1524
Wait for Exception of 1524, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 225
Get the HTML code: https://www.adidas.com/us/courtjam-control-tennis-shoes/ID1535.html

Retrieving Details of record: 226/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Review Exception of 1525-Message: element click intercepted: Element <h2 class="accordion-title___2sTgR">...</h2> is not clickable at point (158, 11). Other element would receive the click: <section class="content-item___3yeDy content-visibility-auto full-width-never___3eH3H accordion-content-item___27JBt" style="contain-intrinsic-size: 890px 88px;">...</section>
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x00007FF74DAC82B2+55298]
	(No symbol) [0x00007FF74DA35E02]
	(No symbol) [0x00007FF74D8F05AB]
	(No symbol) [0x00007FF74D937A77]
	(No symbol) [0x00007FF74D935E39]
	(No symbol) [0x00007FF74D933C08]
	(No symbol) [0x00007FF74D932C8A]
	(No symbol) [0x00007FF74

More-Review Exception of 1529
Elelment loaded: size-selector
Item Harden Volume 7 Basketball Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/harden-volume-7-basketball-shoes/HQ3425.html

Retrieving Details of record: 231/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Review Exception of 1530-Message: element click intercepted: Element <h2 class="accordion-title___2sTgR">...</h2> is not clickable at point (158, 11). Other element would receive the click: <section class="content-item___3yeDy content-visibility-auto full-width-never___3eH3H accordion-content-item___27JBt" style="contain-intrinsic-size: 890px 88px;">...</section>
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x00007FF74DAC82B2+55298]
	(No symbol) [0x00007FF74DA35E02]
	(No symbol) [0x00007FF74D8F05AB]
	(No symbol) [0x00007FF74D937A77]
	(No symbol) [0x00007FF74D935E39]
	(No symbol) [0x00007FF74D933C08]
	(No symbol) [0x00007FF74D932C8A]
	(No symbol) [

Click more reviews of 243
Wait for Exception of 1542, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 243
Get the HTML code: https://www.adidas.com/us/nmd_g1-shoes/ID7958.html

Retrieving Details of record: 244/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 244
Click more reviews of 244
Wait for Exception of 1543, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 244
Get the HTML code: https://www.adidas.com/us/nmd_r1-shoes/IE4694.html

Retrieving Details of record: 245/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 245
Click more reviews of 245
Elelment loaded: size-selector
Item OZELIA Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/ozelia-shoes/IE2000.html

Retrieving Details of record: 246/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Lite Racer Adapt 4.0 Shoes has not reviews.
Click more reviews of 246
Wait for Exception of 1545, xpath = 

Reload sizes of 268
Get the HTML code: https://www.adidas.com/us/x-crazyfast.1-laceless-firm-ground-soccer-cleats/GY7378.html

Retrieving Details of record: 269/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 269
Click more reviews of 269
Wait for Exception of 1568, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 269
Get the HTML code: https://www.adidas.com/us/rivalry-86-low-001-shoes/ID8252.html

Retrieving Details of record: 270/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 270
Click more reviews of 270
Wait for Exception of 1569, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 270
Get the HTML code: https://www.adidas.com/us/znsored-hi-shoes/GZ2291.html

Retrieving Details of record: 271/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 271
Click more reviews of 271
Wait for Exception of 1570, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 271
Get the

Get the HTML code: https://www.adidas.com/us/top-10-2000-shoes/HQ4622.html

Retrieving Details of record: 294/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Superstar 82 Shoes has not reviews.
Click more reviews of 294
Elelment loaded: size-selector
Item Superstar 82 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/superstar-82-shoes/ID2151.html

Retrieving Details of record: 295/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 295
Click more reviews of 295
Elelment loaded: size-selector
Item TERREX MADE TO BE REMADE FREE HIKER sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-made-to-be-remade-free-hiker/S29049.html

Retrieving Details of record: 296/300
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 296
Click more reviews of 296
Elelment loaded: size-selector
Item Rivalry Low 86 Shoes sizes do not need to be reload.
Get the HTML code: http

In [805]:
url_error8

,index,URL,Exception,msg
0,1433,https://www.adidas.com/us/adidas-by-stella-mcc...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
1,1434,https://www.adidas.com/us/predator-accuracy.3-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
2,1435,https://www.adidas.com/us/nasty-2.0-cleats/GX7...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
3,1436,https://www.adidas.com/us/avryn-shoes/HP5968.html,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
4,1437,https://www.adidas.com/us/ultrabounce-running-...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
...,...,...,...,...
84,1570,https://www.adidas.com/us/ultraboost-1.0-shoes...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
85,1572,https://www.adidas.com/us/courtjam-control-ten...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
86,1573,https://www.adidas.com/us/znsored-shoes/HP5988...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...
87,1574,https://www.adidas.com/us/superstar-supermodif...,<class 'selenium.common.exceptions.TimeoutExce...,<function element_to_be_clickable.<locals>._pr...


#### 9th Batch - Adidas

In [806]:
# sleeps the process for 30 mi
time.sleep(30*60)

In [ ]:
%%time

# Defines the urls 
df_urls = df_shoes_adidas[['ShoeName', 'URL']][1600:]

# Scrape from 1600 to 1938 records
df_det9, url_error9 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det9.to_csv('datasets/det9_raw_adidas.csv')

Retrieving Details of record: 1/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 1
Click more reviews of 1
Elelment loaded: size-selector
Item Adilette Comfort Slides sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adilette-comfort-slides/GZ6412.html

Retrieving Details of record: 2/338
Wait time ** 4 ** 
Show reviews of 2
Click more reviews of 2
Elelment loaded: size-selector
Item Five Ten Trail Cross Mid Pro Mountain Bike Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-trail-cross-mid-pro-mountain-bike-shoes/GY5121.html

Retrieving Details of record: 3/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 3
Click more reviews of 3
Elelment loaded: size-selector
Item adizero Ubersonic 4 Clay Court Tennis Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-ubersonic-4-clay-court-tennis-shoes/HQ5930.html

Retrieving Detail

Retrieving Details of record: 27/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 27
Click more reviews of 27
Elelment loaded: size-selector
Item Icon 7 BOOST sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/icon-7-boost/S23851.html

Retrieving Details of record: 28/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 28
Click more reviews of 28
Elelment loaded: size-selector
Item Five Ten NIAD Moccasym Climbing Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/five-ten-niad-moccasym-climbing-shoes/FW2853.html

Retrieving Details of record: 29/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 29
Click more reviews of 29
Elelment loaded: size-selector
Item Oznova Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/oznova-shoes/HQ1970.html

Retrieving Details of record: 30/338
Selecting the market was not necessar

Get the HTML code: https://www.adidas.com/us/country-xlg-shoes/ID0364.html

Retrieving Details of record: 53/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 53
Click more reviews of 53
Elelment loaded: size-selector
Item Adizero Afterburner 8 Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/adizero-afterburner-8-cleats/H00978.html

Retrieving Details of record: 54/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 54
Click more reviews of 54
Elelment loaded: size-selector
Item Forum Mid Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/forum-mid-shoes/GY0005.html

Retrieving Details of record: 55/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 55
Click more reviews of 55
Elelment loaded: size-selector
Item TERREX Free Hiker 2 Hiking Shoe sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/terrex-free-hiker

Show reviews of 78
Click more reviews of 78
Elelment loaded: size-selector
Item Mocaturf Adventure Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/mocaturf-adventure-shoes/GY9853.html

Retrieving Details of record: 79/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 79
Click more reviews of 79
Elelment loaded: size-selector
Item Harden Volume 7 Basketball Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/harden-volume-7-basketball-shoes/HQ3424.html

Retrieving Details of record: 80/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 80
Click more reviews of 80
Elelment loaded: size-selector
Item Freak 23 Mismatch Football Cleats sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/freak-23-mismatch-football-cleats/GW4167.html

Retrieving Details of record: 81/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 81

Wait time ** 4 ** 
Item Stan Smith Recon Shoes has not reviews.
Click more reviews of 104
Wait for Exception of 1703, xpath = //div[@data-auto-id="size-selector"]
Item Stan Smith Recon Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/stan-smith-recon-shoes/IH0019.html

Retrieving Details of record: 105/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 105
Click more reviews of 105
Elelment loaded: size-selector
Item Duramo SL Running Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/duramo-sl-running-shoes/G58108.html

Retrieving Details of record: 106/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 106
Click more reviews of 106
Elelment loaded: size-selector
Item Impact FLX II Turf Training Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/impact-flx-ii-turf-training-shoes/IE9376.html

Retrieving Details of record: 107/338
Sele

Get the HTML code: https://www.adidas.com/us/lite-racer-adapt-4.0-cloudfoam-lifestyle-slip-on-shoes/GX4776.html

Retrieving Details of record: 129/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 129
Click more reviews of 129
Wait for Exception of 1728, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 129
Get the HTML code: https://www.adidas.com/us/znsored-hi-shoes/IE9415.html

Retrieving Details of record: 130/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 130
Click more reviews of 130
Wait for Exception of 1729, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 130
Get the HTML code: https://www.adidas.com/us/alphaboost-v1-shoes/HQ4517.html

Retrieving Details of record: 131/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 131
Click more reviews of 131
Wait for Exception of 1730, xpath = //div[@data-auto-id="size-selector"]
Reload sizes of 131
Get the HTML code: https:/

Retrieving Details of record: 153/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Item Matchbreak Super Shoes has not reviews.
Click more reviews of 153
Wait for Exception of 1752, xpath = //div[@data-auto-id="size-selector"]
Item Matchbreak Super Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/matchbreak-super-shoes/EG2740.html

Retrieving Details of record: 154/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 154
Click more reviews of 154
Elelment loaded: size-selector
Item NMD_V3 Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/nmd_v3-shoes/FZ6498.html

Retrieving Details of record: 155/338
Selecting the market was not necessary.
Wait time ** 4 ** 
Show reviews of 155
Click more reviews of 155
Elelment loaded: size-selector
Item Solarmotion Spikeless Shoes sizes do not need to be reload.
Get the HTML code: https://www.adidas.com/us/solarmotion-spikeless-shoes/GX6424.html

In [808]:
url_error9

,index,URL,Exception,msg


In [810]:
driver.quit()

In [ ]:
# PENDIENTE JOIN de

In [ ]:
# Join the batches
df_det = pd.concat([df_det, df_det7, df_det8, df_det9 ])
url_error = pd.concat([url_error, url_error7, url_error8, url_error9 ])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_adidas_backup = df_det.copy()

# Save a file with the original data recovered
df_det_adidas_backup.to_csv('datasets/det_raw_adidas.csv')
url_error.to_csv('errors/adidas_batch_err.csv') 

In [399]:
filtered_df = df_det2[df_det2['count_of_sizes'].apply(lambda x: len(x) == 0)]
filtered_df

,count_of_sizes,colors,color,product_code,reviews,star_rating,Size,comfort,quality
3,[],"[Core Black / Off White / Grey Six, Core Black...",[],[Product code: IG8337],[],[],[],[],[]
13,[],"[Wonder Beige / Wonder Beige / Silver Pebble, ...",[],[Product code: ID4263],[Reviews (15)],[3.9],[53.3333],[78.3333],[68.3333]
16,[],"[Core Black / Core Black / Core Black, Silver ...",[],[Product code: HQ6179],[],[],[],[],[]
66,[],[Team Colleg Gold 2 / Team Colleg Gold 2 / Tea...,[],[Product code: IG5878],[Reviews (289)],[4.8],[51.6436],[90.8304],[93.7716]
70,[],"[Halo Blue / Blue Dawn / Solar Red, Carbon / O...",[],[Product code: ID9584],[Reviews (157)],[4.4],[62.7389],[86.9427],[88.8535]
71,[],[Team Colleg Gold 2 / Team Colleg Gold 2 / Tea...,[],[Product code: IG5880],[Reviews (294)],[4.8],[51.3605],[90.5612],[93.7075]
72,[],[Team Colleg Gold 2 / Team Colleg Gold 2 / Tea...,[],[Product code: IG5887],[Reviews (289)],[4.8],[51.6436],[90.8304],[93.7716]
73,[],[],[Cloud White / Core Black / Cinder],[Product code: FY4393],[Reviews (1)],[4.0],[50],[100],[100]
74,[],"[Core Black / Off White / Grey Six, Core Black...",[],[Product code: GZ9593],[Reviews (3739)],[4.4],[59.8059],[78.8487],[85.9722]
81,[],[Wonder Blue F23 / Core Black / Lucid Lemon F2...,[],[Product code: ID8473],[Reviews (30)],[4.5],[57.5],[80.8333],[83.3333]


In [327]:
df_shoes_adidas[df_shoes_adidas['no_colors'].isna()]

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand
35,TERREX Swift R3 Mid GORE-TEX Hiking Shoes,https://www.adidas.com/us/terrex-swift-r3-mid-...,NaN,72,180,Women's TERREX,Adidas
54,Adizero SL Running Shoes,https://www.adidas.com/us/adizero-sl-running-s...,NaN,NaN,NaN,Women's Running,Adidas
59,Predator Accuracy Injection+ Firm Ground Socce...,https://www.adidas.com/us/predator-accuracy-in...,NaN,225,300,WOMEN Soccer,Adidas
69,Predator Accuracy.1 Indoor Soccer Shoes,https://www.adidas.com/us/predator-accuracy.1-...,NaN,105,140,WOMEN Soccer,Adidas
71,TERREX Swift R3 Hiking Shoes,https://www.adidas.com/us/terrex-swift-r3-hiki...,NaN,60,150,Women's TERREX,Adidas
...,...,...,...,...,...,...,...
1924,Racer TR Slides,https://www.adidas.com/us/racer-tr-slides/G581...,NaN,25,35,MEN Yoga,Adidas
1925,Predator Edge.3 Firm Ground Soccer Cleats,https://www.adidas.com/us/predator-edge.3-firm...,NaN,72,90,MEN Soccer,Adidas
1931,Freak 22 Big Mood Cleats,https://www.adidas.com/us/freak-22-big-mood-cl...,NaN,60,100,Men's Football,Adidas
1932,Predator Edge.3 Firm Ground Soccer Cleats,https://www.adidas.com/us/predator-edge.3-firm...,NaN,72,90,MEN Soccer,Adidas


In [787]:
new_columns = df_det['colors'].apply(process_colors)
# df = pd.concat([df, new_columns], axis=1)
# display(df)
display(new_columns)

,color_black,color_blue,color_white,color_red,other_color
0,1.0,1.0,1.0,1.0,green
1,1.0,1.0,1.0,1.0,brown
2,1.0,0.0,1.0,0.0,green
3,1.0,1.0,1.0,1.0,pink
4,0.0,1.0,0.0,1.0,green
...,...,...,...,...,...
995,1.0,1.0,1.0,1.0,green
996,1.0,1.0,1.0,1.0,green
997,1.0,1.0,1.0,1.0,NaN
998,0.0,0.0,0.0,0.0,NaN


## 3. Puma.com

In [407]:
sites['puma']

'https://us.puma.com/us/en'

In [411]:
# Open the Driver 
driver = webdriver.Chrome()
url = sites['puma']
driver.get(url) 

driver.maximize_window()

wait_time = 3
time.sleep(wait_time)

# Removing the cookies pop up
popup_xpath = '//div[@data-test-id="cookie-banner"]/button'
remove_popup(popup_xpath)

Pop-up removed


In [412]:
%%time
# Scrap the main data for Men and Women in nike.com
# Initialization of the dataframe
df_shoes_puma = pd.DataFrame()
categories = {'Women': 'Footwear', 'Men': 'Shoes'}

for category, value in categories.items():
    
    # Follows the path to the category specified
    path_puma = {'msg': [f'\n{category} section...\n',
                         'Get 20% off clicked...\n',
                         'Category clicked...\n', 
                         'Shoes selected...\n', 
                         'Closing category menu...\n'],
                 'xpaths': [f'//a[@data-link-name="{category}"]',
                            '//button[@data-test-id="newsletter-sign-up-form-button"]/div',
                            '//button[@aria-controls="productdivName-filter-dropdown"]',
                            f'//label[@for="{value}"]',
                            '//button[@data-test-id="close-btn"]'
                            ],
                'wait_time': [ 5, 5, 8, 4, 4, 1]}

    follow_path_to(path_puma) 
    
    wait_time = 5
    time.sleep(wait_time)

    # Find out the number of shoes to retrieve in the category
    n_shoes = n_shoes_to_scrap('//span[@data-test-id="product-results"]')
    print(f"Shoes to recover: {n_shoes}\n" )

    # Define the xpath for shoes card 
    xpath = '//li[@data-test-id="product-list-item"]'

    # Scroll until recovered all shoes
    recovered_shoes, _ = scroll_to_recovered_shoes(n_shoes, xpath)
    print(f"\nTotal of shoes recovered: {recovered_shoes}\n")

    print('*'*50)
    print(' '*5, f'S C R A P I N G - Category: {category}')
    print('*'*50)

    # Scraping ShoeName ,Category(Men/Women), no.of colors, price
    xpath_grp = '//li[@data-test-id="product-list-item"]'
    soup_config = {
        'ShoeName':['h3',{}, None, None, '-'], 
        'URL': ['a', {'data-test-id': 'product-list-item-link'}, 'href', None, '-'],
        'no_colors': ['div', {'class':"relative h-12 mobile:h-10 flex items-center group"}, None, None, '-'],
        'price_reduce': ['span', {'data-test-id': 'sale-price'}, None, None, '-' ],
        'price': ['span', {'data-test-id':"price"},  None, None, '-'],
    }
    # time.sleep(wait_time)
    df_shoes_cat = scrape_items_by_group(driver, xpath_grp, **soup_config)

    # Setting the category
    df_shoes_cat['Category'] = category

    # Append the new data 
    df_shoes_puma = pd.concat([df_shoes_puma, df_shoes_cat])

print( f'Products in category {category} retrieved successfully: {len(df_shoes_puma)}' )

Wait time ** 5 **

Women section...

Wait time ** 5 **
Get 20% off clicked...

Wait time ** 8 **
Category clicked...

Wait time ** 4 **
Shoes selected...

Wait time ** 4 **
Closing category menu...

Shoes to recover: 234

Recovered shoes: 24
Scrolling 1 times...
Recovered shoes: 48, still to recovered: 186
Scrolling 2 times...
Recovered shoes: 72, still to recovered: 162
Scrolling 3 times...
Recovered shoes: 96, still to recovered: 138
Scrolling 4 times...
Recovered shoes: 120, still to recovered: 114
Scrolling 5 times...
Recovered shoes: 144, still to recovered: 90
Scrolling 6 times...
Recovered shoes: 168, still to recovered: 66
Scrolling 7 times...
Recovered shoes: 192, still to recovered: 42
Scrolling 8 times...
Recovered shoes: 216, still to recovered: 18
Scrolling 9 times...
Recovered shoes: 234, still to recovered: 0

Total of shoes recovered: 234

**************************************************
      S C R A P I N G - Category: Women
*****************************************

In [414]:
# Free Driver
driver.quit()

# Backup dataframe detail brand Puma
df_puma_backup = df_shoes_puma.copy()

# Save a file with the original data recovered
df_puma_backup.to_csv('datasets/raw_puma.csv')

In [431]:
# Cleaning the data frame
def clean_df_puma(df_shoes):
    df_shoes['no_colors'] = df_shoes['no_colors'].str.extract(r'(\d+)')
    
    df_shoes['price_reduce'] = df_shoes['price_reduce'].str.extract(r'\$(\d+(\.\d+)?)')[0]
    df_shoes['price'] = df_shoes['price'].str.extract(r'\$(\d+(\.\d+)?)')[0]
    df_shoes['price_reduce'].fillna(df_shoes['price'], inplace=True)
    
    df_shoes['URL'] = 'https://us.puma.com'+ df_shoes['URL']

    # Setting up the dataframe
    df_shoes['Brand'] = 'Puma'
    df_shoes.reset_index(drop=True, inplace=True)
    
    return df_shoes

In [432]:
# Cleaning the data frame
df_shoes_puma = clean_df_puma(df_shoes_puma)

# Saving the data frame
df_shoes_puma.to_csv('datasets/Table1_puma.csv')
display(df_shoes_puma)

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand
0,FENTY x PUMA Creeper Phatty Women's SneakersPU...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma
1,FENTY x PUMA Creeper Phatty Women's SneakersLa...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma
2,FENTY x PUMA Creeper Phatty Women's SneakersSp...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma
3,Basket Classic Velvet Women's SneakersMalachit...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,64.99,80.00,Women,Puma
4,Basket Classic Velvet Women's SneakersDark Jas...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,64.99,80.00,Women,Puma
...,...,...,...,...,...,...,...
623,Suede Mid XXI SneakersQuarry-Puma White,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma
624,Suede Mid XXI SneakersPumpkin Pie-PUMA White,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma
625,PUMA x PLEASURES Suede XL Men's SneakersPUMA B...,https://us.puma.com/us/en/pd/puma-x-pleasures-...,1,120.00,120.00,Men,Puma
626,evoSPEED Long Jump 11 UltraweaveSun Stream-Sun...,https://us.puma.com/us/en/pd/evospeed-long-jum...,1,140.00,140.00,Men,Puma


### 3.1 Puma Details
#### Configuring the main parameters

In [567]:
# Defines the configuration to retrieve the data
config = {
#     'xpath_remove_popup': '//div[@data-test-id="cookie-banner"]/button',
    'xpath_show_reviews': '//button[@data-test-id="read-all-reviews"]',
    'n_scrolls': 4,
    'height_scroll': 400,
    'wait_time': 4
}

def find_quality(webelement):
    if webelement is None:
        return None
    percentage_match = re.search(r'(\d+\.\d+)% recommend this product', webelement.text.strip() )
    percentage_value = percentage_match.group(1) if percentage_match else 0
    return percentage_value

soup_config = {
            'count_of_sizes':['span',{'data-content':'size-value'}, None, None,'-'], 
            'colors': ['label', {'data-test-id': 'color'}, None, None, '-'],
            'product_code': ['li', {}, None, lambda el: 'Style: ' in el.text if el is not None else '', '-'],
            'reviews': ['div', {'id' : "product-reviews"}, None, None, '-'],
            'quality':['p',{'data-test-id':'percentRecommendThisProduct'}, find_quality, None, '-'], 
        }

#### 1st Batch - Puma

In [464]:
driver.quit()

In [502]:
%%time

# Defines the urls 
df_urls = df_shoes_puma[['ShoeName', 'URL']][:150]

# Scrape the first 150 records
df_det, url_error = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det.to_csv('datasets/det1_raw_puma.csv')

Retrieving Details of record: 1/150
Wait time ** 4 ** 
Show reviews of 1
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 2/150
Wait time ** 4 ** 
Show reviews of 2
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332?swatch=03&referrer-category=womens-shop-all-womens

Retrieving Details of record: 3/150
Wait time ** 4 ** 
Show reviews of 3
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-creeper-phatty-womens-sneakers/399332?swatch=02&referrer-category=womens-shop-all-womens

Retrieving Details of record: 4/150
Wait time ** 4 ** 
Item Basket Classic Velvet Women's SneakersMalachite-Puma Team Gold-PUMA White has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/basket-classic-velvet-womens-sneakers/398949?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 5/

Retrieving Details of record: 31/150
Wait time ** 4 ** 
Show reviews of 31
Get the HTML code: https://us.puma.com/us/en/pd/fenty-x-puma-avanti-l-womens-sneakers/399262?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 32/150
Wait time ** 4 ** 
Show reviews of 32
Get the HTML code: https://us.puma.com/us/en/pd/slipstream-iridescent-womens-sneaker%C2%A0/392886?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 33/150
Wait time ** 4 ** 
Item Blacktop Rider Glimmer Women's SneakersFrosted Ivory-Warm White-Cashew has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/blacktop-rider-glimmer-womens-sneakers/397090?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 34/150
Wait time ** 4 ** 
Item CA Pro Suede Nubuck SneakersSedate Gray-PUMA White has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/ca-pro-suede-nubuck-sneakers/396704?swatch=02&referrer-category=womens-shop-all-wom

Get the HTML code: https://us.puma.com/us/en/pd/puma-cali-court-leather-womens-sneakers/393802?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 64/150
Wait time ** 4 ** 
Show reviews of 64
Get the HTML code: https://us.puma.com/us/en/pd/foreverrun-nitro%E2%84%A2-womens-running-shoes/377758?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 65/150
Wait time ** 4 ** 
Item KING ULTIMATE FG/AG Women's Soccer CleatsAlpine Snow-Asphalt-Yellow Blaze has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/king-ultimate-fg-ag-womens-soccer-cleats/107565?swatch=03&referrer-category=womens-shop-all-womens

Retrieving Details of record: 66/150
Wait time ** 4 ** 
Show reviews of 66
Get the HTML code: https://us.puma.com/us/en/pd/palermo-womens-leather-sneakers/397647?swatch=03&referrer-category=womens-shop-all-womens

Retrieving Details of record: 67/150
Wait time ** 4 ** 
Show reviews of 67
Get the HTML code: https://us.puma.

Get the HTML code: https://us.puma.com/us/en/pd/puma-x-ciele-deviate-nitro%E2%84%A2-2-womens-running-shoes/378437?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 97/150
Wait time ** 4 ** 
Show reviews of 97
Get the HTML code: https://us.puma.com/us/en/pd/palermo-sneakers/396463?swatch=09&referrer-category=womens-shop-all-womens

Retrieving Details of record: 98/150
Wait time ** 4 ** 
Show reviews of 98
Get the HTML code: https://us.puma.com/us/en/pd/puma-180-prm-womens-sneakers/393764?swatch=03&referrer-category=womens-shop-all-womens

Retrieving Details of record: 99/150
Wait time ** 4 ** 
Show reviews of 99
Get the HTML code: https://us.puma.com/us/en/pd/magnify-nitro%E2%84%A2-2-womens-running-shoes/377540?swatch=05&referrer-category=womens-shop-all-womens

Retrieving Details of record: 100/150
Wait time ** 4 ** 
Review Exception of 99-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19

Get the HTML code: https://us.puma.com/us/en/pd/bmw-m-motorsport-rs-puls-womens-sneakers/307761?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 124/150
Wait time ** 4 ** 
Show reviews of 124
Get the HTML code: https://us.puma.com/us/en/pd/seasons-fast-trac-nitro%E2%84%A2-2-womens-running-shoes/307685?swatch=04&referrer-category=womens-shop-all-womens

Retrieving Details of record: 125/150
Wait time ** 4 ** 
Review Exception of 124-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19fr0v0 tw-ozwx86 flex-row after::absolute after::hidden after::pointer-events-none after::-left-1 after::-top-1 after::-right-1 after::-bottom-1 after::border-2 after::border-outline after::rounded-sm tw-10skrdm self-end">...</button> is not clickable at point (1457, 58). Other element would receive the click: <div class="relative">...</div>
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0

Get the HTML code: https://us.puma.com/us/en/pd/fuse-2-0-womens-training-shoes/376169?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 143/150
Wait time ** 4 ** 
Show reviews of 143
Get the HTML code: https://us.puma.com/us/en/pd/fast-r-nitro%E2%84%A2-elite-womens-running-shoes/376311?swatch=04&referrer-category=womens-shop-all-womens

Retrieving Details of record: 144/150
Wait time ** 4 ** 
Review Exception of 143-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19fr0v0 tw-ozwx86 flex-row after::absolute after::hidden after::pointer-events-none after::-left-1 after::-top-1 after::-right-1 after::-bottom-1 after::border-2 after::border-outline after::rounded-sm tw-10skrdm self-end">...</button> is not clickable at point (1457, 58). Other element would receive the click: <div class="relative">...</div>
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x00007FF74DAC82B2

In [504]:
url_error

,index,URL,Exception,msg
0,27,https://us.puma.com/us/en/pd/rs-x-soft-womens-...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
1,62,https://us.puma.com/us/en/pd/puma-cali-court-l...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
2,99,https://us.puma.com/us/en/pd/puma-180-prm-wome...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
3,122,https://us.puma.com/us/en/pd/bmw-m-motorsport-...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
4,124,https://us.puma.com/us/en/pd/seasons-fast-trac...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
5,127,https://us.puma.com/us/en/pd/seasons-voyage-ni...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
6,141,https://us.puma.com/us/en/pd/fuse-2-0-womens-t...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
7,143,https://us.puma.com/us/en/pd/seasons-voyage-ni...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...


In [505]:
# Backup dataframe detail brand Puma
df_det_puma_backup = df_det.copy()

# Save a file with the original data recovered
df_det_puma_backup.to_csv('datasets/det_raw_puma.csv')

#### 2nd Batch - Puma

In [506]:
%%time

# Defines the urls 
df_urls = df_shoes_puma[['ShoeName', 'URL']][150:350]

# Scrape the records from 100 to 200
df_det2, url_error2 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det2.to_csv('datasets/det2_raw_puma.csv')

Retrieving Details of record: 1/200
Wait time ** 4 ** 
Item STEWIE x TEAM Stewie 2 Women's Basketball ShoesPlatinum Gray-PUMA White has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/stewie-x-team-stewie-2-womens-basketball-shoes/379082?swatch=06&referrer-category=womens-shop-all-womens

Retrieving Details of record: 2/200
Wait time ** 4 ** 
Show reviews of 2
Get the HTML code: https://us.puma.com/us/en/pd/cali-court-fall-tones-womens-sneakers/396444?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 3/200
Wait time ** 4 ** 
Item Clyde ATL SneakersBright Melon-Frosted Ivory has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/clyde-atl-sneakers/393084?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 4/200
Wait time ** 4 ** 
Show reviews of 4
Get the HTML code: https://us.puma.com/us/en/pd/palermo-womens-sneakers/397643?swatch=07&referrer-category=womens-shop-all-womens

Retrieving Details of recor

Get the HTML code: https://us.puma.com/us/en/pd/suede-classic-selflove-womens-sneakers/393031?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 29/200
Wait time ** 4 ** 
Show reviews of 29
Get the HTML code: https://us.puma.com/us/en/pd/puma-nyc-rider-fv-flagship-play-womens-sneakers/393343?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 30/200
Wait time ** 4 ** 
Show reviews of 30
Get the HTML code: https://us.puma.com/us/en/pd/mayze-stack-chelsea-womens-winter-boots/393201?swatch=02&referrer-category=womens-shop-all-womens

Retrieving Details of record: 31/200
Wait time ** 4 ** 
Review Exception of 180-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19fr0v0 tw-ozwx86 flex-row after::absolute after::hidden after::pointer-events-none after::-left-1 after::-top-1 after::-right-1 after::-bottom-1 after::border-2 after::border-outline after::rounded-sm tw-10s

Get the HTML code: https://us.puma.com/us/en/pd/mayze-stack-luxe-womens-sneakers/389853?swatch=04&referrer-category=womens-shop-all-womens

Retrieving Details of record: 44/200
Wait time ** 4 ** 
Show reviews of 44
Get the HTML code: https://us.puma.com/us/en/pd/capri-royale-suede-sneakers/393752?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 45/200
Wait time ** 4 ** 
Show reviews of 45
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-lemlem-suede-womens-sneakers/394009?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 46/200
Wait time ** 4 ** 
Item TRC Mira Summer Women's SneakersGranola-Day Dream has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/trc-mira-summer-womens-sneakers/389894?swatch=02&referrer-category=womens-shop-all-womens

Retrieving Details of record: 47/200
Wait time ** 4 ** 
Show reviews of 47
Get the HTML code: https://us.puma.com/us/en/pd/capri-royale-suede-sneakers/393752?swatch=

Get the HTML code: https://us.puma.com/us/en/pd/slipstream-luxury-womens-sneakers/393193?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 60/200
Wait time ** 4 ** 
Show reviews of 60
Get the HTML code: https://us.puma.com/us/en/pd/teveris-nitro%E2%84%A2-metallic-womens-sneakers/391098?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 61/200
Wait time ** 4 ** 
Item TRC Mira Brighter Days Women's SneakersPUMA White-Ravish has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/trc-mira-brighter-days-womens-sneakers/389893?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 62/200
Wait time ** 4 ** 
Show reviews of 62
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-dua-lipa-cell-dome-king-womens-sneakers/387289?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 63/200
Wait time ** 4 ** 
Show reviews of 63
Get the HTML code: https://us.puma.com/us/en

Get the HTML code: https://us.puma.com/us/en/pd/mayze-lucky-charm-womens-sneakers/389585?swatch=01&referrer-category=womens-shop-all-womens

Retrieving Details of record: 79/200
Wait time ** 4 ** 
Review Exception of 228-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19fr0v0 tw-ozwx86 flex-row after::absolute after::hidden after::pointer-events-none after::-left-1 after::-top-1 after::-right-1 after::-bottom-1 after::border-2 after::border-outline after::rounded-sm tw-10skrdm self-end">...</button> is not clickable at point (1457, 62). Other element would receive the click: <div class="h-16 lg:h-20 flex items-center justify-between mx-auto overflow-y-visible max-w-screen-3xl mobile:px-4 tablet:px-6 desktop:px-8" data-test-id="main-nav-bar">...</div>
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x00007FF74DAC82B2+55298]
	(No symbol) [0x00007FF74DA35E02]
	(No symbol) [0x00007FF74D8F05AB]
	(No sym

Get the HTML code: https://us.puma.com/us/en/pd/voltaic-evo-running-shoe/379601?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 96/200
Wait time ** 4 ** 
Show reviews of 96
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-lamelo-ball-mb-01-lo-mens-basketball-shoes/376941?swatch=04&referrer-category=mens-shop-all-mens

Retrieving Details of record: 97/200
Wait time ** 4 ** 
Show reviews of 97
Get the HTML code: https://us.puma.com/us/en/pd/suede-classic-xxi-sneakers/374915?swatch=05&referrer-category=mens-shop-all-mens

Retrieving Details of record: 98/200
Wait time ** 4 ** 
Review Exception of 247-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19fr0v0 tw-ozwx86 flex-row after::absolute after::hidden after::pointer-events-none after::-left-1 after::-top-1 after::-right-1 after::-bottom-1 after::border-2 after::border-outline after::rounded-sm tw-10skrdm self-end">...</button> is not cl

Get the HTML code: https://us.puma.com/us/en/pd/rs-x-hi-mens-sneakers/392718?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 120/200
Wait time ** 4 ** 
Item Suede Pumpkin Pie Men's SneakersPumpkin Pie-Amber has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/suede-pumpkin-pie-mens-sneakers/393260?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 121/200
Wait time ** 4 ** 
Show reviews of 121
Get the HTML code: https://us.puma.com/us/en/pd/deviate-nitro%E2%84%A2-elite-2-marathon-series-mens-running-shoes/378453?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 122/200
Wait time ** 4 ** 
Show reviews of 122
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-tmc-clyde-og-sneakers/387286?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 123/200
Wait time ** 4 ** 
Show reviews of 123
Get the HTML code: https://us.puma.com/us/en/pd/basket-classic-xxi-mens-sneaker

In [507]:
url_error2

,index,URL,Exception,msg
0,162,https://us.puma.com/us/en/pd/defy-mid-womens-t...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
1,180,https://us.puma.com/us/en/pd/cali-dream-preppy...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
2,181,https://us.puma.com/us/en/pd/cali-dream-star-q...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
3,188,https://us.puma.com/us/en/pd/slipstream-womens...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
4,192,https://us.puma.com/us/en/pd/mayze-stack-luxe-...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
5,199,https://us.puma.com/us/en/pd/teveris-nitro%E2%...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
6,201,https://us.puma.com/us/en/pd/suede-classic-bea...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
7,206,https://us.puma.com/us/en/pd/suede-classic-bea...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
8,214,https://us.puma.com/us/en/pd/suede-mayu-womens...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
9,225,https://us.puma.com/us/en/pd/suede-mayu-pop-wo...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...


In [510]:
# Join the batches
df_det = pd.concat([df_det, df_det2])
url_error = pd.concat([url_error, url_error2])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_puma_backup = df_det.copy()

# Save a file with the original data recovered
df_det_puma_backup.to_csv('datasets/det_raw_puma.csv')
url_error.to_csv('errors/puma_batch_err.csv') 

125 records were recovered in the second batch.

#### 3rd Batch - Puma

In [561]:
# Resuming the process
df_det = df_det_puma_backup.copy()

In [562]:
df_det.shape

(275, 5)

In [572]:
%%time

# Defines the urls 
df_urls = df_shoes_puma[['ShoeName', 'URL']][275:450]

# Scrape the records from 275 to 450
df_det3, url_error3 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det3.to_csv('datasets/det3_raw_puma.csv')

Retrieving Details of record: 1/175
Wait time ** 4 ** 
Show reviews of 1
Get the HTML code: https://us.puma.com/us/en/pd/fuse-2-0-mens-training-shoes/376151?swatch=19&referrer-category=mens-shop-all-mens

Retrieving Details of record: 2/175
Wait time ** 4 ** 
Show reviews of 2
Get the HTML code: https://us.puma.com/us/en/pd/porsche-legacy-speedcat-driving-shoes/307211?swatch=07&referrer-category=mens-shop-all-mens

Retrieving Details of record: 3/175
Wait time ** 4 ** 
Item SCUDERIA FERRARI x JOSHUA VIDES Men's Tuff SlippersRosso Corsa-PUMA Black has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/scuderia-ferrari-x-joshua-vides-mens-tuff-slippers/307997?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 4/175
Wait time ** 4 ** 
Show reviews of 4
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-lamelo-ball-mb-01-lo-mens-basketball-shoes/376941?swatch=12&referrer-category=mens-shop-all-mens

Retrieving Details of record: 5/175
Wait time ** 4

Item Suede NT Men's SneakersFall Foliage-Alpine Snow-Cool Light Gray has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/suede-nt-mens-sneakers/394057?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 37/175
Wait time ** 4 ** 
Show reviews of 37
Get the HTML code: https://us.puma.com/us/en/pd/foreverrun-nitro%E2%84%A2-mens-running-shoes/377757?swatch=11&referrer-category=mens-shop-all-mens

Retrieving Details of record: 38/175
Wait time ** 4 ** 
Show reviews of 38
Get the HTML code: https://us.puma.com/us/en/pd/ca-pro-ow-sneakers/393490?swatch=03&referrer-category=mens-shop-all-mens

Retrieving Details of record: 39/175
Wait time ** 4 ** 
Show reviews of 39
Get the HTML code: https://us.puma.com/us/en/pd/foreverrun-nitro%E2%84%A2-knit-mens-running-shoes/379139?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 40/175
Wait time ** 4 ** 
Item Fast-R NITRO™ Elite Men's Running ShoesFor All Time Red-Ultra Blue has not revi

Get the HTML code: https://us.puma.com/us/en/pd/foreverrun-nitro%E2%84%A2-wtr-mens-running-shoes/378472?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 70/175
Wait time ** 4 ** 
Item PUMA x FIRST MILE Electrify NITRO™ 3 Men's Running ShoesPUMA Black-Astro Red-Alpine Snow has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-first-mile-electrify-nitro%E2%84%A2-3-mens-running-shoes/378457?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 71/175
Wait time ** 4 ** 
Show reviews of 71
Get the HTML code: https://us.puma.com/us/en/pd/pwr-nitro%E2%84%A2-squared-mens-training-shoes/378687?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 72/175
Wait time ** 4 ** 
Item Fast-R NITRO™ Elite Men's Running ShoesPUMA White-Ultra Blue has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/fast-r-nitro%E2%84%A2-elite-mens-running-shoes/195336?swatch=09&referrer-category=mens-shop-all-mens



Get the HTML code: https://us.puma.com/us/en/pd/deviate-nitro%E2%84%A2-2-wtrepel-mens-running-shoes/378734?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 97/175
Wait time ** 4 ** 
Item Accelerate NITRO™ SQD Indoor ShoesPUMA White-Vivid Violet has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/accelerate-nitro%E2%84%A2-sqd-indoor-shoes/107473?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 98/175
Wait time ** 4 ** 
Item Suede Brand Love II Unisex SneakersTurquoise Surf-Grape Mist has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/suede-brand-love-ii-unisex-sneakers/395737?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 99/175
Wait time ** 4 ** 
Item CA Pro Quilt Men's SneakersPUMA White-Olive Drab has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/ca-pro-quilt-mens-sneakers/393277?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record

Get the HTML code: https://us.puma.com/us/en/pd/puma-x-lauren-london-rs-trck-sneakers/392086?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 129/175
Wait time ** 4 ** 
Show reviews of 129
Get the HTML code: https://us.puma.com/us/en/pd/seasons-voyage-nitro%E2%84%A2-3-mens-running-shoes/377745?swatch=06&referrer-category=mens-shop-all-mens

Retrieving Details of record: 130/175
Wait time ** 4 ** 
Review Exception of 404-Message: element click intercepted: Element <button data-test-id="read-all-reviews" type="button" class="group tw-19fr0v0 tw-ozwx86 flex-row after::absolute after::hidden after::pointer-events-none after::-left-1 after::-top-1 after::-right-1 after::-bottom-1 after::border-2 after::border-outline after::rounded-sm tw-10skrdm self-end">...</button> is not clickable at point (1457, 58). Other element would receive the click: <div class="relative">...</div>
  (Session info: chrome=120.0.6099.72)
Stacktrace:
	GetHandleVerifier [0x00007FF74DAC82B

Retrieving Details of record: 157/175
Wait time ** 4 ** 
Show reviews of 157
Get the HTML code: https://us.puma.com/us/en/pd/fast-fwd-nitro%E2%84%A2-elite-mens-running-shoes/376591?swatch=08&referrer-category=mens-shop-all-mens

Retrieving Details of record: 158/175
Wait time ** 4 ** 
Item PUMA Star Strawberries And Cream Men's SneakersPUMA White-For All Time Red has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/puma-star-strawberries-and-cream-mens-sneakers/393196?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 159/175
Wait time ** 4 ** 
Show reviews of 159
Get the HTML code: https://us.puma.com/us/en/pd/evospeed-long-distance-nitro%E2%84%A2-elite%2B-track-spikes/378222?swatch=03&referrer-category=mens-shop-all-mens

Retrieving Details of record: 160/175
Wait time ** 4 ** 
Show reviews of 160
Get the HTML code: https://us.puma.com/us/en/pd/palermo-leather-mens-sneakers/396464?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Detail

In [573]:
url_error3

,index,URL,Exception,msg
0,355,https://us.puma.com/us/en/pd/velophasis-techni...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
1,404,https://us.puma.com/us/en/pd/mercedes-amg-petr...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
2,447,https://us.puma.com/us/en/pd/foreverrun-nitro%...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...


In [575]:
# Join the batches
df_det = pd.concat([df_det, df_det3])
url_error = pd.concat([url_error, url_error3])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_puma_backup = df_det.copy()

# Save a file with the original data recovered
df_det_puma_backup.to_csv('datasets/det_raw_puma.csv')
url_error.to_csv('errors/puma_batch_err.csv') 

#### 4th Batch - Puma

In [585]:
%%time

# Defines the urls 
df_urls = df_shoes_puma[['ShoeName', 'URL']][450:]

# Scrape the records from 450 to 628
df_det4, url_error4 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det4.to_csv('datasets/det4_raw_puma.csv')

Retrieving Details of record: 1/178
Wait time ** 4 ** 
Item F1® Austin Grand Prix Speedcat Pro Men's Driving ShoesFlat Medium Gray-Pop Red-PUMA Black has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/f1%C2%AE-austin-grand-prix-speedcat-pro-mens-driving-shoes/308277?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 2/178
Wait time ** 4 ** 
Show reviews of 2
Get the HTML code: https://us.puma.com/us/en/pd/pwr-nitro%E2%84%A2-squared-mens-training-shoes/378687?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 3/178
Wait time ** 4 ** 
Show reviews of 3
Get the HTML code: https://us.puma.com/us/en/pd/ca-pro-classic-sneakers/380190?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 4/178
Wait time ** 4 ** 
Show reviews of 4
Get the HTML code: https://us.puma.com/us/en/pd/future-pro-fg-ag-mens-soccer-cleats/107361?swatch=04&referrer-category=mens-shop-all-mens

Retrieving Details of record: 5/178


Retrieving Details of record: 29/178
Wait time ** 4 ** 
Show reviews of 29
Get the HTML code: https://us.puma.com/us/en/pd/evospeed-sprint-14-track-spikes/377001?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 30/178
Wait time ** 4 ** 
Item PUMA x FIRST MILE Velocity NITRO™ 2 Men's Running ShoesAlpine Snow-Cool Dark Gray-Astro Red has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-first-mile-velocity-nitro%E2%84%A2-2-mens-running-shoes/379113?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 31/178
Wait time ** 4 ** 
Show reviews of 31
Get the HTML code: https://us.puma.com/us/en/pd/fuse-2-0-mens-training-shoes/376151?swatch=15&referrer-category=mens-shop-all-mens

Retrieving Details of record: 32/178
Wait time ** 4 ** 
Item KING ULTIMATE ART OF FOOTBALL FG/AG Men's Soccer CleatsPUMA Black-Alpine Snow-PUMA Gold-PUMA White has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/king-ultimate-art-of-f

Get the HTML code: https://us.puma.com/us/en/pd/california-casual-mens-sneakers/366608?swatch=07&referrer-category=mens-shop-all-mens

Retrieving Details of record: 53/178
Wait time ** 4 ** 
Show reviews of 53
Get the HTML code: https://us.puma.com/us/en/pd/mercedes-amg-petronas-f1-future-cat-og-mens-shoes/307891?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 54/178
Wait time ** 4 ** 
Show reviews of 54
Get the HTML code: https://us.puma.com/us/en/pd/puma-morphic-base-mens-sneakers/392982?swatch=04&referrer-category=mens-shop-all-mens

Retrieving Details of record: 55/178
Wait time ** 4 ** 
Show reviews of 55
Get the HTML code: https://us.puma.com/us/en/pd/puma-morphic-base-mens-sneakers/392982?swatch=05&referrer-category=mens-shop-all-mens

Retrieving Details of record: 56/178
Wait time ** 4 ** 
Show reviews of 56
Get the HTML code: https://us.puma.com/us/en/pd/mercedes-amg-petronas-f1-future-cat-og-mens-shoes/307891?swatch=02&referrer-category=mens-shop

Get the HTML code: https://us.puma.com/us/en/pd/extent-nitro%E2%84%A2-adv-sneakers/387499?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 86/178
Wait time ** 4 ** 
Show reviews of 86
Get the HTML code: https://us.puma.com/us/en/pd/puma-morphic-mens-sneakers/392724?swatch=05&referrer-category=mens-shop-all-mens

Retrieving Details of record: 87/178
Wait time ** 4 ** 
Item Morphic Reflective Men's SneakersPUMA Black-Yellow Burst has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/morphic-reflective-mens-sneakers/393646?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 88/178
Wait time ** 4 ** 
Item evoSPEED Star 8 Men's Track SpikesPUMA Black-PUMA White-PUMA Red has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/evospeed-star-8-mens-track-spikes/377959?swatch=03&referrer-category=mens-shop-all-mens

Retrieving Details of record: 89/178
Wait time ** 4 ** 
Show reviews of 89
Get the HTML code: https://us.p

Get the HTML code: https://us.puma.com/us/en/pd/ferrari-ionf-miami-mens-motorsport-shoes/307506?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 113/178
Wait time ** 4 ** 
Show reviews of 113
Get the HTML code: https://us.puma.com/us/en/pd/capri-royale-suede-sneakers/393752?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 114/178
Wait time ** 4 ** 
Show reviews of 114
Get the HTML code: https://us.puma.com/us/en/pd/suede-brand-love-sneakers/393813?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 115/178
Wait time ** 4 ** 
Item Porsche Design Xetic Men's SneakersAsh Gray-Ash Gray has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/porsche-design-xetic-mens-sneakers/307759?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 116/178
Wait time ** 4 ** 
Show reviews of 116
Get the HTML code: https://us.puma.com/us/en/pd/puma-x-palm-tree-crew-ca-pro-sneakers/394635?sw

Get the HTML code: https://us.puma.com/us/en/pd/ca-pro-lux-cord-mens-sneakers/390129?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 135/178
Wait time ** 4 ** 
Show reviews of 135
Get the HTML code: https://us.puma.com/us/en/pd/rs-xl-playlist-mens-sneakers/394552?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 136/178
Wait time ** 4 ** 
Item Capri Royale Men's SneakersAlpine Snow-Warm White has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/capri-royale-mens-sneakers/392435?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 137/178
Wait time ** 4 ** 
Show reviews of 137
Get the HTML code: https://us.puma.com/us/en/pd/ca-pro-suede-sneakers/387327?swatch=06&referrer-category=mens-shop-all-mens

Retrieving Details of record: 138/178
Wait time ** 4 ** 
Item evoSPEED Star 7 Track SpikesDeep Orchid-Puma White has not reviews.
Get the HTML code: https://us.puma.com/us/en/pd/evospeed-star-7-tra

Retrieving Details of record: 171/178
Wait time ** 4 ** 
Show reviews of 171
Get the HTML code: https://us.puma.com/us/en/pd/speedcat-og-%2B-sparco-driving-shoes/307171?swatch=04&referrer-category=mens-shop-all-mens

Retrieving Details of record: 172/178
Wait time ** 4 ** 
Show reviews of 172
Get the HTML code: https://us.puma.com/us/en/pd/suede-mid-xxi-sneakers/380205?swatch=01&referrer-category=mens-shop-all-mens

Retrieving Details of record: 173/178
Wait time ** 4 ** 
Show reviews of 173
Get the HTML code: https://us.puma.com/us/en/pd/speedcat-og-%2B-sparco-driving-shoes/307171?swatch=06&referrer-category=mens-shop-all-mens

Retrieving Details of record: 174/178
Wait time ** 4 ** 
Show reviews of 174
Get the HTML code: https://us.puma.com/us/en/pd/suede-mid-xxi-sneakers/380205?swatch=02&referrer-category=mens-shop-all-mens

Retrieving Details of record: 175/178
Wait time ** 4 ** 
Show reviews of 175
Get the HTML code: https://us.puma.com/us/en/pd/suede-mid-xxi-sneakers/380205?swatc

In [587]:
url_error4

,index,URL,Exception,msg
0,461,https://us.puma.com/us/en/pd/foreverrun-nitro%...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
1,486,https://us.puma.com/us/en/pd/pwrspin-x-alex-to...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
2,501,https://us.puma.com/us/en/pd/california-casual...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
3,534,https://us.puma.com/us/en/pd/extent-nitro%E2%8...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
4,549,https://us.puma.com/us/en/pd/puma-nyc-suede-mi...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
5,561,https://us.puma.com/us/en/pd/ferrari-ionf-miam...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element is not clic...
6,570,https://us.puma.com/us/en/pd/evospeed-distance...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...
7,583,https://us.puma.com/us/en/pd/ca-pro-lux-cord-m...,<class 'selenium.common.exceptions.ElementClic...,element click intercepted: Element <button dat...


In [588]:
# Join the batches
df_det = pd.concat([df_det, df_det4])
url_error = pd.concat([url_error, url_error3])

# adjusting the indexes
df_det.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_puma_backup = df_det.copy()

# Save a file with the original data recovered
df_det_puma_backup.to_csv('datasets/det_raw_puma.csv')
url_error.to_csv('errors/puma_batch_err.csv') 

We recovered all the data from Puma.com. Now we will process the gathered data to adjust it to the solicited format.

### 3.2 Data Processing

<!-- df_proc_backup = df_proc.copy() -->

We will clean the 628 records recovered from Puma.com, and save the Puma detail data processed in a file. Afterward, we will extract the required data for Table 2 and Table 3 according to the specifications.

In [606]:
def clean_df_det_puma(df):
    ''' Cleaning of Puma detail data frame.
    ''' 
    df['count_of_sizes'] = df['count_of_sizes'].apply(len)

    temp = df['product_code'].dropna().apply(lambda x: re.findall(r'Style: (\d+_\d+)', str(x)))
    df['product_code'] = temp.apply(lambda x: x[0] if x else None)

    df['reviews_data'] = df['reviews'].dropna().apply(lambda x: re.findall(r'Reviews\D*(\d+)\D*Average:\D*(\d+\.\d+)', str(x)))
    df['reviews_data'] = pd.DataFrame(df['reviews_data'].to_list(), index = df.index)
    df[['n_reviews', 'star_rating']] = pd.DataFrame( df['reviews_data'].to_list(), index = df.index)

    df['quality'] = df['quality'].apply(lambda x: x[0] if x else None)

    # Drop columns no longer needed
    df.drop(['reviews','reviews_data'], axis=1, inplace=True)

    return df

In [609]:
# Cleans data frame  
df_proc = clean_df_det_puma(df_det)

# Reset index
df_proc.reset_index(drop=True, inplace=True)

# Save the detail data processed of Puma for reference
df_proc.to_csv('datasets/det_processed_puma.csv')

In [610]:
print(f"Puma detail data: {df_proc.shape}")
display(df_proc.head())
display(df_proc.tail())

Puma detail data: (628, 6)


,count_of_sizes,colors,product_code,quality,n_reviews,star_rating
0,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po...",399332_01,50.0,6,4.5
1,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po...",399332_03,50.0,6,4.5
2,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po...",399332_02,50.0,6,4.5
3,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas...",398949_01,None,0,0.0
4,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas...",398949_02,None,0,0.0


,count_of_sizes,colors,product_code,quality,n_reviews,star_rating
623,17,"[Puma Black-Puma White-Amazon Green, Quarry-Pu...",380205_02,70.4,31,4.4
624,11,"[Puma Black-Puma White-Amazon Green, Quarry-Pu...",380205_16,70.4,31,4.4
625,19,[PUMA Black-Frosted Ivory],396057_01,None,0,0.0
626,18,[Sun Stream-Sunset Glow-PUMA Black],379098_01,None,0,0.0
627,13,[PUMA White-Archive Green],393199_01,None,0,0.0


In [611]:
# Setting tables
df_table2_puma = df_proc[['product_code', 'count_of_sizes', 'colors']]
df_table3_puma = df_proc[['product_code', 'n_reviews', 'quality', 'star_rating']]
df_table3_puma['size'] = np.NaN
df_table3_puma['comfort'] = np.NaN

# Saving the data frame
df_table2_puma.to_csv('datasets/Table2_puma.csv')
display(df_table2_puma)

df_table3_puma.to_csv('datasets/Table3_puma.csv')
display(df_table3_puma)

,product_code,count_of_sizes,colors
0,399332_01,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po..."
1,399332_03,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po..."
2,399332_02,12,"[PUMA Black-Warm White-Gum, Speed Blue-Lime Po..."
3,398949_01,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas..."
4,398949_02,12,"[Malachite-Puma Team Gold-PUMA White, Dark Jas..."
...,...,...,...
623,380205_02,17,"[Puma Black-Puma White-Amazon Green, Quarry-Pu..."
624,380205_16,11,"[Puma Black-Puma White-Amazon Green, Quarry-Pu..."
625,396057_01,19,[PUMA Black-Frosted Ivory]
626,379098_01,18,[Sun Stream-Sunset Glow-PUMA Black]


,product_code,n_reviews,quality,star_rating,size,comfort
0,399332_01,6,50.0,4.5,NaN,NaN
1,399332_03,6,50.0,4.5,NaN,NaN
2,399332_02,6,50.0,4.5,NaN,NaN
3,398949_01,0,None,0.0,NaN,NaN
4,398949_02,0,None,0.0,NaN,NaN
...,...,...,...,...,...,...
623,380205_02,31,70.4,4.4,NaN,NaN
624,380205_16,31,70.4,4.4,NaN,NaN
625,396057_01,0,None,0.0,NaN,NaN
626,379098_01,0,None,0.0,NaN,NaN


Now we will assign the product_code to Table 1.

In [621]:
# Establishing product code 
df_shoes_puma['product_code'] = df_proc['product_code']

# Save the Table1 including the product_code
df_shoes_puma.to_csv('datasets/Table1_puma.csv')
df_shoes_puma

df_shoes_puma

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand,product_code
0,FENTY x PUMA Creeper Phatty Women's SneakersPU...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma,399332_01
1,FENTY x PUMA Creeper Phatty Women's SneakersLa...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma,399332_03
2,FENTY x PUMA Creeper Phatty Women's SneakersSp...,https://us.puma.com/us/en/pd/fenty-x-puma-cree...,3,140.00,140.00,Women,Puma,399332_02
3,Basket Classic Velvet Women's SneakersMalachit...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,64.99,80.00,Women,Puma,398949_01
4,Basket Classic Velvet Women's SneakersDark Jas...,https://us.puma.com/us/en/pd/basket-classic-ve...,2,64.99,80.00,Women,Puma,398949_02
...,...,...,...,...,...,...,...,...
623,Suede Mid XXI SneakersQuarry-Puma White,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma,380205_02
624,Suede Mid XXI SneakersPumpkin Pie-PUMA White,https://us.puma.com/us/en/pd/suede-mid-xxi-sne...,4,85.00,85.00,Men,Puma,380205_16
625,PUMA x PLEASURES Suede XL Men's SneakersPUMA B...,https://us.puma.com/us/en/pd/puma-x-pleasures-...,1,120.00,120.00,Men,Puma,396057_01
626,evoSPEED Long Jump 11 UltraweaveSun Stream-Sun...,https://us.puma.com/us/en/pd/evospeed-long-jum...,1,140.00,140.00,Men,Puma,379098_01


For now, we have gathered all the required data from Nike. Later on, we will reorganize the colors with the rest of the data recovered.

## 4. Woodland 

In [667]:
driver.quit()

In [650]:
sites['woodlww']

'https://www.woodlandworldwide.com/'

In [669]:
# Initialization of the dataframe
df_shoes_woodl = pd.DataFrame()

# Config to scrape ShoeName ,Category(Men/Women), no.of colors, price
url = sites['woodlww']
xpath_grp = '//div[@class="productcardplp_productwrapper__x83B9"]'
soup_config = {
    'ShoeName':['p',{'class':"productcardplp_productname__F8lks"},None, None ,'-'], 
    'URL': ['div', {'class':"productcardplp_card__TuA8g"},
            lambda el: el.find('a')['href'] if el.find('a') is not None else '-', 
            None, '-'],
    'price_reduce': ['p', {'class':"productcardplp_productprice__jdlz6"}, 
                     lambda el: search_with_pattern(el.text.strip(), pattern=r'([\d.-]+)$') if el is not None else '', 
                     None, '-'], 
    'price': ['p', {'class':"productcardplp_productprice__jdlz6"}, 
                     lambda el: search_with_pattern(el.text.strip(), pattern = r'₹([\d.-]+)') if el is not None else '', 
                     None, '-'],
    'price_': ['p', {'class':"productcardplp_productprice__jdlz6"}, None,None, '-'],  
}

# Scrap the main data for Men and Women in woodlandworldwide.com
for category in ['Women','Men']:

    # Path to gathering the items according to the category
    path_woodlww = {
        'Men': {'msg': [f'\n{category} Category...'],
                       'xpaths': [ '//h3[contains(text(), "Sneakers")]/following::button[contains(text(),"Shop Now")]' ],
                       'wait_time': [ 5 ]},
        'Women': {'msg': [f'\n{category} Category...', 'Selecting Sneakers/Sports...'],
                       'xpaths': ['//button[contains(text(),"Women")]',
                                  '//h5[@class="womencategory_category__yqK9S" and contains(text(),"Sneaker/Sports")]'],
                       'wait_time': [ 2, 4 ]}
        }


    try:
        # 1. Open the Driver 
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(url) 

        wait_time = 4
        time.sleep(wait_time)

        # 2.Scrolling to load all the data 
        if category == 'Men':
            smooth_scroll(driver, scrolls=20, scroll_speed=5, scroll_break=3)
            time.sleep(5)
    
        # 3. Go to the items according to category
        follow_path_to(path_woodlww[category]) 
        time.sleep(wait_time)

        # 4. Find out the number of shoes to retrieve in the category
        n_shoes = n_shoes_to_scrap('//p[@class="MuiTypography-root MuiTypography-body1 css-1s2byso"]', 
                                   pattern=r'(\d+)$')  
        print(f"Shoes to recover: {n_shoes}\n" )

        # 5. Scroll until recovered all shoes
        recovered_shoes, _ = scroll_to_recovered_shoes(n_shoes, xpath_grp)

        print(f"\nTotal of shoes recovered: {recovered_shoes}\n")

        print('*'*50)
        print(' '*5, f'S C R A P I N G - Category: {category}')
        print('*'*50)

        # 5.Scraping ShoeName ,Category(Men/Women), no.of colors, price
        time.sleep(2)
        df_shoes_cat = scrape_items_by_group(driver, xpath_grp, **soup_config)

        # 6. Set the category and Append the new data 
        df_shoes_cat['Category'] = category
        df_shoes_woodl = pd.concat([df_shoes_woodl, df_shoes_cat])
        print( f'Products in category {category} retrieved successfully: {df_shoes_cat.shape}' )
        
        # Free driver
        driver.quit()
        
        time.sleep(wait_time)
        
    except Exception as e:
        print(f"Error: {str(e)}")

        
# adjusting the indexes
df_shoes_woodl.reset_index(drop=True, inplace=True)

Wait time ** 2 **

Women Category...
Wait time ** 4 **
Selecting Sneakers/Sports...
Shoes to recover: 77

Recovered shoes: 12
Scrolling 1 times...
Recovered shoes: 24, still to recovered: 53
Scrolling 2 times...
Recovered shoes: 36, still to recovered: 41
Scrolling 3 times...
Recovered shoes: 48, still to recovered: 29
Scrolling 4 times...
Recovered shoes: 60, still to recovered: 17
Scrolling 5 times...
Recovered shoes: 72, still to recovered: 5

Total of shoes recovered: 72

**************************************************
      S C R A P I N G - Category: Women
**************************************************
Recovered group-tags: 72
Records converted to HTML strings: 72
Products in category Women retrieved successfully: (72, 6)
Scrolled 4 times.
Wait time ** 5 **

Men Category...
Shoes to recover: 110

Recovered shoes: 110
Scrolled 15 times.

Total of shoes recovered: 110

**************************************************
      S C R A P I N G - Category: Men
******************

The number of colors will be gathered with the detail data.
The prices are in Rupee (INR ₹). 1₹ is equivalent to 0.012 USD $. So we will transform the prices to American dollars.

In [671]:
# Backup dataframe detail brand WorldLand
df_shoes_woodl_backup = df_shoes_woodl.copy()

# Save a file with the original data recovered
df_shoes_woodl_backup.to_csv('datasets/raw_woodland.csv')

In [678]:
# Cleaning the data frame
def clean_df_woodland(df):
    df_shoes = df.copy()
    df_shoes['URL'] = 'https://www.woodlandworldwide.com'+ df_shoes['URL']
    df_shoes['price'] = df_shoes['price'].apply(lambda x: round(float(x) * 0.012, 2))
    df_shoes['price_reduce'] = df_shoes['price_reduce'].apply(lambda x: round(float(x) * 0.012, 2))
    df_shoes.drop(['price_'], axis=1, inplace=True)

    # Setting up the dataframe
    df_shoes['Brand'] = 'WoodLand'
    df_shoes.reset_index(drop=True, inplace=True)
    
    return df_shoes

In [681]:
# Cleaning the data frame
df_shoes_woodl = clean_df_woodland(df_shoes_woodl)

# Saving the data frame
df_shoes_woodl.to_csv('datasets/Table1_woodland.csv')
display(df_shoes_woodl)

,ShoeName,URL,price_reduce,price,Category,Brand
0,Brown/maroon Canvas shoes for women,https://www.woodlandworldwide.com/productinfo/...,26.96,53.94,Women,WoodLand
1,Lgrey sneaker for women,https://www.woodlandworldwide.com/productinfo/...,59.94,59.94,Women,WoodLand
2,Woodland DCadet Blue casual sneakers,https://www.woodlandworldwide.com/productinfo/...,59.94,59.94,Women,WoodLand
3,Variglia Sneakers for Women,https://www.woodlandworldwide.com/productinfo/...,53.94,53.94,Women,WoodLand
4,DBLUE casual sneakers,https://www.woodlandworldwide.com/productinfo/...,29.96,59.94,Women,WoodLand
...,...,...,...,...,...,...
177,Brown Sports Shoe for men,https://www.woodlandworldwide.com/productinfo/...,57.56,95.94,Men,WoodLand
178,PARIS KHAKI sneaker for men,https://www.woodlandworldwide.com/productinfo/...,35.96,59.94,Men,WoodLand
179,Dubai Khaki sneakers for men,https://www.woodlandworldwide.com/productinfo/...,44.34,44.34,Men,WoodLand
180,Navy sneakers for men,https://www.woodlandworldwide.com/productinfo/...,23.72,39.54,Men,WoodLand


### 4.1 WoodLand Details
#### Configuring the main parameters

In [686]:
# Defines the configuration to retrieve the data
config = {
    'n_scrolls': 1,
    'height_scroll': 200,
    'wait_time': 7,
}

# Defines the attributes to recover with all the special configuration according to the property.
soup_config = { 
            'count_of_sizes':['button', {'class':"sizepicker_option__OGKzO"}, 
                              None,
                              lambda el: 'disabled' not in el.attrs,  '-'],  
            'colors': ['div', {'class': re.compile(r'colorpicker_option')}, 
                       lambda el: search_with_pattern(el.find('img')['src'], pattern = r'https://assets\.woodlandworldwide\.app/compressed/images/[^/]+/([^/]+)' ) if el.find('img') is not None else '' ,
                       None, ''],
            'color': ['h5', {'class':"colorpicker_colorname__sRdk8"},
                     None, None, ''],
            'product_code': ['div', {'class':"colorpicker_option__B8oxU colorpicker_active___RkXc"}, 
                       lambda el: search_with_pattern(el.find('img')['src'], pattern = r"https://assets\.woodlandworldwide\.app/compressed/images/([^/]+)" ) if el.find('img') is not None else '' ,
                       None, '-'],
        }

In [685]:
driver.quit()

##### 1st Batch WoodLand

In [687]:
%%time

# Defines the urls 
df_urls = df_shoes_woodl[['ShoeName', 'URL']][:100]

# Scrape the first 100 records
df_det1, url_error1 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)


Retrieving Details of record: 1/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0310S2602A-BROWN_MAROON

Retrieving Details of record: 2/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC015044221A-LGREY

Retrieving Details of record: 3/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0H9026391B-DCADET_BLUE

Retrieving Details of record: 4/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC051044211A-VARIGLIA

Retrieving Details of record: 5/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0AI024031B-DBLUE

Retrieving Details of record: 6/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0310S2622A-WHITE

Retrieving Details of record: 7/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0260S2522A-WHIT

Retrieving Details of record: 59/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0260U2552A-RED

Retrieving Details of record: 60/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0BA025051A-BLACK

Retrieving Details of record: 61/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0510T0792A-LBLUE

Retrieving Details of record: 62/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0Z3022441A-ORANGE

Retrieving Details of record: 63/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0Z3028911A-BLACK

Retrieving Details of record: 64/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0260V3482A-RED

Retrieving Details of record: 65/100
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FLC0180U6582A-GRAFFITI_WHITE



In [688]:
df_det1.to_csv('datasets/det1_raw_woodland.csv')

# Backup dataframe detail brand Nike
df_det_woodl_backup = df_det1.copy()

# Save a file with the original data recovered
df_det_woodl_backup.to_csv('datasets/det_raw_woodland.csv')
url_error1.to_csv('errors/woodland_batch_err.csv') 

In [689]:
df_det1

,count_of_sizes,colors,color,product_code
0,"[36, 37, 38, 39, 40, 41]","[BROWN_MAROON, YELLOW_OLIVE]",[BROWN/MAROON],[FLC0310S2602A]
1,"[36, 37, 38, 39, 40, 41]","[LGREY, KHAKI, BROWN]",[LGREY],[FLC015044221A]
2,"[36, 37, 38, 39, 40, 41]","[DCADET_BLUE, CAMEL]",[DCADET BLUE],[FLC0H9026391B]
3,"[36, 37, 38, 39, 40, 41]","[VARIGLIA, LPINK, AIR_BLUE]",[VARIGLIA],[FLC051044211A]
4,"[36, 37, 38, 39, 40, 41]",[DBLUE],[DBLUE],[FGC0AI024031B]
...,...,...,...,...
95,"[40, 41, 42, 43, 44, 45]","[RB_BROWN, NBLUE]",[RB BROWN],[FGC0Z3024211C]
96,"[40, 41, 42, 43, 44, 45]","[BEIGE, GREY]",[BEIGE],[FGC0510R2012A]
97,"[39, 40, 41, 42, 43, 44, 45]",[WHITE],[WHITE],[FGC0DN0AR912A]
98,"[42, 43, 44, 45]","[WHITE, BLACK]",[WHITE],[FGC0510R2022A]


In [691]:
url_error1

,index,URL,Exception,msg


##### 2nd Batch WoodLand

In [692]:
%%time

# Defines the urls 
df_urls = df_shoes_woodl[['ShoeName', 'URL']][100:]

# Scrape the rest of the records
df_det2, url_error2 = scrape_details_by_URL_in_batches(df_urls, soup_config, **config)
df_det2.to_csv('datasets/det2_raw_woodland.csv')

Retrieving Details of record: 1/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0H40S7542A-BLUE

Retrieving Details of record: 2/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0H40S7572A-BLACK

Retrieving Details of record: 3/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0510R2032A-GREY

Retrieving Details of record: 4/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0EM0S0152A-BLACK_LGREY

Retrieving Details of record: 5/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0H40T3262A-OLIVE

Retrieving Details of record: 6/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0180T8762A-BLACK_GREY

Retrieving Details of record: 7/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0510R1992A-BLACK

Retrieving 

Retrieving Details of record: 59/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0C30V0672A-GREY

Retrieving Details of record: 60/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0H40U8432A-BLUE

Retrieving Details of record: 61/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0510S0062A-NAVY

Retrieving Details of record: 62/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0260U3112A-GREY

Retrieving Details of record: 63/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0Y9032762A-OLIVE_BLACK

Retrieving Details of record: 64/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0Y9032772A-RED_NAVY

Retrieving Details of record: 65/82
Wait time ** 7 ** 
Get the HTML code: https://www.woodlandworldwide.com/productinfo/FGC0Y9039172A-GREY

Retrievin

In [694]:
url_error2

,index,URL,Exception,msg


In [696]:
# Join the batches
df_det1 = pd.concat([df_det1, df_det2])
url_error1 = pd.concat([url_error1, url_error2])

# adjusting the indexes
df_det1.reset_index(drop=True, inplace=True)

# Backup dataframe detail brand Adidas
df_det_woodl_backup = df_det1.copy()

# Save a file with the original data recovered
df_det_woodl_backup.to_csv('datasets/det_raw_puma.csv')
url_error1.to_csv('errors/puma_batch_err.csv')

We recovered all the data from Woodlandworldwide.com. Now we will process the gathered data to adjust it to the solicited format.

### 4.2 Data Processing

<!-- df_proc_backup = df_proc.copy() -->

We will clean the 182 records recovered from Puma.com, and save the Puma detail data processed in a file. Afterward, we will extract the required data for Table 2 and Table 3 according to the specifications.

In [697]:
df_det1

,count_of_sizes,colors,color,product_code
0,"[36, 37, 38, 39, 40, 41]","[BROWN_MAROON, YELLOW_OLIVE]",[BROWN/MAROON],[FLC0310S2602A]
1,"[36, 37, 38, 39, 40, 41]","[LGREY, KHAKI, BROWN]",[LGREY],[FLC015044221A]
2,"[36, 37, 38, 39, 40, 41]","[DCADET_BLUE, CAMEL]",[DCADET BLUE],[FLC0H9026391B]
3,"[36, 37, 38, 39, 40, 41]","[VARIGLIA, LPINK, AIR_BLUE]",[VARIGLIA],[FLC051044211A]
4,"[36, 37, 38, 39, 40, 41]",[DBLUE],[DBLUE],[FGC0AI024031B]
...,...,...,...,...
177,[44],[BROWN],[BROWN],[FGC0DY0S0082A]
178,[],"[PARIS_KHAKI, DNAVY_4, DBROWN]",[PARIS KHAKI],[FGC0L4033131A]
179,[44],"[DUBAI_KHAKI, BLACK, OLIVE_GREEN, CAMEL]",[DUBAI KHAKI],[FGC0L4037301A]
180,[],"[NAVY, BLACK]",[NAVY],[FGC0Y9032792A]


In [698]:
df_det1['count_of_sizes'].apply(len)

0      6
1      6
2      6
3      6
4      6
      ..
177    1
178    0
179    1
180    0
181    0
Name: count_of_sizes, Length: 182, dtype: int64

In [700]:
df_det1['product_code'].apply(lambda x: x[0] if x else None)

0      FLC0310S2602A
1      FLC015044221A
2      FLC0H9026391B
3      FLC051044211A
4      FGC0AI024031B
           ...      
177    FGC0DY0S0082A
178    FGC0L4033131A
179    FGC0L4037301A
180    FGC0Y9032792A
181    FGC0Z7041652A
Name: product_code, Length: 182, dtype: object

In [704]:
def clean_df_det_woodl(df):
    ''' Cleaning of WoodLand detail data frame.
    ''' 
    df_shoes = df.copy()
    df_shoes['count_of_sizes'] = df_shoes['count_of_sizes'].apply(len)
    df_shoes['product_code'] = df_shoes['product_code'].apply(lambda x: x[0] if x else None)

    # Drop columns no longer needed
    df_shoes.drop(['color'], axis=1, inplace=True)

    return df_shoes

In [707]:
# Cleans data frame  
df_proc = clean_df_det_woodl(df_det1)

# Reset index
df_proc.reset_index(drop=True, inplace=True)

# Save the detail data processed of WoodLand for reference
df_proc.to_csv('datasets/det_processed_woodland.csv')

In [708]:
print(f"WoodLand detail data: {df_proc.shape}")
display(df_proc.head())
display(df_proc.tail())

WoodLand detail data: (182, 3)


,count_of_sizes,colors,product_code
0,6,"[BROWN_MAROON, YELLOW_OLIVE]",FLC0310S2602A
1,6,"[LGREY, KHAKI, BROWN]",FLC015044221A
2,6,"[DCADET_BLUE, CAMEL]",FLC0H9026391B
3,6,"[VARIGLIA, LPINK, AIR_BLUE]",FLC051044211A
4,6,[DBLUE],FGC0AI024031B


,count_of_sizes,colors,product_code
177,1,[BROWN],FGC0DY0S0082A
178,0,"[PARIS_KHAKI, DNAVY_4, DBROWN]",FGC0L4033131A
179,1,"[DUBAI_KHAKI, BLACK, OLIVE_GREEN, CAMEL]",FGC0L4037301A
180,0,"[NAVY, BLACK]",FGC0Y9032792A
181,0,"[BLACK, DGREY]",FGC0Z7041652A


In [712]:
# Setting tables
df_table2_woodl = df_proc[['product_code', 'count_of_sizes', 'colors']]
df_table3_woodl = df_proc[['product_code']]

for col in ['n_reviews', 'quality', 'star_rating', 'size', 'comfort']:
        df_table3_woodl[col] = np.NaN

# Saving the data frame
df_table2_woodl.to_csv('datasets/Table2_woodland.csv')
display(df_table2_woodl)

df_table3_woodl.to_csv('datasets/Table3_woodland.csv')
display(df_table3_woodl)

,product_code,count_of_sizes,colors
0,FLC0310S2602A,6,"[BROWN_MAROON, YELLOW_OLIVE]"
1,FLC015044221A,6,"[LGREY, KHAKI, BROWN]"
2,FLC0H9026391B,6,"[DCADET_BLUE, CAMEL]"
3,FLC051044211A,6,"[VARIGLIA, LPINK, AIR_BLUE]"
4,FGC0AI024031B,6,[DBLUE]
...,...,...,...
177,FGC0DY0S0082A,1,[BROWN]
178,FGC0L4033131A,0,"[PARIS_KHAKI, DNAVY_4, DBROWN]"
179,FGC0L4037301A,1,"[DUBAI_KHAKI, BLACK, OLIVE_GREEN, CAMEL]"
180,FGC0Y9032792A,0,"[NAVY, BLACK]"


,product_code,n_reviews,quality,star_rating,size,comfort
0,FLC0310S2602A,NaN,NaN,NaN,NaN,NaN
1,FLC015044221A,NaN,NaN,NaN,NaN,NaN
2,FLC0H9026391B,NaN,NaN,NaN,NaN,NaN
3,FLC051044211A,NaN,NaN,NaN,NaN,NaN
4,FGC0AI024031B,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
177,FGC0DY0S0082A,NaN,NaN,NaN,NaN,NaN
178,FGC0L4033131A,NaN,NaN,NaN,NaN,NaN
179,FGC0L4037301A,NaN,NaN,NaN,NaN,NaN
180,FGC0Y9032792A,NaN,NaN,NaN,NaN,NaN


Now we will assign the product_code to Table 1.

In [722]:
# Establishing product code and n_colors
df_shoes_woodl['no_colors'] = df_proc['colors'].apply(len)
df_shoes_woodl['product_code'] = df_proc['product_code']

# Sets the column's order
df_shoes_woodl = df_shoes_woodl[['ShoeName', 'URL', 'no_colors', 'price_reduce', 'price', 'Category', 'Brand', 'product_code']]

# Save the Table1 including the product_code
df_shoes_woodl.to_csv('datasets/Table1_woodland.csv')

df_shoes_woodl

,ShoeName,URL,no_colors,price_reduce,price,Category,Brand,product_code
0,Brown/maroon Canvas shoes for women,https://www.woodlandworldwide.com/productinfo/...,2,2,53.94,Women,WoodLand,FLC0310S2602A
1,Lgrey sneaker for women,https://www.woodlandworldwide.com/productinfo/...,3,3,59.94,Women,WoodLand,FLC015044221A
2,Woodland DCadet Blue casual sneakers,https://www.woodlandworldwide.com/productinfo/...,2,2,59.94,Women,WoodLand,FLC0H9026391B
3,Variglia Sneakers for Women,https://www.woodlandworldwide.com/productinfo/...,3,3,53.94,Women,WoodLand,FLC051044211A
4,DBLUE casual sneakers,https://www.woodlandworldwide.com/productinfo/...,1,1,59.94,Women,WoodLand,FGC0AI024031B
...,...,...,...,...,...,...,...,...
177,Brown Sports Shoe for men,https://www.woodlandworldwide.com/productinfo/...,1,1,95.94,Men,WoodLand,FGC0DY0S0082A
178,PARIS KHAKI sneaker for men,https://www.woodlandworldwide.com/productinfo/...,3,3,59.94,Men,WoodLand,FGC0L4033131A
179,Dubai Khaki sneakers for men,https://www.woodlandworldwide.com/productinfo/...,4,4,44.34,Men,WoodLand,FGC0L4037301A
180,Navy sneakers for men,https://www.woodlandworldwide.com/productinfo/...,2,2,39.54,Men,WoodLand,FGC0Y9032792A


For now, we have gathered all the required data from WoodLand. Later on, we will reorganize the colors with the rest of the data recovered.

AQUI

### Processing the gathered data 

Now, we will join the gathered data from different sites in unified tables to save the solicited files: Table1.csv, Table2.csv, and Table2.csv.

### Table 1

In [ ]:
df_table1_nike = df_shoes_nike.copy()
df_table1_adidas = df_shoes_adidas.copy()
df_table1_puma = df_shoes_puma.copy()
df_table1_woodl = df_shoes_woodl.copy()

In [ ]:
# Inspect the partial tables 1
display(df_tale1_nike.head())
display(df_tale1_adidas.head())
display(df_tale1_puma.head())
display(df_tale1_woodl.head())

In [ ]:
df_table1 = pd.concat([df_table1_nike, df_table1_adidas, df_table1_puma, df_table1_woodl])

# adjusting the indexes
df_table1.reset_index(drop=True, inplace=True)

# Save a file with the Table1 with aditional data like Brand and product_code 
df_table1 = df_table1[['ShoeName', 'Category', 'no_colors', 'price_reduce', 'price', 'Brand', 'product_code']]
df_table1.to_csv('datasets_final/Table1_complete.csv')

# Save a file with Table1 only with the solicited fields.
df_table1_short = df_table1[['ShoeName', 'Category', 'no_colors', 'price_reduce', 'price']]
df_table1_short.to_csv('datasets_final/Table1.csv')

display(df_table1)
display(df_table1_short)

### Table 2

In [ ]:
df_table2_nike = pd.read_csv('datasets/Table2_nike.csv', index_col=0)
# df_table2_adidas = pd.read_csv('datasets/Table2_adidas.csv', index_col=0)
df_table2_puma = pd.read_csv('datasets/Table2_puma.csv', index_col=0)
df_table2_woodl = pd.read_csv('datasets/Table2_woodland.csv', index_col=0)

In [ ]:
# Inspect the partial tables 2
display(df_table2_nike)
# df_table2_adidas 
display(df_table2_puma)
display(df_table2_woodl)

Since there is a variety of colors we will select the following most common colors: black, blue, white, red, and as a 5th color a category for different colors. For that purpose, the function process_colors was defined to establish 5 color columns as the requirements of the task.

In [ ]:
def process_colors(row):
    ''' Classifies the colors in the row in black, blue, white, red and other_color(multi_color)    
    '''

    # Remove None from the row
    row = [color for color in row if color is not None]
    
    # If row is empty, return zeros for all color categories
    if not row:
        return pd.Series({'color_black': 0,
                          'color_blue': 0,
                          'color_white': 0,
                          'color_red': 0,
                          'other_color': None})

    row_text = " / ".join(row).lower()

    color_black = any(color in row_text for color in ['black', 'carbon', 'charcoal'])
    color_blue = any(color in row_text for color in ['navy', 'blue', 'indigo', 'azul'])
    color_white = any(color in row_text for color in ['white', 'bone', 'grey', 'silver', 'beige'])
    color_red = any(color in row_text for color in ['scarlet', 'red', 'burgundy',])
    
    # Get the first color in the other_color list
    other_color = next((color for color in [
                                            'yelow',
                                            'brown',
                                            'purple',
                                            'green',
                                            'gold', 
                                            'pink', 
                                            'orange',
                                            'rose',
                                            'violet', 
                                            'orchid',
                                            'lilac',
                                            'maroon', 
                                            'olive',
                                            'lime',
                                            'camel',
                                            'khaki'
                                           ] if color in row_text), None)
    
    return pd.Series({'color_black': int(color_black),
                      'color_blue': int(color_blue),
                      'color_white': int(color_white),
                      'color_red': int(color_red),
                      'other_color': other_color})

In [ ]:
df_table2 = pd.concat([df_table2_nike, 
#                        df_table2_adidas, 
                       df_table2_puma, 
                       df_table2_woodl])

# adjusting the indexes
df_table2.reset_index(drop=True, inplace=True)

# Reestructure column colors  
df_table2['colors'] = df_table2['colors'].apply(lambda x: ast.literal_eval(x))

# Process colors
new_columns = df_table2['colors'].apply(process_colors)
df_table2 = pd.concat([df_table2, new_columns], axis=1)

# Drop not needed columns
df_table2.drop(['colors'], axis=1, inplace=True)

# Sets the order of the columns
df_table2 = df_table2[['count_of_sizes', 'color_black', 'color_blue',
       'color_white', 'color_red', 'other_color', 'product_code']]

# Save a file with the final Table2
df_table2.to_csv('datasets_final/Table2.csv')

display(df_table2)

### Table 3

In [ ]:
df_table3_nike = pd.read_csv('datasets/Table3_nike.csv', index_col=0)
# df_table3_adidas = pd.read_csv('datasets/Table3_adidas.csv', index_col=0)
df_table3_puma = pd.read_csv('datasets/Table3_puma.csv', index_col=0)
df_table3_woodl = pd.read_csv('datasets/Table3_woodland.csv', index_col=0)

In [ ]:
# Correcting column' names
df_table3_nike.columns = ['product_code', 'n_reviews', 'quality', 'star_rating', 'size', 'comfort']

display(df_table3_nike.head())
# display(df_table3_adidas.head())
display(df_table3_puma.head())
display(df_table3_woodl.head())

In [ ]:
df_table3 = pd.concat([df_table3_nike, 
#                        df_table3_adidas, 
                       df_table3_puma, 
                       df_table3_woodl])

# adjusting the indexes
df_table3.reset_index(drop=True, inplace=True)

# Sets the order of the columns
df_table3 = df_table3[['n_reviews','size', 'comfort', 'quality', 'star_rating', 'product_code']]

# Save a file with the final Table3
df_table3.to_csv('datasets_final/Table3.csv')

display(df_table3)


### Custom Functions

In [13]:
def get_attribute_el(webelement, attribute):
    '''Wildcard function to retrieve an attribute other than text from a web-element.
    '''
    return (webelement.get_attribute(attribute))

def text_el(webelement):
    return webelement.text

# Defining lambda functions to retrieved element
fn_href = lambda web: get_attribute_el(web,'href')
fn_text = lambda web: text_el(web)

In [305]:
def add_category(category, element):
    '''Check if <category> is in <element>, if not, adds it.'''
    if category not in element.upper():
        return f"{category} {element}"
    else:
        return element

In [649]:
def search_with_pattern(txt, pattern):
    match = re.search(pattern, txt)
    return (match.group(1) if match else '')

In [14]:
def remove_popup(xpath):
    ''' Checks for an element corresponding to <xpath> and clicks it. 
        Also, prints the corresponding message according to the result.
        '''
    # Check if an element on the new page is present
    try:
        close_button  = driver.find_element(By.XPATH, xpath)
        close_button.click() 
        print("Pop-up removed")
    except NoSuchElementException:
        print("No pop-up windows was found")
    except ElementNotInteractableException as e:
        print(f"Pop-up windows not Interactable: {e.msg}")

In [304]:
def go_next_page(driver, xpath):
    ''' Go to the next page
    
    ATTIBUTES
    ---------
    driver: webdriver
    xpath: string, the next page button xpath
    '''
        
    next_btn = driver.find_element(By.XPATH, xpath)
    print(f"Going to {next_btn.text}")
    
    # scroll until button be visible
    actions = ActionChains(driver)
    actions.move_to_element(next_btn).perform()

    # Wait until the button be visible (loading compleated)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))

    # Click the button once it is clickable
    next_btn.click()


In [15]:
def n_shoes_to_scrap(xpath, pattern = r'(\d+)' ):
    ''' Retrieve the total number of shoes in the category ''' 
    reg = driver.find_element(By.XPATH, xpath)

    # Remove the parentheses
    # pattern = r'\((\d+)\)' 
    match = re.search(pattern, reg.text)
    
    if match:
        nreg = int(match.group(1))
    else:
        nreg = 0

    return nreg

In [303]:
def get_number_of_pages(driver, xpath, pattern):
    ''' Retrieve the total number of products from the search and 
        the number of products on the retrieved page
    '''
    try:
        element = driver.find_element(By.XPATH, xpath)
    except NoSuchElementException as e:
        print("The number of pages was not found.")
        return 0
        
    if pattern:
        matches = re.search(pattern, element.text)
        pages = int(matches.group(0)) if matches else 0
        print(f"Number of pages: {pages}")
        
    return pages

In [16]:
def follow_path_to(config_path):
    ''' Follows every xpath sended in path to retrieve the data.
    Atributes
    ---------
    path: Dict with properties 'xpaths' and 'msg'. With the configurations to follow.
    '''
    actions = ActionChains(driver)
    # Path towards the data 
    for i, xpath_link in enumerate(config_path['xpaths']):
        
        # Waiting for the data
        if 'wait_time' in config_path:
            if len(config_path['wait_time']) > 1:
                wait_time = config_path['wait_time'][i]
            else:
                wait_time = config_path['wait_time'][0]
        else:
            wait_time = 5
            
        print(f"Wait time ** {wait_time} **")
        driver.implicitly_wait(wait_time)

        try:
#             actions.move_by_offset(100, 100).perform()
            link = driver.find_element(By.XPATH, xpath_link)
            actions.move_to_element(link).perform()
            
            link.click()
            print(config_path['msg'][i])
        except NoSuchElementException as e:
            print(f"The link can not be found: {e.msg}")
        except ElementClickInterceptedException as e:
            print(f"{e.msg}")    
        except Exception as e:
            print(f"{e.msg}")
        

In [178]:
def smooth_scroll(driver, scrolls, scroll_speed=0.5, scroll_break=None):
    for i in range(scrolls):
        # Calculates the scroll position based on progress
        progress = i / scrolls
        scroll_y = int(driver.execute_script(f"return Math.max(0, Math.min(1, arguments[0])) * document.body.scrollHeight;", progress))
        
        # If I want to avoid the last scrolls
        if scroll_break and i==scroll_break:
            print(f"Scrolled {i + 1} times.")
            break
        
        # Soft scroll with JavaScript
        driver.execute_script(f"window.scrollTo(0, {scroll_y});")

        # Wait time
        time.sleep(scroll_speed)

In [668]:
def scroll_to_recovered_shoes(n_shoes, xpath):
    '''Scroll the page until almost <n_shoes> will be recovered.
    PARAMS
    -------
    n_shoes: int, number of shoes to recover in the page
    xpath: string, XPath to recover the records
    
    RETURN
    -------
    recovered_shoes: int, total of shoes recovered
    n_shoes_: int, number of shoes to recovered (reference)
    '''
    # Retrieve the total shoes in the page
    tags = driver.find_elements(By.XPATH, xpath )  # scraping available shoes
    recovered_shoes = len(tags)
    print(f"Recovered shoes: {recovered_shoes}")
    j = 1
    
    # Checks if the data are loaded
    if recovered_shoes == n_shoes:
        # If the numbers of records match then we will scroll to be sure the data is loaded
        smooth_scroll(driver, scrolls=15, scroll_speed=10, scroll_break=14)
        return recovered_shoes, n_shoes

    # Scrolling to load all shoes in the category
    wait = WebDriverWait(driver,15)
    total = (n_shoes // 10) * 10

    while( total > recovered_shoes ):

        print(f"Scrolling {j} times...")
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(10)

        tags = driver.find_elements(By.XPATH, xpath )  # scraping available shoes
        recovered_shoes = len(tags)
        print(f"Recovered shoes: {recovered_shoes}, still to recovered: {n_shoes - recovered_shoes}")
        j = j + 1

    return recovered_shoes, n_shoes

In [18]:
def recover_det_raw_data(file_name, err_file_name):
    # Recovers det_raw data of nike
    df_data = pd.read_csv(file_name, index_col=0)
    print(f"Data Recovered: {df_data.shape} from {file_name}")
    
    # Recovers log of errors
    df_error = pd.read_csv(err_file_name, index_col=0)
    
    # Reestructure df_det to join with the new batch of data
    for col in df_data.columns:
        df_data[col] = df_data[col].apply(lambda x: ast.literal_eval(x))
    
    return df_data, df_error

In [19]:
def load_items_by_xpath(driver, n=None, **kwargs):
    ''' Load items on the page with the attributes sent in kwargs.
        n: number of products required
        kwargs: dictionary with key, value = label, XPATH to retrieve 
        Return: a DataFrame with the info of the products
    '''
    data_retrieved = {}
    lenght_ref = -1
    
    for key, value in kwargs.items():
        
        tags = driver.find_elements(By.XPATH, value[0] )  # scraping

        if len(value)>1:
            items = [ value[1](tag) for tag in tags] # If lambda is specified it applies
        else:
            items = [ tag.text for tag in tags]  # Default: text function
            
        if(lenght_ref <0):
            lenght_ref = len(items)
        
        if(lenght_ref == len(items)):
            data_retrieved[key] = items
            print(f"{key} retrieved and included:", len(items))
        else:
            data_retrieved[key] = [None for _ in range(lenght_ref)]
            print(f"{key} retrieved NOT COMPATIBLE:", len(items))

    df = pd.DataFrame(data_retrieved)
        
    if ( bool(n) and (n < len(df))):
        return (df.iloc[0:n,:])
    
    return df

In [306]:
def scrape_items_by_group(driver, xpath_grp, **kwargs):
    '''Retrieves the data by group, in case any attribute is not 
       defined it assigns the not_found_value for that specific item. For this purpouse we use Selenium.  
       
       ATRIBUTES
       ---------
       xpath_grp : string, XPATH to retrieve the group
       kwargs : dict, attributes to find in each group the corresponding element. 
                Since we use BeautifulSoup we need to specify: the tag name, the particular 
                attributes, the function to retrieve the data, and the not_found_value.
       
               e.g.'URL': ['a', {'data-test-id': 'product-list-item-link'}, 'href', '-']
               
       RETURN
       -------
       a data frame with the data recovered.
       '''
    # Scrapes products group-tags using Selenium
    try:
        elements = driver.find_elements(By.XPATH, xpath_grp)
        print(f"Recovered group-tags: {len(elements)}")
    except NoSuchElementException as e:
        print(f"No elements corresponding to {xpath_grp}: {e.msg}")

    # Convert the elements to HTML strings
    html_strings = [el.get_attribute("outerHTML") for el in elements]
    print(f"Records converted to HTML strings: {len(html_strings)}")  

    # Scraping data using BeautifulSoup for better performance

    # Initialization of the result dataframe
    df = pd.DataFrame()

    for html_string in html_strings:
        soup = BeautifulSoup(html_string, 'html.parser')

        # Initialization of the new row
        row = {}
        for key, config in kwargs.items():
            tag, attr_dict, fn, condition, not_found_value = config
            
            # Apply a filter if <condition> is defined
            if condition is not None and callable(condition):
                # Recover all elements corresponding to tag and attrs
                elements = soup.find_all(tag, attrs)
                # Apply condition to filter
                elements = [el for el in elements if condition(el)]
                element = elements[0] if len(elements)>0 else None
            else:
                # Recover element corresponding to attributes defining in kwargs
                element = soup.find(tag, attr_dict)
          
            if fn is not None and callable(fn):
                # If a <fn> function is provided, apply it to elements
                row[key] = (fn(element) if element!=None else not_found_value)
            elif fn is not None: 
                # If a prop is provided
                row[key] = element.get(fn, not_found_value) if element!=None else not_found_value
            else:
                # Return text value by default
                row[key] = (element.text.strip() if element!=None else not_found_value)

        df = df.append(row, ignore_index=True)

    return df

In [498]:
def bs_scrape_item_detail(html_string, **kwargs):
    ''' Scrapes the data specified in the Kwargs arguments using Beautiful Soup.
        ATRIBUTES
       ---------
       html_string : string, html string
       kwargs : dict, attributes to find in each group the corresponding element. 
                Since we use BeautifulSoup we need to specify: the tag name, the particular 
                attributes, the function to retrieve the data, and the not_found_value.

               e.g.'URL': ['a', {'data-test-id': 'product-list-item-link'}, 'href', 
                               lambda el: el.text.strip() and 'Shown:' in el.text.strip(),'-']
                          [tag, attrs, fn, condition, not_found_value]
       RETURN
       -------
       a dict with the data recovered.
    '''
    soup = BeautifulSoup(html_string, 'html.parser')

    # Initialization of the new row
    row = {}
    for key, config in kwargs.items():
        tag, attrs, fn, condition, not_found_value = config

        # Recover element corresponding to attributes defining in kwargs
        elements = soup.find_all(tag, attrs)

        # Apply a filter if <condition> is defined
        if condition is not None and callable(condition):
            elements = [el for el in elements if condition(el)]
        
        if fn is not None and callable(fn):
            # If a <fn> function is provided, apply it to elements
            row[key] = [fn(el) for el in elements]
        elif fn is not None: 
            # If a prop is provided
            row[key] = [el.get(fn, not_found_value) for el in elements]
        else:
            # Return text value by default
            row[key] = [el.text.strip() if el is not None else not_found_value for el in elements]
            
    return row

In [394]:
def scrape_details_by_URL_in_batches(df_urls, soup_config, **kwargs):

    # Open the Driver 
    driver = webdriver.Chrome()
    driver.maximize_window()

    # Initializing the dataframe
    df_det = pd.DataFrame()
    total_urls = df_urls.shape[0]
    url_error = pd.DataFrame(columns=['index','URL','Exception','msg' ])

    for i, url in enumerate(df_urls['URL']):
        try:
            driver.get(url) 
            print(f"Retrieving Details of record: {i+1}/{total_urls}")       
                    
            wait_time = kwargs['wait_time'] if 'wait_time' in kwargs else 3
            time.sleep(wait_time)

            # 1.Removing cookie pop-up if first iteration and specified
            if 'xpath_remove_popup' in kwargs and i == 0:
                remove_popup(kwargs['xpath_remove_popup'])

            # 2.Closing the Market Selection pop-up
            if 'xpath_market_esc' in kwargs:
                try:
                    button = driver.find_element(By.XPATH, kwargs['xpath_market_esc'])
                    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # Sending a ESC
                except NoSuchElementException as e:
                    print(f"Selecting the market was not necessary.")

            # 3.Break the scroll into smaller steps 
            if 'n_scrolls' in kwargs: 
                for _ in range(kwargs['n_scrolls']):
                    height = kwargs['height_scroll'] if 'height_scroll' in kwargs else 400
                    driver.execute_script(f"window.scrollBy(0, {height});")
                    time.sleep(3)

            print(f"Wait time ** {wait_time} ** ")
            time.sleep(wait_time)

            # 4.Show Reviews
            if 'xpath_show_reviews' in kwargs:
                try:
                    link = driver.find_element(By.XPATH, kwargs['xpath_show_reviews'])
                    link.click()
                    print(f'Show reviews of {i + 1}')
                except NoSuchElementException as e:
                    print(f"Item {df_urls.iloc[i]['ShoeName']} has not reviews.")
                except Exception as e:
                    print(f"Review Exception of {df_urls.index[i]}-{str(e)}")
                    row_error = {'index': df_urls.index[i],
                                 'URL': url,
                                 'Exception': type(e),
                                 'msg': e.msg,} 
                    url_error = url_error.append(row_error, ignore_index=True)
                time.sleep(wait_time)

            # 5.Show more-reviews/Details
            if 'xpath_more_reviews' in kwargs:
                try:
                    link = driver.find_element(By.XPATH, kwargs['xpath_more_reviews'])
                    link.click()
                    print(f'Click more reviews of {i + 1}')
                except NoSuchElementException as e:
                    print(f"Item {df_urls.iloc[i]['ShoeName']} has not reviews.")
                except Exception as e:
                    print(f"More-Review Exception of {df_urls.index[i]}")
                    row_error = {'index': df_urls.index[i],
                                 'URL': url,
                                 'Exception': type(e),
                                 'msg': e.msg,} 
                    url_error = url_error.append(row_error, ignore_index=True)
                time.sleep(wait_time)
                
            # 6.Check for '//div[@data-auto-id="size-selector"]'
            if 'xpath_wait_for' in kwargs:
                for xpath in kwargs['xpath_wait_for']:
                    try:
                        wait = WebDriverWait(driver, 10)
                        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)),
                                            EC.element_to_be_clickable((By.XPATH, xpath)))
                        print(f"Elelment loaded: {element.get_attribute('data-auto-id')}")
                    except Exception as e:
                        print(f"Wait for Exception of {df_urls.index[i]}, xpath = {xpath}")
                        row_error = {'index': df_urls.index[i],
                                     'URL': url,
                                     'Exception': type(e),
                                     'msg': e.msg,} 
                        url_error = url_error.append(row_error, ignore_index=True)
            
            # 7.Reload sizes
            if 'xpath_reload_sizes' in kwargs:
                try:
                    link = driver.find_element(By.XPATH, kwargs['xpath_reload_sizes'])
                    link.click()
                    print(f'Reload sizes of {i + 1}')
                except NoSuchElementException as e:
                    print(f"Item {df_urls.iloc[i]['ShoeName']} sizes do not need to be reload.")
                except Exception as e:
                    print(f"Reload sizes Exception of {df_urls.index[i]}")
                    row_error = {'index': df_urls.index[i],
                                 'URL': url,
                                 'Exception': type(e),
                                 'msg': str(e),} 
                    url_error = url_error.append(row_error, ignore_index=True)
                time.sleep(wait_time)
            
            # Getting the HTML code
            html_string = driver.page_source   
            print(f"Get the HTML code: {url}\n")  

            # Scrape details with Beautiful Soup
            row = bs_scrape_item_detail(html_string, **soup_config)
            df_det = df_det.append(row, ignore_index=True)
        
        # Error to get the product
        except Exception as e:
            print(f"Error to get {url}\n{e.msg}")
            row_error = {'index': df_urls.index[i],
                         'URL': url,
                         'Exception': type(e),
                         'msg': e.msg,} 
            url_error = url_error.append(row_error, ignore_index=True)
            return [df_det, url_error] 
        
    # Free driver
    driver.quit()
    
    return [df_det, url_error]